In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import scipy.stats as st
import cufflinks as cf
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
init_notebook_mode(connected = True)
cf.go_offline()
import chart_studio.plotly as py
import plotly.graph_objs as go 
import datetime as dt
import dataframe_image as dfi
import Levenshtein as lev
import re

In [ ]:
df = pd.read_csv("GCST.csv", low_memory = False)

In [ ]:
# PW_CODE CLEAN UP
df["PW_SOC_CODE"] = df["PW_SOC_CODE"].astype(str)
# PW_SOC_CODES that do not follow the standard XX-XXXX, missing "-"
wrongsoc = df[~df["PW_SOC_CODE"].str.contains("-", na = False)]
# DROPPING ANY APPLICANTS THAT DO NOT HAVE "-" IN PW SOC CODE
df = df[df["PW_SOC_CODE"].str.contains("-", na = False)]

In [ ]:
# RESET DF INDEX
df = df.reset_index(drop = True)

In [ ]:
# SPLITTING PW_SOC_CODE FOR CLEANING DECIMALS
# COL1 will be pre "-", COL2 will be after "-"... DROP THE DECIMALS (OLDER FORMAT, NOW MORE CONSOLIDATED)
symbol = df["PW_SOC_CODE"].str.find("-")
list1 = []
for x in range(len(df)):
    j = df["PW_SOC_CODE"][x][:symbol[x]]
    list1.append(j)
list2 = []
for x in range(len(df)):
    j = df["PW_SOC_CODE"][x][symbol[x]:]
    list2.append(j)

In [ ]:
# COL1: HAS SOME WEIRD FORMATTING FROM CSV... TURNED IT INTO DATES... NOVEMBER IS IN THERE, SHOULD BE 11
col1 = pd.DataFrame(list1)
#col1[col1[0].str.len()>2][0].unique()
col1[0] = col1[0].replace("Nov",11)

In [ ]:
(col1[0]=="Nov").sum()

In [ ]:
# DROPPING ANYTHING AFTER DECIMAL FOR COL2
list2 = [sub.replace('-', '') for sub in list2]
col2 = pd.DataFrame(list2)
decimal = col2[col2[0].str.find(".")>0][0].str[:-3]
col2[0].loc[decimal.index] = decimal
col2[0] = col2[0].astype(str)

In [ ]:
# IF COL1 HAS NOV THAN, COL2 SHOULD BE THE YEAR in the 1900s
col2[0] = np.where(col1[0]=="Nov","19"+col2[0],col2[0])

In [ ]:
# IF COL2 HAS NOV, THEN IT SHOULD BE JUST THE 2 DIGITS FROM COL1 AS COL2... FLIP THE TOP LOGIC
p1 = np.where(col2[0]=="Nov",col1[0],col2[0])
p2 = np.where(col2[0]=="Nov",11,col1[0])
col2[0] = p1
col1[0] = p2

In [ ]:
# TURNING 2 DIGIT Col2 and adding 00 at the end
col2[0] = np.where(col2[0].str.len()<4,col2[0]+"00",col2[0])

In [ ]:
# PUTTING BACK TOGETHER PW CODE CODE
df["PW_SOC_CODE"] = col1.astype(str)+"-"+col2.astype(str)

In [ ]:
# RESET INDEX
df = df.reset_index(drop = True)

In [ ]:
# CHANGING DECISION DATE AND APPLICATION RECEIVE DATE INTO MONTH OF YEAR 
df["MONTH"] = pd.to_datetime(df["DECISION_DATE"]).apply(lambda x:x.month)
df["YEAR"] = pd.to_datetime(df["DECISION_DATE"]).apply(lambda x:x.year)

In [ ]:
# changing certified and certified expired to 1. and withdrawn and denied to 0: CLASSIFICATION MODEL OUTPUT
df["CASE_STATUS"] = df["CASE_STATUS"].replace(["CERTIFIED-EXPIRED","CERTIFIED"],[1,1]).replace(["WITHDRAWN","DENIED"],[0,0])

In [ ]:
# HOW MANY WAGE OFFERED AND PW_WAGE ARE NULL? HOW MANY UNITS ARE NULL?
print("Number of PW Wage null:",len(df[df["PW_WAGE"].isnull()]))
print("Number of Wage offered null:",len(df[df["WAGE_OFFER_FROM"].isnull()]))
print("Number of PW Wage Unit Null:",len(df[df["PW_UNIT_OF_PAY"].isnull()]))
print("Number of Wage Unit null:",len(df[df["WAGE_OFFER_UNIT_OF_PAY"].isnull()]))

In [ ]:
# DF COPY TO SEE HOW MANY WERE DROPPED
dfc = df

In [ ]:
# DROP ALL APPLICANTS WITH NO WAGE UNIT
df = df[df["WAGE_OFFER_UNIT_OF_PAY"].notnull()]
# DROP ALL APPLICANTS WITH NO PW UNIT
df = df[df["PW_UNIT_OF_PAY"].notnull()]
# DROP ALL APPLICANTS WITH NO WAGE OFFERED
df = df[df["WAGE_OFFER_FROM"].notnull()]
# DROP ALL APPLICANTS WITH NO PW WAGE
df = df[df["PW_WAGE"].notnull()]

In [ ]:
# HOW MANY DROPPED DUE TO NULL
len(dfc)-len(df)

In [ ]:
# DF COPY TO SEE HOW MANY WERE DROPPED
dfc = df

In [ ]:
# TURNING ALL WAGES TO FLOATS
df["WAGE_OFFER_FROM"] = df["WAGE_OFFER_FROM"].str.replace(",","")
df = df[df["WAGE_OFFER_FROM"]!="#############"]
df["PW_WAGE"] = df["PW_WAGE"].str.replace(",","")
df = df[df["PW_WAGE"]!="#############"]
df["WAGE_OFFER_FROM"] = df["WAGE_OFFER_FROM"].astype(float)
df["PW_WAGE"] = df["PW_WAGE"].astype(float)

In [ ]:
# HOW MANY DROPPED DUE TO #######?
len(dfc)-len(df)

In [ ]:
# STANDARD THE PW and SALARY UNIT OF PAY
df["PW_UNIT_OF_PAY"] = df["PW_UNIT_OF_PAY"].replace(["HOUR","YEAR","WEEK","MONTH","BI-WEEKLY"],["HR","YR","WK","MTH","BI"])
df["WAGE_OFFER_UNIT_OF_PAY"] = df["WAGE_OFFER_UNIT_OF_PAY"].replace(["HOUR","YEAR","WEEK","MONTH","BI-WEEKLY"],["HR","YR","WK","MTH","BI"])

In [ ]:
# DISTRIBUTION OF UNIT OF PAY
pd.DataFrame([df["WAGE_OFFER_UNIT_OF_PAY"].value_counts(),df["PW_UNIT_OF_PAY"].value_counts(),df["WAGE_OFFER_UNIT_OF_PAY"].value_counts()-df["PW_UNIT_OF_PAY"].value_counts()]).transpose()

In [ ]:
# ANNUALIZING WAGE OFFER AND PW
# Annualized Salaries: Actual Salary: multiplier
def annual(row):
    if row["WAGE_OFFER_UNIT_OF_PAY"] == "YR":
        return 1
    elif row["WAGE_OFFER_UNIT_OF_PAY"] == "MTH": 
        return 12
    elif row["WAGE_OFFER_UNIT_OF_PAY"] == "WK": 
        return 52
    elif row["WAGE_OFFER_UNIT_OF_PAY"] == "BI": 
        return 26
    elif row["WAGE_OFFER_UNIT_OF_PAY"] == "HR": 
        return 2080
mult = pd.DataFrame(df.apply(lambda row:annual(row), axis = 1))
mult.columns = ["WAGE_OFFER_FROM"]
# ANNUALIZED WAGE OFFER
df["WO_A"] = pd.DataFrame(df["WAGE_OFFER_FROM"])*mult

# Annualized Salaries: PW Salary
def annuali(row):
    if row["PW_UNIT_OF_PAY"] == "YR":
        return 1
    elif row["PW_UNIT_OF_PAY"] == "MTH": 
        return 12
    elif row["PW_UNIT_OF_PAY"] == "WK": 
        return 52
    elif row["PW_UNIT_OF_PAY"] == "BI": 
        return 26
    elif row["PW_UNIT_OF_PAY"] == "HR": 
        return 2080
multi = pd.DataFrame(df.apply(lambda row:annuali(row), axis = 1))
multi.columns = ["PW_WAGE"]
# ANNUALIZED WAGE PW
df["PW_A"] = pd.DataFrame(df["PW_WAGE"])*multi

In [ ]:
# ANNUALIZED WAGE OFFER - ANNUALIZED PW
df["WO-PW"] = df["WO_A"] - df["PW_A"]

In [ ]:
# STANDARDIZING THE NULLS
df = df.replace(np.nan,"", regex = True)

In [ ]:
# HAVE WAGE OFFER BEEN < ANNUALIZED PW? THIS SOULD NEVER HAPPEN...
lowwage = df[df["WO-PW"]<0][["CASE_STATUS","WAGE_OFFER_UNIT_OF_PAY","PW_UNIT_OF_PAY","WAGE_OFFER_FROM","PW_WAGE","WO_A","PW_A"]]
lowwage

In [ ]:
# HOW MANY HAVE SUCCESSFUL APPLICATIONS WHEN WAGE < PW?
print("Number of Successful applicants:",lowwage["CASE_STATUS"].sum())
print("Out of:",len(lowwage),",",lowwage["CASE_STATUS"].sum()/len(lowwage))

In [ ]:
# SUCESSFULL APPLICATIONS WITH THE SAME UNIT
lowwage[lowwage["WAGE_OFFER_UNIT_OF_PAY"]==lowwage["PW_UNIT_OF_PAY"]]

In [ ]:
# CREATE A UNIQUE IDENTIFIER: THIS WILL BE TO UPDATE ANY OUTLIERS
df["ID"] = df["CASE_NUMBER"].astype(str)+df["DECISION_DATE"].astype(str)+df["CASE_STATUS"].astype(str)+df["RECEIVED_DATE"].astype(str)+df["DAYS_TO_DECIDE"].astype(str)+df["EMPLOYER_NAME"].astype(str)

In [ ]:
# PATTERN FOR MIS-LABELED: UNIT OF PAY IS NOT YEAR BUT THE WAGE OFFER IS YEAR AMOUNT
# FIND THIS BY UNIT OF PAY NOT YEAR, BUT WAGE OFFER > btm 5% OF WAGE OFFER of each YEAR
top = df[df["WAGE_OFFER_UNIT_OF_PAY"]=="YR"].groupby("YEAR")["WAGE_OFFER_FROM"].quantile(.05) 
top

In [ ]:
# CREATING THE LIST FOR THE FOR LOOP TO REPLACE IF ANNUALIZED SALARY IS > OUTLIER CUTOFF
year = top.index.tolist()
top = top.tolist()

In [ ]:
# GRABBING THE APPLICANTS THAT ARE NON YR BUT > btm 5% of YEAR WAGE
todrop = pd.DataFrame()
for i in range(len(year)):
    j = df[(df["YEAR"]==year[i])&(df["WAGE_OFFER_FROM"]>=top[i])&(df["WAGE_OFFER_UNIT_OF_PAY"]!="YR")]
    todrop = pd.concat([todrop,j])
todrop[["CASE_STATUS","WAGE_OFFER_UNIT_OF_PAY","PW_UNIT_OF_PAY","WAGE_OFFER_FROM","PW_WAGE","WO_A","PW_A"]]

In [ ]:
# CHANGING THE WO_A TO THE WAGE OFFER (LEAVE UNIT OF PAY AS WHAT THE APPLICATION SAYS, ONLY CHANGE MY* Annualized WAGES)
todrop["WO_A"]=todrop["WAGE_OFFER_FROM"]

In [ ]:
# CHECK IF UNIQUE IDENTIFIER IS UNIQUE
if len(df) == df["ID"].nunique():
    print("ID IS A UNIQUE IDENTIFIER")
else:
    print("ID IS NOT A UNIQUE IDENTIFIER")

In [ ]:
# WHAT IS THE MAX REPEAT OF IDs
df["ID"].value_counts()[df["ID"].value_counts()>1].max()

In [ ]:
# ISOLATE THE APPLICATIONS WITH THE SAME ID
idcheck = df["ID"].value_counts()[df["ID"].value_counts()>1].index.tolist()

In [ ]:
# CHECK TO SEE IF EACH DUPLICATE ID HAS IDENTICAL FIELDS (WOULD MEAN A MISTAKE AND THEN WE DROP). IF 0 THEN ALL IDENTICAL
empty = []
for i in range(len(idcheck)):
    df[df["ID"] == idcheck[i]]
    j = df[df["ID"] == idcheck[1]]
    if (j.iloc[0] == j.iloc[1]).sum() == len(j.columns):
        y = 0
        empty.append(y)
    else:
        y=[1]
sum(empty)

In [ ]:
# DROP THE REPEAT APPLICATION FROM THE DF
df = df.drop_duplicates(subset=["ID"])

In [ ]:
# CHANGING THE EXISTING APPLICANTS WITH UPDATED WAGE and UNIT OF PAY IN THE ORIGINAL DF
# MATCH APPLICATIONS IN TODROP DF WITH ORIGINAL DF AND UPDATE WAGE DATA
todrop = todrop.set_index("ID")
df = df.set_index("ID")
df.loc[todrop.index] = todrop
df = df.reset_index()

In [ ]:
# PATTERN FOR MIS-LABELED: UNIT OF PAY IS NOT YEAR BUT THE PW WAGE IS YEAR AMOUNT
# FIND THIS BY UNIT OF PAY NOT YEAR, BUT PW WAGE > btm 5% OF PW WAGE of each YEAR
top1 = df[df["PW_UNIT_OF_PAY"]=="YR"].groupby("YEAR")["PW_WAGE"].quantile(.05) 
top1

In [ ]:
# CREATING THE LIST FOR THE FOR LOOP TO REPLACE IF ANNUALIZED SALARY IS > OUTLIER CUTOFF
year1 = top1.index.tolist()
top1 = top1.tolist()

In [ ]:
# GRABBING THE APPLICANTS THAT ARE OVER NON YR BUT > btm 5% of PW_WAGE
todrop1 = pd.DataFrame()
for i in range(len(year)):
    j = df[(df["YEAR"]==year1[i])&(df["PW_WAGE"]>=top1[i])&(df["PW_UNIT_OF_PAY"]!="YR")]
    todrop1 = pd.concat([todrop1,j])
todrop1[["CASE_STATUS","WAGE_OFFER_UNIT_OF_PAY","PW_UNIT_OF_PAY","WAGE_OFFER_FROM","PW_WAGE","WO_A","PW_A"]]

In [ ]:
# CHANGING THE PW_A TO THE WAGE OFFER AGAIN LEAVING THE UNIT LIKE WO_A
todrop1["PW_A"]=todrop1["PW_WAGE"]

In [ ]:
# CHANGING THE EXISTING APPLICANTS WITH UPDATED WAGE and UNIT OF PAY IN THE ORIGINAL DF
# MATCH APPLICATIONS IN TODROP DF WITH ORIGINAL DF AND UPDATE WAGE DATA
todrop1 = todrop1.set_index("ID")
df = df.set_index("ID")
df.loc[todrop1.index] = todrop1
df = df.reset_index()

In [ ]:
# CONTINUE CLEANING OUTLIERS IF PW_UNIT IS NOT THE SAME AS WAGE_UNIT BUT WAGE_OFFER AND PW_WAGE IS THE SAME & PW UNIT IS YR:
# UPDATE WO_A TO PW_A
clean = df[(df["WAGE_OFFER_FROM"]==df["PW_WAGE"])&(df["PW_UNIT_OF_PAY"]!=df["WAGE_OFFER_UNIT_OF_PAY"])&(df["PW_UNIT_OF_PAY"]=="YR")]
clean["WO_A"]=clean["PW_A"]
clean = clean.set_index("ID")
df = df.set_index("ID")
df.loc[clean.index] = clean
df = df.reset_index()

In [ ]:
# DROPPING THE BTM 1% FROM EACH YEAR
# SETTING UP THE FOR LOOP
wagebtm = df.groupby("YEAR")["WO_A"].quantile(.01)
pwbtm = df.groupby("YEAR")["PW_A"].quantile(.01)
yr = wagebtm.index.tolist()
wagebtm = wagebtm.tolist()
pwbtm = pwbtm.tolist()

In [ ]:
# FOR LOOP TO GET ALL APPLICANTS WITH WO_A < CUTOFF
emp = pd.DataFrame()
for x in range(len(yr)):
    j = df[(df["YEAR"] == yr[x])&(df["WO_A"]<wagebtm[x])]
    emp = pd.concat([emp,j])

In [ ]:
# HOW MANY ARE DROPPED FOR WO_A<CUTOFF?
len(emp)

In [ ]:
# DROPPING ALL APPLICANTS WO_A < CUTOFF
emp = emp["ID"].tolist()
df = df[~df['ID'].isin(emp)]

In [ ]:
# FOR LOOP TO GET ALL APPLICANTS WITH PW_A < CUTOFF
empt = pd.DataFrame()
for x in range(len(yr)):
    j = df[(df["YEAR"] == yr[x])&(df["PW_A"]<pwbtm[x])]
    empt = pd.concat([empt,j])

In [ ]:
# HOW MANY ARE DROPPED FOR PW_A < CUTOFF?
len(empt)

In [ ]:
# DROPPING ALL APPLICANTS PW < CUTOFF
empt = empt["ID"].tolist()
df = df[~df['ID'].isin(empt)]

In [ ]:
# CLEANING CITIES: EVENTUALLY WILL BE USED TO CREATE AVG WAGE TO REPLACE EXTREME OUTLIER WAGES
# CLEAN UP JOB CITY: distribution of the length of the cities
df["WORKSITE_CITY"].str.len().plot(kind = "hist")

In [ ]:
# TOO MANY TO MANUALLY CLEAN: BUT WE WILL TARGET CITIES UNDER 5 CHARECTERS TO SPELL OUT ANY ABR.
df[df["WORKSITE_CITY"].str.len()<5]["WORKSITE_CITY"].unique()

In [ ]:
# WHAT ARE THE TOP CITIES?
df["WORKSITE_CITY"].value_counts()[:15]

In [ ]:
# HOW MANY VARIATIONS OF THE TOP CITIES?
#df[df["WORKSITE_CITY"].str.contains("edison", case = False, na = False)]["WORKSITE_CITY"].value_counts()

In [ ]:
# TURNING ALL THE NYC VARATIONS INTO NEW YORK + SPELL OUT NEW YORK
df["WORKSITE_CITY"] =  df["WORKSITE_CITY"].replace("NY","NEW YORK").replace("NYC","NEW YORK")
df.loc[df["WORKSITE_CITY"].str.contains("new york", case = False, na = False), "WORKSITE_CITY"] = "NEW YORK"
# MERGING THE TOP 15 cities (COLLEGE STATION, PLANO DOES NOT NEED CLEANING)
df.loc[df["WORKSITE_CITY"].str.contains("seattle", case = False, na = False), "WORKSITE_CITY"] = "SEATTLE"
df.loc[df["WORKSITE_CITY"].str.contains("san francisco", case = False, na = False), "WORKSITE_CITY"] = "SAN FRANCISCO"
df.loc[df["WORKSITE_CITY"].str.contains("REDMOND", case = False, na = False), "WORKSITE_CITY"] = "REDMOND"
df.loc[df["WORKSITE_CITY"].str.contains("mountain view", case = False, na = False), "WORKSITE_CITY"] = "MOUNTAIN VIEW"
df.loc[df["WORKSITE_CITY"].str.contains("san jose", case = False, na = False), "WORKSITE_CITY"] = "SAN JOSE"
df.loc[df["WORKSITE_CITY"].str.contains("santa clara", case = False, na = False), "WORKSITE_CITY"] = "SANTA CLARA"
df.loc[df["WORKSITE_CITY"].str.contains("chicago", case = False, na = False), "WORKSITE_CITY"] = "CHICAGO"
df.loc[df["WORKSITE_CITY"].str.contains("houston", case = False, na = False), "WORKSITE_CITY"] = "HOUSTON"
df.loc[df["WORKSITE_CITY"].str.contains("sunnyvale", case = False, na = False), "WORKSITE_CITY"] = "SUNNYVALE"
df.loc[df["WORKSITE_CITY"].str.contains("austin/|austin,", case = False, na = False), "WORKSITE_CITY"] = "AUSTIN"
df.loc[df["WORKSITE_CITY"].str.contains("los angeles", case = False, na = False), "WORKSITE_CITY"] = "LOS ANGELES"
df.loc[df["WORKSITE_CITY"].str.contains("edison", case = False, na = False), "WORKSITE_CITY"] = "EDISON"

In [ ]:
# HOW MANY STATES ARE APPLICANTS TRYING TO WORK IN? SHOULD BE NO MORE THAN 60...
df["WORKSITE_STATE"].nunique()

In [ ]:
# CREATING THE MAIN LIST TO TRY AND CLEAN MOST STATES INTO ABBREVIATIONS
mainst= ["Alabama",
"Alaska",
"Arizona",
"Arkansas",
"California",
"Colorado",
"Connecticut",
"Delaware",
"Florida",
"Georgia",
"Hawaii",
"Idaho",
"Illinois",
"Indiana",
"Iowa",
"Kansas",
"Kentucky",
"Louisiana",
"Maine",
"Maryland",
"Massachusetts",
"Michigan",
"Minnesota",
"Mississippi",
"Missouri",
"Montana",
"Nebraska",
"Nevada",
"New Hampshire",
"New Jersey",
"New Mexico",
"New York",
"North Carolina",
"North Dakota",
"Ohio",
"Oklahoma",
"Oregon",
"Pennsylvania",
"Rhode Island",
"South Carolina",
"South Dakota",
"Tennessee",
"Texas",
"Utah",
"Vermont",
"Virginia",
"Washington",
"West Virginia",
"Wisconsin",
"Wyoming",
"District of Columbia"]

In [ ]:
# CREATING THE MAIN ABR TO TRY AND CLEAN MOST STATES INTO ABBREVIATIONS
mainabr = ["AL",
"AK",
"AZ",
"AR",
"CA",
"CO",
"CT",
"DE",
"FL",
"GA",
"HI",
"ID",
"IL",
"IN",
"IA",
"KS",
"KY",
"LA",
"ME",
"MD",
"MA",
"MI",
"MN",
"MS",
"MO",
"MT",
"NE",
"NV",
"NH",
"NJ",
"NM",
"NY",
"NC",
"ND",
"OH",
"OK",
"OR",
"PA",
"RI",
"SC",
"SD",
"TN",
"TX",
"UT",
"VT",
"VA",
"WA",
"WV",
"WI",
"WY",
"DC"]

In [ ]:
# ROUND 1 CLEANING STATES
for j in range(len(mainst)):
    df.loc[df["WORKSITE_STATE"].str.contains(mainst[j], case = False, na = False), "WORKSITE_STATE"] = mainabr[j]

In [ ]:
# HOW MANY STATES ARE THERE NOT ABR AFTER ROUND 1 CLEANING?
df["WORKSITE_STATE"].unique()[np.vectorize(len)(df["WORKSITE_STATE"].unique())>2]

In [ ]:
# CREATING THE SECONDARY LIST TO FINISH CLEANING STATES
secondst = ['PUERTO RICO', 'GUAM', 'NORTHERN MARIANA ISLANDS',
       'VIRGIN ISLANDS', 'MARSHALL ISLANDS',
       'FEDERATED STATES OF MICRONESIA']
secondabr = ["PR","GU","MP","VI","MH","FSM"]
# ROUND 2 CLEANING STATES
for j in range(len(secondst)):
    df.loc[df["WORKSITE_STATE"].str.contains(secondst[j], case = False, na = False), "WORKSITE_STATE"] = secondabr[j]

In [ ]:
# CREATING US GEOGRAPHIC REGION: STATE LIST
geostate = ["AK",
"AL",
"AR",
"AZ",
"CA",
"CO",
"CT",
"DC",
"DE",
"FL",
"GA",
"HI",
"IA",
"ID",
"IL",
"IN",
"KS",
"KY",
"LA",
"MA",
"MD",
"ME",
"MI",
"MN",
"MO",
"MS",
"MT",
"NC",
"ND",
"NE",
"NH",
"NJ",
"NM",
"NV",
"NY",
"OH",
"OK",
"OR",
"PA",
"RI",
"SC",
"SD",
"TN",
"TX",
"UT",
"VA",
"VT",
"WA",
"WI",
"WV",
"WY",
"PR",
"GU",
"MP",
"VI",
"MH",
"FSM"]

In [ ]:
# CREATING US GEOGRAPHIC REGION: REGION LIST
region= ["WEST",
"SOUTH",
"SOUTH",
"WEST",
"WEST",
"WEST",
"NORTHEAST",
"SOUTH",
"SOUTH",
"SOUTH",
"SOUTH",
"WEST",
"MIDWEST",
"WEST",
"MIDWEST",
"MIDWEST",
"MIDWEST",
"SOUTH",
"SOUTH",
"NORTHEAST",
"SOUTH",
"NORTHEAST",
"MIDWEST",
"MIDWEST",
"MIDWEST",
"SOUTH",
"WEST",
"SOUTH",
"MIDWEST",
"MIDWEST",
"NORTHEAST",
"NORTHEAST",
"WEST",
"WEST",
"NORTHEAST",
"MIDWEST",
"SOUTH",
"WEST",
"NORTHEAST",
"NORTHEAST",
"SOUTH",
"MIDWEST",
"SOUTH",
"SOUTH",
"WEST",
"SOUTH",
"NORTHEAST",
"WEST",
"MIDWEST",
"SOUTH",
"WEST",
"TERRITORY",
"TERRITORY",
"TERRITORY",
"TERRITORY",
"TERRITORY",
"TERRITORY"]

In [ ]:
# CREATING A WORKSITE REGION COLUMN
df["WORKSITE_REGION"]=df["WORKSITE_STATE"]
for j in range(len(geostate)):
    df.loc[df["WORKSITE_REGION"].str.contains(geostate[j], case = False, na = False), "WORKSITE_REGION"] = region[j]

In [ ]:
# UNIQUE REGIONS (CHECK IF ALL STATES WERE TRANSLATED CORRECTLY)
df["WORKSITE_REGION"].unique()

In [ ]:
# OUTLIER WO_A USING IQR (EXTREME OUTLIER PER YEAR) TOP SIDE
wagetop = df.groupby("YEAR")["WO_A"].quantile(.75) + (df.groupby("YEAR")["WO_A"].quantile(.75)-df.groupby("YEAR")["WO_A"].quantile(.25)) *3
pwtop = df.groupby("YEAR")["PW_A"].quantile(.75) + (df.groupby("YEAR")["PW_A"].quantile(.75)-df.groupby("YEAR")["PW_A"].quantile(.25)) *3

In [ ]:
# CREATING LISTS TO CLEAN UP WAGE BASED ON OUTLIERS : TOP END ONLY... DROP BOTTOM 1% (LAWS PROTECTING THE BTM END... MORE LIKELY TOP END IS ERROR)
yr = wagetop.index.tolist()
wagetop = wagetop.tolist()
pwtop = pwtop.tolist()

In [ ]:
# FOR LOOP TO GET ALL APPLICANTS WITH WO_A > CUTOFF: BIGWAGE
bigwage = pd.DataFrame()
for x in range(len(yr)):
    j = df[(df["YEAR"] == yr[x])&(df["WO_A"]>wagetop[x])]
    bigwage = pd.concat([bigwage,j])

In [ ]:
# GETTING THE REST OF THE APPLICANTS WHO ARE NOT > CUTOFF: REGWAGE
bwlist = bigwage["ID"].tolist()
regwage = df[~df['ID'].isin(bwlist)]

In [ ]:
# CREATING A UNIQUE IDENTIFIER TO GET AN AVG WAGE FOR EACH IDENTIFIER TO REPLACE OUTLIER SALARIES
regwage["BID"] = regwage["PW_SOC_CODE"].astype(str)+regwage["WORKSITE_STATE"].astype(str) + regwage["WORKSITE_CITY"].astype(str) + regwage["PW_SKILL_LEVEL"].astype(str)
# IS THIS TOO GRANULAR OF AN AVG? LETS SEE AVG COUNT FOR EACH UNIQUE IDENTIFIER
regwage.groupby("BID")["WO_A"].value_counts().mean()

In [ ]:
# NEW IDENTIFIER : USING STATE ONLY
regwage["BID"] = regwage["PW_SOC_CODE"].astype(str)+regwage["WORKSITE_STATE"].astype(str)+ regwage["PW_SKILL_LEVEL"].astype(str)
regwage.groupby("BID")["WO_A"].value_counts().mean()
# STILL ONLY 2.7  APPLICANTS A CODE

In [ ]:
# NEW IDENTIFIER : USING REGION ONLY
regwage["BID"] = regwage["PW_SOC_CODE"].astype(str)+regwage["WORKSITE_REGION"].astype(str)+ regwage["PW_SKILL_LEVEL"].astype(str)
regwage.groupby("BID")["WO_A"].value_counts().mean()
# BETTER with 3.3... 

In [ ]:
# NORMALIZED AVG WAGE FOR EACH UNIQUE IDENTIFIER
normwage = pd.DataFrame(regwage.groupby("BID")["WO_A"].mean())
normpw = pd.DataFrame(regwage.groupby("BID")["PW_A"].mean())
normwage.columns=["NORM_WAGE"]
normpw.columns = ["NORM_PW"]

In [ ]:
# CREATING THE UNIQUE IDENTIFIER FOR BIG WAGE TO MATCH UP WITH REG WAGE AVG
bigwage["BID"] = bigwage["PW_SOC_CODE"].astype(str)+bigwage["WORKSITE_REGION"].astype(str)+ bigwage["PW_SKILL_LEVEL"].astype(str)

In [ ]:
# MERGING BASED ON BID AND BRINGING IN NORM WAGE
matchbidwage = bigwage
matchbidwage = matchbidwage.merge(normwage, how = "inner", on = "BID")

In [ ]:
# HOW MANY APPLICANTS WERE NOT MATCHED THROUGH BID
len(bigwage) - len(matchbidwage)

In [ ]:
# LIST THE BIDs THAT DO NOT MATCH IN NORM WAGE: THESE ARE THE BIDs NOT IN THE NORM WAGE...
bwregion = bigwage["BID"].unique().tolist()
normwageregion = normwage.index.tolist()
list(set(bwregion) - set(normwageregion))

In [ ]:
# GETTING THE APPLICANTS THAT DID NOT MATCH IN BID for NORM WAGE
nonmatchwage = pd.DataFrame()
for x in range(len(list(set(bwregion) - set(normwageregion)))):
    j = bigwage[bigwage["BID"]==list(set(bwregion) - set(normwageregion))[x]]
    nonmatchwage = pd.concat([nonmatchwage, j],axis = 0)

In [ ]:
# DOES THE NUMBER OF APPLICANTS THAT DO NOT MATCH = TO 35?
len(nonmatchwage)
# YES

In [ ]:
# CORRELATION BETWEEN REGION OR LEVEL WITH WAGE: TO DETERMINE HOW TO ADJUST THE BID
regiondummy = pd.get_dummies(df["WORKSITE_REGION"])
leveldummy = pd.get_dummies(df["PW_SKILL_LEVEL"])

In [ ]:
# CORREL BETWEEN REGIONS AND WO_A
pd.concat([regiondummy,df["WO_A"]],axis = 1).corr()["WO_A"]

In [ ]:
# CORREL BETWEEN LEVEL AND WO_A
pd.concat([leveldummy,df["WO_A"]],axis = 1).corr()["WO_A"]
# LEVELS LOOK MUCH HIGHER OVERALL 

In [ ]:
# GETTING THE ADJUSTED BID FOR NORM WAGE: ONLY KEEPING LEVELS SINCE IT IS MORE CORREL WITH WAGE THAN REGION
adjregw = regwage
adjregw["BID"] = adjregw["PW_SOC_CODE"].astype(str)+adjregw["PW_SKILL_LEVEL"].astype(str)
# NORMALIZED AVG WAGE FOR EACH UNIQUE IDENTIFIER
adjnormwage = pd.DataFrame(adjregw.groupby("BID")["WO_A"].mean())
adjnormwage.columns=["NORM_WAGE"]

# CREATING THE UNIQUE IDENTIFIER FOR MISSING BIG WAGE TO MATCH UP WITH REG WAGE AVG
adjbigwage = nonmatchwage
adjbigwage["BID"] = adjbigwage["PW_SOC_CODE"].astype(str)+ adjbigwage["PW_SKILL_LEVEL"].astype(str)

# MERGING BASED ON BID AND BRINGING IN NORM WAGE
matchadjbidwage = adjbigwage
matchadjbidwage = matchadjbidwage.merge(adjnormwage, how = "inner", on = "BID")

# HOW MANY APPLICANTS WERE NOT MATCHED THROUGH AdjBID
len(adjbigwage) - len(matchadjbidwage)

In [ ]:
# LIST THE Adj BIDs THAT DO NOT MATCH IN NORM WAGE: THESE ARE THE Adj BIDs NOT IN THE NORM WAGE...
adjbwregion = adjbigwage["BID"].unique().tolist()
adjnormwageregion = adjnormwage.index.tolist()
list(set(adjbwregion) - set(adjnormwageregion))

In [ ]:
# LIST THE APPLICANTS THAT ARE STILL NOT MATCHED THROUGH BID...
nonmatchadjwage = pd.DataFrame()
for x in range(len(list(set(adjbwregion) - set(adjnormwageregion)))):
    j = adjbigwage[adjbigwage["BID"]==list(set(adjbwregion) - set(adjnormwageregion))[x]]
    nonmatchadjwage = pd.concat([nonmatchadjwage, j],axis = 0)

# DOES THE NUMBER OF APPLICANTS THAT DO NOT MATCH = TO 2?
len(nonmatchadjwage)
# YES

In [ ]:
# GETTING THE WAGE FOR SOC CODE ONLY FOR NORM WAGE
soconly = regwage
soconly["BID"] = soconly["PW_SOC_CODE"].astype(str)
# NORMALIZED AVG WAGE FOR EACH UNIQUE IDENTIFIER
socnormwage = pd.DataFrame(soconly.groupby("BID")["WO_A"].mean())
socnormwage.columns=["NORM_WAGE"]

# CREATING THE UNIQUE IDENTIFIER FOR MISSING BIG WAGE TO MATCH UP WITH REG WAGE AVG
socbigwage = nonmatchadjwage
socbigwage["BID"] = socbigwage["PW_SOC_CODE"].astype(str)

# MERGING BASED ON BID AND BRINGING IN NORM WAGE
matchsocbidwage = socbigwage
matchsocbidwage = matchsocbidwage.merge(socnormwage, how = "inner", on = "BID")

# HOW MANY APPLICANTS WERE NOT MATCHED THROUGH AdjBID
len(socbigwage) - len(matchsocbidwage)

In [ ]:
# MERGING BACK TO MAKE THE FINAL BIG WAGE : USING NORM WAGE...
bigwage = pd.concat([matchbidwage,
matchadjbidwage,
matchsocbidwage],axis = 0, ignore_index = True)
bigwage["WO_A"] = bigwage["NORM_WAGE"]
bigwage = bigwage.drop("NORM_WAGE", axis = 1)

In [ ]:
# CLEANING UP THE PW_A
# CREATING THE UNIQUE IDENTIFIER FOR BIG WAGE TO MATCH UP WITH REG WAGE AVG
bigwage["BID"] = bigwage["PW_SOC_CODE"].astype(str)+bigwage["WORKSITE_REGION"].astype(str)+ bigwage["PW_SKILL_LEVEL"].astype(str)

In [ ]:
# MERGING BASED ON BID AND BRINGING IN NORM PW
matchbidwage = bigwage
matchbidwage = matchbidwage.merge(normpw, how = "inner", on = "BID")

In [ ]:
# HOW MANY APPLICANTS WERE NOT MATCHED THROUGH BID
len(bigwage) - len(matchbidwage)

In [ ]:
# LIST THE BIDs THAT DO NOT MATCH IN NORM WAGE: THESE ARE THE BIDs NOT IN THE NORM WAGE...
bwregion = bigwage["BID"].unique().tolist()
normwageregion = normpw.index.tolist()
list(set(bwregion) - set(normwageregion))

In [ ]:
# GETTING THE APPLICANTS THAT DID NOT MATCH IN BID for NORM PW
nonmatchwage = pd.DataFrame()
for x in range(len(list(set(bwregion) - set(normwageregion)))):
    j = bigwage[bigwage["BID"]==list(set(bwregion) - set(normwageregion))[x]]
    nonmatchwage = pd.concat([nonmatchwage, j],axis = 0)

In [ ]:
# DOES THE NUMBER OF APPLICANTS THAT DO NOT MATCH = TO 35?
len(nonmatchwage)
# YES

In [ ]:
# GETTING THE ADJUSTED BID FOR NORM PW: ONLY KEEPING LEVELS SINCE IT IS MORE CORREL WITH WAGE THAN REGION
adjregw = regwage
adjregw["BID"] = adjregw["PW_SOC_CODE"].astype(str)+adjregw["PW_SKILL_LEVEL"].astype(str)
# NORMALIZED AVG WAGE FOR EACH UNIQUE IDENTIFIER
adjnormwage = pd.DataFrame(adjregw.groupby("BID")["PW_A"].mean())
adjnormwage.columns=["NORM_PW"]

# CREATING THE UNIQUE IDENTIFIER FOR MISSING BIG WAGE TO MATCH UP WITH REG WAGE AVG
adjbigwage = nonmatchwage
adjbigwage["BID"] = adjbigwage["PW_SOC_CODE"].astype(str)+ adjbigwage["PW_SKILL_LEVEL"].astype(str)

# MERGING BASED ON BID AND BRINGING IN NORM WAGE
matchadjbidwage = adjbigwage
matchadjbidwage = matchadjbidwage.merge(adjnormwage, how = "inner", on = "BID")

# HOW MANY APPLICANTS WERE NOT MATCHED THROUGH AdjBID
len(adjbigwage) - len(matchadjbidwage)

In [ ]:
# LIST THE Adj BIDs THAT DO NOT MATCH IN NORM WAGE: THESE ARE THE Adj BIDs NOT IN THE NORM WAGE...
adjbwregion = adjbigwage["BID"].unique().tolist()
adjnormwageregion = adjnormwage.index.tolist()
list(set(adjbwregion) - set(adjnormwageregion))

In [ ]:
# LIST THE APPLICANTS THAT ARE STILL NOT MATCHED THROUGH BID...
nonmatchadjwage = pd.DataFrame()
for x in range(len(list(set(adjbwregion) - set(adjnormwageregion)))):
    j = adjbigwage[adjbigwage["BID"]==list(set(adjbwregion) - set(adjnormwageregion))[x]]
    nonmatchadjwage = pd.concat([nonmatchadjwage, j],axis = 0)

# DOES THE NUMBER OF APPLICANTS THAT DO NOT MATCH = TO 2?
len(nonmatchadjwage)
# YES

In [ ]:
# GETTING THE WAGE FOR SOC CODE ONLY FOR NORM WAGE
soconly = regwage
soconly["BID"] = soconly["PW_SOC_CODE"].astype(str)
# NORMALIZED AVG WAGE FOR EACH UNIQUE IDENTIFIER
socnormwage = pd.DataFrame(soconly.groupby("BID")["PW_A"].mean())
socnormwage.columns=["NORM_PW"]

# CREATING THE UNIQUE IDENTIFIER FOR MISSING BIG WAGE TO MATCH UP WITH REG WAGE AVG
socbigwage = nonmatchadjwage
socbigwage["BID"] = socbigwage["PW_SOC_CODE"].astype(str)

# MERGING BASED ON BID AND BRINGING IN NORM WAGE
matchsocbidwage = socbigwage
matchsocbidwage = matchsocbidwage.merge(socnormwage, how = "inner", on = "BID")

# HOW MANY APPLICANTS WERE NOT MATCHED THROUGH AdjBID
len(socbigwage) - len(matchsocbidwage)

In [ ]:
# MERGING BACK TO MAKE THE FINAL BIG WAGE : USING NORM WAGE...
bigwage = pd.concat([matchbidwage,
matchadjbidwage,
matchsocbidwage],axis = 0, ignore_index = True)
bigwage["PW_A"] = bigwage["NORM_PW"]
bigwage = bigwage.drop(["BID","NORM_PW"], axis = 1)

In [ ]:
# MATCH APPLICATIONS IN BIGWAGE DF WITH ORIGINAL DF AND UPDATE WAGE AND PW DATA
bigwage = bigwage.set_index("ID")
df = df.set_index("ID")
df.loc[bigwage.index] = bigwage
df = df.reset_index()

In [ ]:
# ANNUALIZED WAGE OFFER - ANNUALIZED PW: REDO, WITH CLEANED UP ANNUAL WAGE AND PW
df["WO-PW"] = df["WO_A"] - df["PW_A"]

In [ ]:
# CLEANING UP DAYS TO DECIDE
# IQR EXTREME OUTLIER... SINCE WE HAVE NO IDEA IF AN APPLICANT TOOK THAT LONG, WE CANNOT NORMALIZE IT
# DROP THE OUTLIERS ABOVE 99.5%
ddtop = df.groupby("YEAR")["DAYS_TO_DECIDE"].quantile(.995)
ddtop

In [ ]:
# NEW COLUMN TO DROP
ddcutoff = ddtop.tolist()
for x in range(len(yr)):
    df["drop"] = np.where((df["YEAR"]==yr[x])&(df["DAYS_TO_DECIDE"]>=ddcutoff[x]),"DROP",np.nan)

In [ ]:
# HOW MANY ARE WE DROPPING?
(df["drop"]=="DROP").sum()

In [ ]:
# DROPPING ALL DAYS TO DECIDE > 99 PERCENTILE
df = df[df["drop"]!="DROP"].drop("drop",axis = 1).reset_index(drop = True)

In [ ]:
# CLEANING EMPLOYER HQ CITIES
# CLEAN UP JOB CITY: distribution of the length of the cities
df["EMPLOYER_CITY"].str.len().plot(kind = "hist")

In [ ]:
# TOO MANY TO MANUALLY CLEAN: 
# WHAT ARE THE TOP CITIES?
df["EMPLOYER_CITY"].value_counts()[:15]

In [ ]:
# HOW MANY VARIATIONS OF THE TOP CITIES?
#df[df["EMPLOYER_CITY"].str.contains("mountain view", case = False, na = False)]["EMPLOYER_CITY"].value_counts()

In [ ]:
# TURNING ALL THE NYC VARATIONS INTO NEW YORK + SPELL OUT NEW YORK
df["EMPLOYER_CITY"] =  df["EMPLOYER_CITY"].replace("NY","NEW YORK").replace("NYC","NEW YORK")
df.loc[df["EMPLOYER_CITY"].str.contains("new york", case = False, na = False), "EMPLOYER_CITY"] = "NEW YORK"
# MERGING THE TOP 15 cities (santa clara, COLLEGE STATION,redmond, cupertino, menlo park, and PLANO did NOT NEED CLEANING)
df.loc[df["EMPLOYER_CITY"].str.contains("seattle", case = False, na = False), "EMPLOYER_CITY"] = "SEATTLE"
df.loc[df["EMPLOYER_CITY"].str.contains("san francisco", case = False, na = False), "EMPLOYER_CITY"] = "SAN FRANCISCO"
df.loc[df["EMPLOYER_CITY"].str.contains("san jose", case = False, na = False), "EMPLOYER_CITY"] = "SAN JOSE"
df.loc[df["EMPLOYER_CITY"].str.contains("chicago", case = False, na = False), "EMPLOYER_CITY"] = "CHICAGO"
df.loc[df["EMPLOYER_CITY"].str.contains("houston", case = False, na = False), "EMPLOYER_CITY"] = "HOUSTON"
df.loc[df["EMPLOYER_CITY"].str.contains("sunnyvale", case = False, na = False), "EMPLOYER_CITY"] = "SUNNYVALE"
df.loc[df["EMPLOYER_CITY"].str.contains("los angeles", case = False, na = False), "EMPLOYER_CITY"] = "LOS ANGELES"
df.loc[df["EMPLOYER_CITY"].str.contains("edison", case = False, na = False), "EMPLOYER_CITY"] = "EDISON"

In [ ]:
# HOW MANY STATES ARE EMPLOYERS HQ IN? SHOULD BE NO MORE THAN 60...
df["EMPLOYER_STATE_PROVINCE"].nunique()
# ... 1157

In [ ]:
# ROUND 1 CLEANING STATES
for j in range(len(mainst)):
    df.loc[df["EMPLOYER_STATE_PROVINCE"].str.contains(mainst[j], case = False, na = False), "EMPLOYER_STATE_PROVINCE"] = mainabr[j]

In [ ]:
# HOW MANY STATES ARE THERE NOT ABR AFTER ROUND 1 CLEANING?
df["EMPLOYER_STATE_PROVINCE"].unique()[np.vectorize(len)(df["EMPLOYER_STATE_PROVINCE"].unique())>2]

In [ ]:
df["EMPLOYER_STATE_PROVINCE"].unique()[np.vectorize(len)(df["EMPLOYER_STATE_PROVINCE"].unique())>2].tolist()

In [ ]:
# CREATING THE SECONDARY LIST TO FINISH CLEANING STATES
secondst = ['PUERTO RICO', 'GUAM', 'NORTHERN MARIANA ISLANDS',
       'VIRGIN ISLANDS', 'MARSHALL ISLANDS',
       'FEDERATED STATES OF MICRONESIA'," PR","FL 32773","FL BUILDIN","OR OR","BRITISH COLUMBIA"]
secondabr = ["PR","GU","MP","VI","MH","FSM","PR","FL","FL","OR","BC"]
# ROUND 2 CLEANING STATES
for j in range(len(secondst)):
    df.loc[df["EMPLOYER_STATE_PROVINCE"].str.contains(secondst[j], case = False, na = False), "EMPLOYER_STATE_PROVINCE"] = secondabr[j]

In [ ]:
# CREATING A WORKSITE REGION COLUMN
df["EMPLOYER_REGION"]=df["EMPLOYER_STATE_PROVINCE"]
for j in range(len(geostate)):
    df.loc[df["EMPLOYER_REGION"].str.contains(geostate[j], case = False, na = False), "EMPLOYER_REGION"] = region[j]


In [ ]:
# UNIQUE REGIONS (CHECK IF ALL STATES WERE TRANSLATED CORRECTLY)
df["EMPLOYER_REGION"] = df["EMPLOYER_REGION"].replace("BC","OTHER")
df["EMPLOYER_REGION"].unique()

In [ ]:
# ADDING A CATEGORICAL VARIABLE OF DISCREPANCY BETWEEN EMPLOYER HQ STATE , REGION, AND CITY WITH JOB WORK SITE. UNIT OF WAGE AS WELL
df["DISC_ST"] = np.where(df["EMPLOYER_STATE_PROVINCE"]==df["WORKSITE_STATE"], 0, 1)
df["DISC_CTY"] = np.where(df["EMPLOYER_CITY"]==df["WORKSITE_CITY"], 0, 1)
df["DISC_RGION"] = np.where(df["EMPLOYER_REGION"]==df["WORKSITE_REGION"], 0, 1)
df["DISC_UNIT"] = np.where(df["WAGE_OFFER_UNIT_OF_PAY"]==df["PW_UNIT_OF_PAY"], 0, 1)

In [ ]:
# MANUAL CLEANING OF EMPLOYER NAMES: THIS CODE IS TAKEN FROM THE US IMMIGRANT EMPLOYEMENT MINIPROJECT IN MY GITHUB
df.loc[df["EMPLOYER_NAME"].str.contains("GOOGLE", case = False, na = False),"EMPLOYER_NAME"]="GOOGLE"
df.loc[(df["EMPLOYER_NAME"].str.contains("amazon |amazon.com", case = False, na = False))&(~df["EMPLOYER_NAME"].str.contains("amazon conservation team|amazon stones", case = False, na = False)), "EMPLOYER_NAME"] = "AMAZON"
df.loc[df["EMPLOYER_NAME"].str.contains("MICROSOFT", case = False, na = False), "EMPLOYER_NAME"] = "MICROSOFT"
df.loc[df["EMPLOYER_NAME"].str.contains("Cognizant", case = False, na = False), "EMPLOYER_NAME"] = "COGNIZANT"
df.loc[df["EMPLOYER_NAME"].str.contains("intel corporation|intel mobile|intel massachusetts|intel america|intel federal", case = False, na = False), "EMPLOYER_NAME"] = "INTEL"
df.loc[df["EMPLOYER_NAME"].str.contains("apple inc.|apple computer|apple educational", case = False, na = False), "EMPLOYER_NAME"] = "APPLE"
df.loc[df["EMPLOYER_NAME"].str.contains("infosys ltd.|infosys technologies|infosys public|infosys solutions|infosys consulting|infosys international|infosys BPO", case = False, na = False), "EMPLOYER_NAME"] = "INFOSYS"
df.loc[df["EMPLOYER_NAME"].str.contains("facebook", case = False, na = False), "EMPLOYER_NAME"] = "FACEBOOK"
df.loc[df["EMPLOYER_NAME"].str.contains("tata consultancy|tata technologies|tata communications|tata industries|tata consltancy|tata enterprises|tata america international|tata autocomp|tata international metals|tata chemicals|tata elxsi", case = False, na = False), "EMPLOYER_NAME"] = "TATA"
df.loc[df["EMPLOYER_NAME"].str.contains("oracle america|oracle usa|oracle financial", case = False, na = False), "EMPLOYER_NAME"] = "FACEBOOK"
df.loc[df["EMPLOYER_NAME"].str.contains("hcl america", case = False, na = False), "EMPLOYER_NAME"] = "HCL AMERICA"
df.loc[df["EMPLOYER_NAME"].str.contains("deloitte", case = False, na = False), "EMPLOYER_NAME"] = "DELOITTE"
df.loc[df["EMPLOYER_NAME"].str.contains("qualcomm", case = False, na = False), "EMPLOYER_NAME"] = "QUALCOMM"
df.loc[df["EMPLOYER_NAME"].str.contains("IBM ", case = False, na = False), "EMPLOYER_NAME"] = "IBM"
df.loc[df["EMPLOYER_NAME"].str.contains("Cisco systems|cisco consumer", case = False, na = False), "EMPLOYER_NAME"] = "CISCO"
df.loc[df["EMPLOYER_NAME"].str.contains("salesforce", case = False, na = False), "EMPLOYER_NAME"] = "SALESFORCE"
df.loc[df["EMPLOYER_NAME"].str.contains("morgan stanley", case = False, na = False), "EMPLOYER_NAME"] = "MORGAN STANLEY"
df.loc[df["EMPLOYER_NAME"].str.contains("JP Morgan", case = False, na = False), "EMPLOYER_NAME"] = "JP MORGAN"
df.loc[df["EMPLOYER_NAME"].str.contains("goldman sachs|goldman, sachs|goldman,sachs", case = False, na = False), "EMPLOYER_NAME"] = "GOLDMAN SACHS"
df.loc[df["EMPLOYER_NAME"].str.contains("MERRILL LYNCH", case = False, na = False), "EMPLOYER_NAME"] = "MERRILL LYNCH"
df.loc[df["EMPLOYER_NAME"].str.contains("Citibank|citigroup", case = False, na = False), "EMPLOYER_NAME"] = "CITI GROUP"
df.loc[df["EMPLOYER_NAME"].str.contains("CREDIT SUISSE", case = False, na = False), "EMPLOYER_NAME"] = "CREDIT SUISSE"
df.loc[df["EMPLOYER_NAME"].str.contains("pricewater|pwc", case = False, na = False), "EMPLOYER_NAME"] = "PRICEWATERHOUSECOOPERS"
df.loc[df["EMPLOYER_NAME"].str.contains("ernst  |ernst & Young", case = False, na = False), "EMPLOYER_NAME"] = "ERNST YOUNG"
df.loc[df["EMPLOYER_NAME"].str.contains("capgemini", case = False, na = False), "EMPLOYER_NAME"] = "CAPGEMINI"
df.loc[df["EMPLOYER_NAME"].str.contains("technip usa", case = False, na = False), "EMPLOYER_NAME"] = "TECHNIP USA"
df.loc[df["EMPLOYER_NAME"].str.contains("bechtel ", case = False, na = False), "EMPLOYER_NAME"] = "BECHTEL"
df.loc[(df["EMPLOYER_NAME"].str.contains("hp enterprise services|hp inc.|Hewlett-|Hewlett P|HewlettP", case = False, na = False))&(~df["EMPLOYER_NAME"].str.contains("queen nails hp inc|backer ehp inc", case = False, na = False)), "EMPLOYER_NAME"] = "AMAZON"
df.loc[df["EMPLOYER_NAME"].str.contains("CGGVERITAS", case = False, na = False), "EMPLOYER_NAME"] = "CGGVERITAS"
df.loc[df["EMPLOYER_NAME"].str.contains("national oilwell", case = False, na = False), "EMPLOYER_NAME"] = "NATIONAL OILWELL VARCO"
df.loc[df["EMPLOYER_NAME"].str.contains("IH SERVICES", case = False, na = False), "EMPLOYER_NAME"] = "IH SERVICES"
df.loc[df["EMPLOYER_NAME"].str.contains("paypal", case = False, na = False), "EMPLOYER_NAME"] = "PAYPAL"
df.loc[df["EMPLOYER_NAME"].str.contains("EBAY INC.|EBAY INC|EBAY ENTERPRISE|EBAY MOBILE LABS| EBAY PAYMENTS", case = False, na = False), "EMPLOYER_NAME"] = "EBAY"
df.loc[df["EMPLOYER_NAME"].str.contains("brocade", case = False, na = False), "EMPLOYER_NAME"] = "BROCADE"
df.loc[df["EMPLOYER_NAME"].str.contains("ADOBE INC.|ADOBE SYSTEMS", case = False, na = False), "EMPLOYER_NAME"] = "ADOBE"
df.loc[df["EMPLOYER_NAME"].str.contains("University of Chicago", case = False, na = False), "EMPLOYER_NAME"] = "THE UNIVERSITY OF CHICAGO"
df.loc[df["EMPLOYER_NAME"].str.contains("mckinsey", case = False, na = False), "EMPLOYER_NAME"] = "MCKINSEY COMPANY"
df.loc[df["EMPLOYER_NAME"].str.contains("motorola", case = False, na = False), "EMPLOYER_NAME"] = "MOTOROLA"
df.loc[df["EMPLOYER_NAME"].str.contains("citadel LLC|citadel americas|citadel Securities|citadel investment|citadel enterprise", case = False, na = False), "EMPLOYER_NAME"] = "CITADEL LLC"
df.loc[df["EMPLOYER_NAME"].str.contains("yahoo", case = False, na = False), "EMPLOYER_NAME"] = "YAHOO"
df.loc[df["EMPLOYER_NAME"].str.contains("Juniper networks", case = False, na = False), "EMPLOYER_NAME"] = "JUNIPER NETWORKS"
df.loc[df["EMPLOYER_NAME"].str.contains("a2z development", case = False, na = False), "EMPLOYER_NAME"] = "A2Z DEVELOPMENT"
df.loc[df["EMPLOYER_NAME"].str.contains("walmart|wal-mart", case = False, na = False), "EMPLOYER_NAME"] = "WALMART"
df.loc[df["EMPLOYER_NAME"].str.contains("fortinet", case = False, na = False), "EMPLOYER_NAME"] = "FORTINET"
df.loc[df["EMPLOYER_NAME"].str.contains("larsen &|larsen  ", case = False, na = False), "EMPLOYER_NAME"] = "LARSEN TOUBRO"
df.loc[df["EMPLOYER_NAME"].str.contains("mindtree", case = False, na = False), "EMPLOYER_NAME"] = "MINDTREE LIMITED"
df.loc[df["EMPLOYER_NAME"].str.contains("igate tech|igate mastech|igate global", case = False, na = False), "EMPLOYER_NAME"] = "IGATE TECHNOLOGIES"
df.loc[df["EMPLOYER_NAME"].str.contains("3i info", case = False, na = False), "EMPLOYER_NAME"] = "3I INFOTECH"
df.loc[df["EMPLOYER_NAME"].str.contains("NTT Data", case = False, na = False), "EMPLOYER_NAME"] = "NTT DATA"
df.loc[df["EMPLOYER_NAME"].str.contains("Texas instrument", case = False, na = False), "EMPLOYER_NAME"] = "TEXAS INSTRUMENTS"
df.loc[df["EMPLOYER_NAME"].str.contains("tech mahindra", case = False, na = False), "EMPLOYER_NAME"] = "TECH MAHINDRA"
df.loc[df["EMPLOYER_NAME"].str.contains("DELL USA LP|DELL PRODUCTS LP|DELL MARKETING LP|DELL SOFTWARE INC|DELL FINANCIAL", case = False, na = False), "EMPLOYER_NAME"] = "DELL"
df.loc[df["EMPLOYER_NAME"].str.contains("uber technologies", case = False, na = False), "EMPLOYER_NAME"] = "UBER TECHNOLOGIES"
df.loc[df["EMPLOYER_NAME"].str.contains("case farms", case = False, na = False), "EMPLOYER_NAME"] = "CASE FARMS"
df.loc[df["EMPLOYER_NAME"].str.contains("house of raeford", case = False, na = False), "EMPLOYER_NAME"] = "HOUSE OF RAEFORD FARMS"
df.loc[df["EMPLOYER_NAME"].str.contains("george's processing", case = False, na = False), "EMPLOYER_NAME"] = "GEORGE'S PROCESSING"
df.loc[df["EMPLOYER_NAME"].str.contains("abbyland", case = False, na = False), "EMPLOYER_NAME"] = "ABBYLAND FOODS"
df.loc[df["EMPLOYER_NAME"].str.contains("koch foods of cinc", case = False, na = False), "EMPLOYER_NAME"] = "KOCH FOODS OF CINCINNATI"
df.loc[df["EMPLOYER_NAME"].str.contains("koch foods of al", case = False, na = False), "EMPLOYER_NAME"] = "KOCH FOODS OF ALABAMA"


In [ ]:
# CREATING THE LIST OF COLUMNS THAT NEED CLEANING UP: "Y" OR "N" INTO 1 or 0
Y_N_cols = ["REFILE",
"REQUIRED_TRAINING",
"REQUIRED_EXPERIENCE",
"ACCEPT_FOREIGN_EDUCATION",
"ACCEPT_ALT_OCCUPATION",
"JOB_OPP_REQUIREMENTS_NORMAL",
"FOREIGN_LANGUAGE_REQUIRED",
"PROFESSIONAL_OCCUPATION",
"APP_FOR_COLLEGE_U_TEACHER",
"SUNDAY_EDITION_NEWSPAPER",
"SECOND_ADVERTISEMENT",
"JOB_FAIR",
"ON_CAMPUS_RECRUITING",
"EMPLOYER_WEBSITE",
"PRO_ORG_AD",
"JOB_SEARCH_WEBSITE",
"PVT_EMPLOYMENT_FIRM",
"EMPLOYEE_REF_PROG",
"CAMPUS_PLACEMENT",
"LOCAL_ETHNIC_PAPER",
"RADIO_TV_AD",
"EMP_RECEIVED_PAYMENT",
"BARGAINING_REP_NOTIFIED",
"POSTED_NOTICE_AT_WORKSITE",
"LAYOFF_IN_PAST_SIX_MONTHS",
"US_WORKERS_CONSIDERED",
"FOREIGN_WORKER_TRAINING_COMP",
"FOREIGN_WORKER_REQ_EXPERIENCE",
"FOREIGN_WORKER_ALT_ED_EXP",
"FOREIGN_WORKER_ALT_OCC_EXP",
"EMPLOYER_COMPLETED_APPLICATION"]

In [ ]:
# CHECKING WHAT THE MAX UNIQUE VALUES ARE FOR THE FIELDS WITH "Y" AND "N": THIS SHOULD BE NO MORE THAN 4 (Y,N,NA,Blank)
exlist = []
edf = pd.DataFrame()
for i in range(len(Y_N_cols)):
    j = df[Y_N_cols[i]].nunique()
    x = pd.DataFrame([Y_N_cols[i],j]).transpose()
    edf = pd.concat([edf,x])
edf

In [ ]:
# LOOKING INTO ACCEPT_ALT_OCCUPATION
df["ACCEPT_ALT_OCCUPATION"].value_counts()
# LOOKS LIKE SOME APPLICANTS HAVE Y AND N WHILE OTHERS WRITE THE ACCEPTED OCCUPATION...

In [ ]:
# GETTING A LIST TO RUN A LOOP AND FIND ALL THE UNIQUES FOR EACH COLUMN, EXCEPT FOR ALT_OCCUPATION
new_Y_N_cols = ["REFILE",
"REQUIRED_TRAINING",
"REQUIRED_EXPERIENCE",
"ACCEPT_FOREIGN_EDUCATION",
"JOB_OPP_REQUIREMENTS_NORMAL",
"FOREIGN_LANGUAGE_REQUIRED",
"PROFESSIONAL_OCCUPATION",
"APP_FOR_COLLEGE_U_TEACHER",
"SUNDAY_EDITION_NEWSPAPER",
"SECOND_ADVERTISEMENT",
"JOB_FAIR",
"ON_CAMPUS_RECRUITING",
"EMPLOYER_WEBSITE",
"PRO_ORG_AD",
"JOB_SEARCH_WEBSITE",
"PVT_EMPLOYMENT_FIRM",
"EMPLOYEE_REF_PROG",
"CAMPUS_PLACEMENT",
"LOCAL_ETHNIC_PAPER",
"RADIO_TV_AD",
"EMP_RECEIVED_PAYMENT",
"BARGAINING_REP_NOTIFIED",
"POSTED_NOTICE_AT_WORKSITE",
"LAYOFF_IN_PAST_SIX_MONTHS",
"US_WORKERS_CONSIDERED",
"FOREIGN_WORKER_TRAINING_COMP",
"FOREIGN_WORKER_REQ_EXPERIENCE",
"FOREIGN_WORKER_ALT_ED_EXP",
"FOREIGN_WORKER_ALT_OCC_EXP",
"EMPLOYER_COMPLETED_APPLICATION"]

In [ ]:
# CHECKING THE UNIQUE VALUES ARE FOR THE FIELDS WITH "Y" AND "N": THIS SHOULD BE NO MORE THAN 4 (Y,N,NA,Blank)
exlist = []
edf = pd.DataFrame()
for i in range(len(new_Y_N_cols)):
    j = df[new_Y_N_cols[i]].nunique()
    a = df[new_Y_N_cols[i]].unique()
    x = pd.DataFrame([new_Y_N_cols[i],j,a]).transpose()
    edf = pd.concat([edf,x])
edf

In [ ]:
# CHANGING "Y" TO 1, N to 0, A to 0, and "" to 0
for i in range(len(new_Y_N_cols)):
    df[new_Y_N_cols[i]] = np.where(df[new_Y_N_cols[i]]=="Y", 1, 0)

In [ ]:
# CHANGING ALT OCCUPATION Y to 1, N to 0, "" to 0, anything else will be a 1
df["ACCEPT_ALT_OCCUPATION"] = np.where(df["ACCEPT_ALT_OCCUPATION"]=="Y", 1, df["ACCEPT_ALT_OCCUPATION"])
df["ACCEPT_ALT_OCCUPATION"] = np.where((df["ACCEPT_ALT_OCCUPATION"]=="N")|(df["ACCEPT_ALT_OCCUPATION"]==""), 0, df["ACCEPT_ALT_OCCUPATION"])
df["ACCEPT_ALT_OCCUPATION"] = np.where(df["ACCEPT_ALT_OCCUPATION"]!=0, 1, df["ACCEPT_ALT_OCCUPATION"])

In [ ]:
# DROP THE COLUMNS NOT IN USE
df = df.drop(["ID","CASE_NUMBER","DECISION_DATE","RECEIVED_DATE"], axis = 1).reset_index(drop = True)

In [ ]:
# TURN ALL "" TO NP.NAN
df = df.replace("",np.nan)

In [ ]:
############################################################

In [ ]:
########## EDA ##########

In [ ]:
### EMPLOYER RELATED DATA ###

In [ ]:
# TURNING EMPLOYER EMPLOYEE NUMBER, "" INTO 0, THIS WILL ALLOW FOR EASIER CLEANING
df["EMPLOYER_NUM_EMPLOYEES"] = df["EMPLOYER_NUM_EMPLOYEES"].replace(np.nan,0)

In [ ]:
# CLEANING UP APPLICANT DATA BASED ON FREQUENCY OF EMPLOYER SPONSORSHIP...
# IF AN EMPLOYER ONLY HAS 1 APPLICANT EVER, PROBABLY WILL NOT SPONSOR AGAIN - COMPARED TO COMPANIES THAT SPONSOR FREQUENTLY
# HOW MANY EMPLOYERS HAVE SPONSORED ONLY ONCE? NOT COMPLETELY ACCURATE DUE TO LACK OF STANDARDIZATION RULES
print("There are around",(df["EMPLOYER_NAME"].value_counts()==1).sum(),"companies that have only sponsored one applicant.")
print("There are a total of",len(df["EMPLOYER_NAME"].value_counts()),"unique companies in the entire dataset.")
print("Dropping companies with only 1 applicant would be",round(((df["EMPLOYER_NAME"].value_counts()==1).sum()/len(df["EMPLOYER_NAME"].value_counts()))*100,2),"% of all companies.")
print("There would be",df["EMPLOYER_NAME"].value_counts()[df["EMPLOYER_NAME"].value_counts()>1].sum(),"applicants left out of",len(df),"in the entire dataset, or dropping",round((len(df)-df["EMPLOYER_NAME"].value_counts()[df["EMPLOYER_NAME"].value_counts()>1].sum())/len(df)*100,2),"%.")

In [ ]:
# BEFORE WE DROP, WHATS THE SUCCESS % OF THESE APPLICANTS?
# CREATE A DF WITH ONLY APPLICANTS TO COMPANIES THAT HAVE SPONSORED ONLY 1 APPLICANT
onlyone = df["EMPLOYER_NAME"].value_counts()[df["EMPLOYER_NAME"].value_counts()<=1].index.tolist()
onlyonedf = df[df["EMPLOYER_NAME"].isin(onlyone)]

In [ ]:
# SUCCESS RATE OF APPLICANTS SPONSORED BY COMPANIES THAT HAVE ONLY SPONSORED ONE APPLICANT
# COMPARE THIS TO THE REST OF THE DATA SET
print("Applicants sponsored by companies that have only sponsored one applicant have a,",round((onlyonedf["CASE_STATUS"].sum())/len(onlyonedf)*100,2),"% success rate.")
rest = df["EMPLOYER_NAME"].value_counts()[df["EMPLOYER_NAME"].value_counts()>1].index.tolist()
df = df[df["EMPLOYER_NAME"].isin(rest)]
print("The rest of the applicants have a,",round((df["CASE_STATUS"].sum())/len(df)*100,2),"% success rate.")

In [ ]:
# CLEANING UP APPLICANT DATA BASED ON EMPLOYEE NUM FIELD
# IF AN EMPLOYER HAS 0 or DID NOT FILL OUT THE EMPLOYEE NUM FIELD, WE WILL DROP: INACCURATE DATA
# HOW MANY EMPLOYERS HAVE 0 EMPLOYEES?
print("There are around",(df["EMPLOYER_NUM_EMPLOYEES"]==0).sum(),"applications from companies that have no employees or did not fill out the field.")
print("Dropping applicants with 0 employees or who did not fill out would be",round((df["EMPLOYER_NUM_EMPLOYEES"]==0).sum()/len(df)*100,2),"% of all applications.")
print("There would be",len(df[df["EMPLOYER_NUM_EMPLOYEES"]>0]),"applications left out of",len(df),".")

In [ ]:
# BEFORE WE DROP, WHATS THE SUCCESS % OF THESE APPLICANTS?
# CREATE A DF WITH ONLY APPLICANTS TO COMPANIES THAT HAVE 0 EMPLOYEES
none = df[df["EMPLOYER_NUM_EMPLOYEES"]==0]
# SUCCESS RATE OF APPLICANTS SPONSORED BY COMPANIES HAVE 0 EMPLOYEES / DID NOT FILL OUT THE FIELD
# COMPARE THIS TO THE REST OF THE DATA SET
print("Applicants sponsored by companies with 0 employees have a,",round((none["CASE_STATUS"].sum())/len(none)*100,2),"% success rate.")
df = df[df["EMPLOYER_NUM_EMPLOYEES"]>0]
print("The rest of the applicants have a,",round((df["CASE_STATUS"].sum())/len(df)*100,2),"% success rate.")

In [ ]:
# CLEANING UP APPLICANT DATA BASED ON EMPLOYER YR COMMENCED BUSINESS
# IF AN EMPLOYER COMMENCED BUSINESS IN YEAR 1600, DROP
print("There are around",(df["EMPLOYER_YEAR_COMMENCED_BUSINESS"]<=1600).sum(),"applications from companies that operated before the year 1600.")
print("Dropping applicants from companies operating before 1600 would be",round((df["EMPLOYER_YEAR_COMMENCED_BUSINESS"]<=1600).sum()/len(df)*100,2),"% of all applications.")
print("There would be",len(df[df["EMPLOYER_YEAR_COMMENCED_BUSINESS"]>1600]),"applications left out of",len(df),".")

In [ ]:
# BEFORE WE DROP, WHATS THE SUCCESS % OF THESE APPLICANTS?
# CREATE A DF WITH ONLY APPLICANTS TO COMPANIES THAT WERE OPERATING BEFORE 1600
none = df[df["EMPLOYER_YEAR_COMMENCED_BUSINESS"]<=1600]
# SUCCESS RATE OF APPLICANTS SPONSORED BY COMPANIES OPERATING BEFORE 1600
# COMPARE THIS TO THE REST OF THE DATA SET
print("Applicants sponsored by companies operating before 1600,",round((none["CASE_STATUS"].sum())/len(none)*100,2),"% success rate.")
df = df[df["EMPLOYER_YEAR_COMMENCED_BUSINESS"]>1600]
print("The rest of the applicants have a,",round((df["CASE_STATUS"].sum())/len(df)*100,2),"% success rate.")

In [ ]:
# Top 15 Employers of all time 2014-2021
allemp = pd.DataFrame(df["EMPLOYER_NAME"].value_counts()[:15]).reset_index()
allemp.columns = ["AEMP","COUNT AEMP"]
sucemp = pd.DataFrame(df[df["CASE_STATUS"]==1]["EMPLOYER_NAME"].value_counts()[:15]).reset_index()
sucemp.columns = ["SEMP","COUNT SEMP"]
pd.concat([allemp,sucemp], axis = 1)

In [ ]:
# Grabbing the top 15 for each year employers: only succesful applications
top15semp = pd.DataFrame()
def grab (year):
    return df[(df["YEAR"]==year)&(df["CASE_STATUS"]==1)]["EMPLOYER_NAME"].value_counts()[:15]
for x in range(len(yr)):
    j = pd.DataFrame(grab(yr[x]))
    top15semp = pd.concat([top15semp,j], axis = 1)
top15semp.columns= yr
# King of Consistency: HOW MANY YEARS IS THIS COMPANY NOT IN THE TOP 15?: TOP 10 CONSISTENT COMPANIES...
kingcons = top15semp.isnull().replace(False,0).replace(True,1)
kingcons["Yrs not in Top 15"] = top15semp.isnull().replace(False,0).replace(True,1).sum(axis = 1)
kingcons["Yrs not in Top 15"].sort_values()[:10]

In [ ]:
# Grabbing the top 15 for each year employers: total applications
top15aemp = pd.DataFrame()
def grab (year):
    return df[df["YEAR"]==year]["EMPLOYER_NAME"].value_counts()[:15]
for x in range(len(yr)):
    j = pd.DataFrame(grab(yr[x]))
    top15aemp = pd.concat([top15aemp,j], axis = 1)
top15aemp.columns= yr
# King of Consistency: HOW MANY YEARS IS THIS COMPANY NOT IN THE TOP 15?: TOP 10 CONSISTENT COMPANIES...
kingconsA = top15aemp.isnull().replace(False,0).replace(True,1)
kingconsA["Yrs not in Top 15"] = top15aemp.isnull().replace(False,0).replace(True,1).sum(axis = 1)
kingconsA["Yrs not in Top 15"].sort_values()[:10]

In [ ]:
# NEW COLUMN: 1 is in the top 15 of the YEAR, 0 is not: Applications
filler = pd.DataFrame()
for x in range(len(yr)):
    listy = top15aemp[yr[x]].dropna().index.tolist()
    filler = pd.concat([filler,pd.DataFrame((df[df["YEAR"]==yr[x]]["EMPLOYER_NAME"].isin(listy)).replace(True,1).replace(False,0))])

filler = filler.reset_index().sort_values(by = "index").set_index("index")
filler.columns = ["TOP15COMPANY"]
df["TOP15COMPANY"] = filler

In [ ]:
# CORRELATION ANALYSIS: NUMBER OF EMPLOYEEs WITH CASE STATUS: CASE BY CASE
st.linregress(x = df.dropna()["TOP15COMPANY"], y = df.dropna()["CASE_STATUS"])
# DOES NOT SHOW MUCH, AGGREGATE MAY SHOW A DIFFERENT STORY

In [ ]:
# PLOTTING THE DIFFERENT SUCCESS RATES OVER TIME BY TOP 15 COMPANY (IN TERMS OF ANNUAL APPLICATIONS)

top15 = df[df["TOP15COMPANY"]==1]
not15 = df[df["TOP15COMPANY"]==0]

topdf = pd.DataFrame()
notdf = pd.DataFrame()
analysisdf = pd.DataFrame()

for x in range(len(yr)):
    j = pd.DataFrame([len(top15[(top15["YEAR"]==yr[x])&(top15["CASE_STATUS"]==1)])/len(top15[top15["YEAR"]==yr[x]])])
    b = pd.DataFrame([len(not15[(not15["YEAR"]==yr[x])&(not15["CASE_STATUS"]==1)])/len(not15[not15["YEAR"]==yr[x]])])
    topdf = pd.concat([topdf,j])
    notdf = pd.concat([notdf,b])
    analysisdf = pd.concat([topdf,notdf], axis = 1)

analysisdf.index = yr
analysisdf.columns = ["Top 15 Success %", "Not 15 Success %"]
plt.figure(figsize = (12,10), dpi = 100)
analysisdf.plot(kind = "bar")
plt.tight_layout()
plt.legend(bbox_to_anchor=(1.05, 1), loc=2, borderaxespad=0.)
plt.title("Success Rates by Companies in the Top 15 and not in the Top 15, Over Time")

In [ ]:
# PREPARING DATA FOR LIN REGRESS AND GETTING P VALUE
top15apps = []
not15apps = []

for x in range(len(yr)):
    t = (top15["YEAR"]==yr[x]).sum()
    n = (not15["YEAR"]==yr[x]).sum()
    top15apps.append(t)
    not15apps.append(n)
analysisdf = analysisdf.reset_index()
analysisdf["Top 15 Apps"] = pd.DataFrame(top15apps)
analysisdf["Not 15 Apps"] = pd.DataFrame(not15apps)
analysisdf = analysisdf.set_index("index")
# PLOTTING APPLICATION RATES OVER TIME BY TOP 15 or NOT TOP 15
plt.figure(figsize = (12,10), dpi = 100)
analysisdf.reset_index().plot(x = "index", y = ["Top 15 Apps","Not 15 Apps"], kind = "bar")
plt.tight_layout()
plt.legend(bbox_to_anchor=(1.05, 1), loc=2, borderaxespad=0.)
plt.title("Application Rates by Companies in the Top 15 and not in the Top 15, Over Time")

In [ ]:
# P VALUE PER YEAR: ON THE SUCCESS RATE
top_not = []
for x in range(len(yr)):
    sample1_phat = analysisdf.loc[yr[x]]["Top 15 Success %"]
    sample2_phat = analysisdf.loc[yr[x]]["Not 15 Success %"]
    sample1_size = analysisdf.loc[yr[x]]["Top 15 Apps"]
    sample2_size = analysisdf.loc[yr[x]]["Not 15 Apps"]
    phat = (sample1_phat*sample1_size+sample2_phat*sample2_size)/(sample1_size+sample2_size)
    zscore = (sample1_phat - sample2_phat)/np.sqrt(phat*(1-phat)*((1/sample1_size)+(1/sample2_size)))    
    pval = st.norm.sf(abs(zscore))*2
    top_not.append(pval)

pvaldf = pd.DataFrame(top_not)
pvaldf.columns = ["Top 15 Vs. Not Top 15"]
pvaldf.index = yr
analysisdf["PVAL Success %"] = pvaldf["Top 15 Vs. Not Top 15"]

In [ ]:
# LINEAR REGRESSION BETWEEN COMPANY SIZE AND SUCCESS RATE OVER TIME: TOP 15 COMPANY
print("Top 15 Company Success Rate Over Time Slope:",round(st.linregress(y = analysisdf["Top 15 Success %"], x = yr).slope,6),", P Val:",round(st.linregress(y = analysisdf["Top 15 Success %"], x = yr).pvalue,4),", R Val:",round(st.linregress(y = analysisdf["Top 15 Success %"], x = yr).rvalue,4),"\nModerate growth, .68% growth in success rate a year, but not a statistically significant slope.")
# LINEAR REGRESSION BETWEEN COMPANY SIZE AND APPLICATIONS OVER TIME: TOP 15 COMPANY
print("\nTop 15 Company Application Rate Over Time Slope:",round(st.linregress(y = analysisdf["Top 15 Apps"], x = yr).slope,3),", P Val:",round(st.linregress(y = analysisdf["Top 15 Apps"], x = yr).pvalue,4),", R Val:",round(st.linregress(y = analysisdf["Top 15 Apps"], x = yr).rvalue,4),"\nModerate growth, 1379 applications a year, but not a statistically significant slope.")

In [ ]:
# LINEAR REGRESSION BETWEEN COMPANY SIZE AND SUCCESS RATE OVER TIME: NOT TOP 15 COMPANY
print("Not Top 15 Company Success Rate Over Time Slope:",round(st.linregress(y = analysisdf["Not 15 Success %"], x = yr).slope,6),", P Val:",round(st.linregress(y = analysisdf["Not 15 Success %"], x = yr).pvalue,4),", R Val:",round(st.linregress(y = analysisdf["Not 15 Success %"], x = yr).rvalue,4),"\nStrong growth, .54% growth in success rate a year, and a statistically significant slope.")
# LINEAR REGRESSION BETWEEN COMPANY SIZE AND APPLICATIONS OVER TIME: NOT TOP 15 COMPANY
print("\nNot Top 15 Company Application Rate Over Time Slope:",round(st.linregress(y = analysisdf["Not 15 Apps"], x = yr).slope,3),", P Val:",round(st.linregress(y = analysisdf["Not 15 Apps"], x = yr).pvalue,4),", R Val:",round(st.linregress(y = analysisdf["Not 15 Apps"], x = yr).rvalue,4),"\nWeak growth, 3516 applications a year, but not a statistically significant slope.")

In [ ]:
# GETTING THE TOTAL PVAL
analysisdf.loc["ALL YRS"] = analysisdf.sum()
top15suc = []
not15suc = []

for x in range(len(yr)):
    t15 = analysisdf.loc[yr[x]].iloc[0]*analysisdf.loc[yr[x]].iloc[2]
    n15 = analysisdf.loc[yr[x]].iloc[1]*analysisdf.loc[yr[x]].iloc[3]
    top15suc.append(t15)
    not15suc.append(n15)
analysisdf.loc["ALL YRS"].iloc[0] = sum(top15suc)/analysisdf.loc["ALL YRS"].iloc[2]
analysisdf.loc["ALL YRS"].iloc[1] = sum(not15suc)/analysisdf.loc["ALL YRS"].iloc[3]


for x in range(len(yr)):
    sample1_phat = analysisdf.loc["ALL YRS"].iloc[0]
    sample2_phat = analysisdf.loc["ALL YRS"].iloc[1]
    sample1_size = analysisdf.loc["ALL YRS"].iloc[2]
    sample2_size = analysisdf.loc["ALL YRS"].iloc[3]
    phat = (sample1_phat*sample1_size+sample2_phat*sample2_size)/(sample1_size+sample2_size)
    zscore = (sample1_phat - sample2_phat)/np.sqrt(phat*(1-phat)*((1/sample1_size)+(1/sample2_size)))    
    pval = st.norm.sf(abs(zscore))*2
analysisdf.loc["ALL YRS"].iloc[4] = pval

analysisdf = analysisdf.head(9).style.format("{:,.2f}")
analysisdf

In [ ]:
# CREATE A NEW COLUMN: YRS EDUCATION - DECISION YEAR, HOW MANY YEARS AFTER GRADUATION: POSTGRADYRS
# THIS IS FOR A DEEPER DIVE INTO THE TOP EMPLOYERS... 
df["YRSPOSTGRAD"] = df["YEAR"]-df["FOREIGN_WORKER_YRS_ED_COMP"]
# CLEANING UP WORKER EDUCATION AND EDUCATION REQUIREMENTS
df["FOREIGN_WORKER_EDUCATION"]= df["FOREIGN_WORKER_EDUCATION"].replace("NONE",np.nan)
df["MINIMUM_EDUCATION"] = df["MINIMUM_EDUCATION"].replace("NONE",np.nan)

In [ ]:
# DIVING DEEPER INTO MICROSOFT, AMAZON, GOOGLE, APPLE (COMPANY VARIABLE CAN BE CHANGED FOR ANY COMPANY)
#ANCHORED BY STATE: PW SKILL,APPLICATIONS, MEDIAN WAGE, YRS POST GRADE, MEDIAN DAYS

company = "MICROSOFT"
# CHANGE THIS FOR ANY COMPANY

# MEDIAN WAGE, MEDIAN YEARS AFTER GRADUATING, MEDIAN DAYS TO DECIDE : FOR EACH STATE IN THE TOP 15 OF THE COMPANY AND BY PW LEVEL
topstate = df[df["EMPLOYER_NAME"] == company]["WORKSITE_STATE"].value_counts()[:15].index.tolist()
pwskill = df["PW_SKILL_LEVEL"].dropna().unique().tolist()
pwskill.sort()


statelist = []
pwlist = pwskill*len(topstate)
apps = []
medwage = []
yrsgrad = []
meddays = []


for x in range(len(topstate)):
    s = (topstate[x]+",")*len(pwskill)
    s = "".join(s).split(",")
    s = list(filter(("").__ne__, s))
    statelist.append(s) 
for x in range(len(topstate)):
    for j in range(len(pwskill)):
        a = len(df[(df["EMPLOYER_NAME"] == company)&(df["WORKSITE_STATE"] == topstate[x])][df[(df["EMPLOYER_NAME"] == company)&(df["WORKSITE_STATE"] == topstate[x])]["PW_SKILL_LEVEL"]==pwskill[j]])
        apps.append(a)
        b = df[(df["EMPLOYER_NAME"] == company)&(df["WORKSITE_STATE"] == topstate[x])][df[(df["EMPLOYER_NAME"] == company)&(df["WORKSITE_STATE"] == topstate[x])]["PW_SKILL_LEVEL"]==pwskill[j]]["WO_A"].median()
        medwage.append(b)
        c = df[(df["EMPLOYER_NAME"] == company)&(df["WORKSITE_STATE"] == topstate[x])][df[(df["EMPLOYER_NAME"] == company)&(df["WORKSITE_STATE"] == topstate[x])]["PW_SKILL_LEVEL"]==pwskill[j]]["YRSPOSTGRAD"].median()
        yrsgrad.append(c)
        d = df[(df["EMPLOYER_NAME"] == company)&(df["WORKSITE_STATE"] == topstate[x])][df[(df["EMPLOYER_NAME"] == company)&(df["WORKSITE_STATE"] == topstate[x])]["PW_SKILL_LEVEL"]==pwskill[j]]["DAYS_TO_DECIDE"].median()
        meddays.append(d)
empstatedf = pd.DataFrame([sum(statelist, []),pwlist,apps,medwage,yrsgrad,meddays]).transpose()
empstatedf.columns = ["STATE","PW LEVEL", "TOTAL APPLICATIONS","MEDIAN WAGE","MEDIAN YRS POST GRAD","MEDIAN DAYS TO DECIDE"]
empstatedf = empstatedf.set_index(["STATE","PW LEVEL"])
empstatedf

In [ ]:
# ANCHORED BY STATE: EDU LEVEL,APPLICATIONS, MEDIAN WAGE, YRS POST GRADE, MEDIAN DAYS
# MEDIAN WAGE, MEDIAN YEARS AFTER GRADUATING, MEDIAN DAYS TO DECIDE : FOR EACH STATE IN THE TOP 15 OF THE COMPANY AND BY EDU LEVEL
edu = df["FOREIGN_WORKER_EDUCATION"].dropna().unique().tolist()
edu[0],edu[1],edu[2],edu[3],edu[4],edu[5] = edu[3],edu[5],edu[0],edu[2],edu[1],edu[4]

statelist = []
edulist = edu*len(topstate)
apps = []
medwage = []
yrsgrad = []
meddays = []


for x in range(len(topstate)):
    s = (topstate[x]+",")*len(edu)
    s = "".join(s).split(",")
    s = list(filter(("").__ne__, s))
    statelist.append(s) 
for x in range(len(topstate)):
    for j in range(len(edu)):
        a = len(df[(df["EMPLOYER_NAME"] == company)&(df["WORKSITE_STATE"] == topstate[x])][df[(df["EMPLOYER_NAME"] == company)&(df["WORKSITE_STATE"] == topstate[x])]["FOREIGN_WORKER_EDUCATION"]==edu[j]])
        apps.append(a)
        b = df[(df["EMPLOYER_NAME"] == company)&(df["WORKSITE_STATE"] == topstate[x])][df[(df["EMPLOYER_NAME"] == company)&(df["WORKSITE_STATE"] == topstate[x])]["FOREIGN_WORKER_EDUCATION"]==edu[j]]["WO_A"].median()
        medwage.append(b)
        c = df[(df["EMPLOYER_NAME"] == company)&(df["WORKSITE_STATE"] == topstate[x])][df[(df["EMPLOYER_NAME"] == company)&(df["WORKSITE_STATE"] == topstate[x])]["FOREIGN_WORKER_EDUCATION"]==edu[j]]["YRSPOSTGRAD"].median()
        yrsgrad.append(c)
        d = df[(df["EMPLOYER_NAME"] == company)&(df["WORKSITE_STATE"] == topstate[x])][df[(df["EMPLOYER_NAME"] == company)&(df["WORKSITE_STATE"] == topstate[x])]["FOREIGN_WORKER_EDUCATION"]==edu[j]]["DAYS_TO_DECIDE"].median()
        meddays.append(d)
empstatedf1 = pd.DataFrame([sum(statelist, []),edulist,apps,medwage,yrsgrad,meddays]).transpose()
empstatedf1.columns = ["STATE","EDU LEVEL", "TOTAL APPLICATIONS","MEDIAN WAGE","MEDIAN YRS POST GRAD","MEDIAN DAYS TO DECIDE"]
empstatedf1 = empstatedf1.set_index(["STATE","EDU LEVEL"])
empstatedf1

In [ ]:
# WHERE ARE EMPLOYERS HEADQUARTED? 
# TOP 15 STATES WHERE EMPLOYERS HQ AND SPONSOR GREEN CARDS
allHQ = pd.DataFrame(df["EMPLOYER_STATE_PROVINCE"].value_counts()[:15]).reset_index()
allHQ.columns = ["ASTATE","COUNT ASTATE"]
sucHQ = pd.DataFrame(df[df["CASE_STATUS"]==1]["EMPLOYER_STATE_PROVINCE"].value_counts()[:15]).reset_index()
sucHQ.columns = ["SSTATE","COUNT SSTATE"]
pd.concat([allHQ,sucHQ], axis = 1)

In [ ]:
# CHOROPLETH MAP OF THE TOP STATES BY SUCCESS
# ACCEPT % BASED ON STATE HQ IN
empy = df.groupby("EMPLOYER_STATE_PROVINCE")["CASE_STATUS"].value_counts()
empyer = pd.DataFrame(empy.groupby(level=0).transform(lambda x: (x / x.sum()).round(2)))
empyer.columns = ["Percent"]
f1 = empyer.reset_index()["EMPLOYER_STATE_PROVINCE"].tolist()
t1 = empyer.reset_index()["CASE_STATUS"].replace(1,"Accept").replace(0,"Reject").tolist()
idx1 = []
idx1.extend([tuple(a) for a in zip (f1,t1)])
empyer.index = pd.MultiIndex.from_tuples(idx1)
empyer = empyer.reset_index()
empyer = empyer[empyer["level_1"]=="Accept"]
empyer = empyer.drop("level_1",axis = 1)

empycount = pd.DataFrame(df.groupby("EMPLOYER_STATE_PROVINCE")["CASE_STATUS"].value_counts())
empycount.columns = ["Count"]
empycount = empycount.reset_index()
empycount = empycount[empycount["CASE_STATUS"]==1]
empycount = empycount["Count"]

empyer = pd.concat([empyer, empycount], axis = 1)
empyer.columns = ["State","Percent","Count"]
empyer["Count"] = empyer["State"].astype(str)+": "+empyer["Count"].astype(str)+" Successful Applications"
# Choropleth Map of Employer State Accept %
data = dict(type = "choropleth",
           locations = empyer["State"],
           locationmode = "USA-states",
           z = empyer["Percent"].astype(float),
           text = empyer["Count"],
           colorbar={"title":"Percent Accepted"},
           colorscale = "delta",
           marker = dict(line = dict(color = "rgb(255,255,255)")) )
layout = dict(title = "Percent Accepted in Employer Headquartered States",
              geo = dict(scope = "usa",showlakes = True, lakecolor = "rgb(255,255,255)"))
choro = go.Figure(data, layout)
iplot(choro)

In [ ]:
# HOW MANY OF THE TOP COMPANIES, IDENTIFIED ABOVE, ARE HQ IN THE TOP 5 STATES
companies = kingconsA["Yrs not in Top 15"].sort_values()[:7].index.tolist()
states = allHQ["ASTATE"][:5].tolist()
apps = allHQ["COUNT ASTATE"][:5].tolist()
companystates = pd.DataFrame()
for x in range(len(companies)):
    for i in range(len(states)):
        c = (df[df["EMPLOYER_NAME"]==companies[x]]["EMPLOYER_STATE_PROVINCE"]==states[i]).sum()
        n = pd.DataFrame([companies[x],states[i],c]).transpose()
        companystates = pd.concat([companystates,n])
companystates.columns=["EMPLOYER","STATE","APPLICATIONS"]
companystates["TOTALAPP"]=np.nan
for x in range(len(states)):
    companystates["TOTALAPP"] = np.where(companystates["STATE"]==states[x],apps[x],companystates["TOTALAPP"])
companystates = companystates.set_index(["EMPLOYER","STATE"])
companystates["% OF TOTAL APPLICATIONS"] = (companystates["APPLICATIONS"]/companystates["TOTALAPP"])*100
companystates = companystates.drop("TOTALAPP", axis = 1)
companystates

In [ ]:
# WHERE ARE EMPLOYERS HEADQUARTED? 
# TOP REGIONS WHERE EMPLOYERS HQ AND SPONSOR GREEN CARDS
allRHQ = pd.DataFrame(df["EMPLOYER_REGION"].value_counts()[:15]).reset_index()
allRHQ.columns = ["AREGION","COUNT AREGION"]
sucRHQ = pd.DataFrame(df[df["CASE_STATUS"]==1]["EMPLOYER_REGION"].value_counts()[:15]).reset_index()
sucRHQ.columns = ["SREGION","COUNT SREGION"]
pd.concat([allRHQ,sucRHQ], axis = 1)

In [ ]:
# HOW MANY OF THE TOP COMPANIES, IDENTIFIED ABOVE, ARE HQ IN THE TOP 4 REGION
companies = kingconsA["Yrs not in Top 15"].sort_values()[:7].index.tolist()
region = allRHQ["AREGION"][:4].tolist()
apps = allRHQ["COUNT AREGION"][:4].tolist()
companyregions = pd.DataFrame()
for x in range(len(companies)):
    for i in range(len(region)):
        c = (df[df["EMPLOYER_NAME"]==companies[x]]["EMPLOYER_REGION"]==region[i]).sum()
        n = pd.DataFrame([companies[x],region[i],c]).transpose()
        companyregions = pd.concat([companyregions,n])
companyregions.columns=["EMPLOYER","REGION","APPLICATIONS"]
companyregions["TOTALAPP"]=np.nan
for x in range(len(region)):
    companyregions["TOTALAPP"] = np.where(companyregions["REGION"]==region[x],apps[x],companyregions["TOTALAPP"])
companyregions = companyregions.set_index(["EMPLOYER","REGION"])
companyregions["% OF TOTAL APPLICATIONS"] = (companyregions["APPLICATIONS"]/companyregions["TOTALAPP"])*100
companyregions = companyregions.drop("TOTALAPP", axis = 1)
companyregions

In [ ]:
# HOW MANY EMPLOYERS ARE NOT HQ IN USA?
print("Only",df["EMPLOYER_COUNTRY"].value_counts()[1:].sum(),"countries are not headquartered in the U.S.")
print(df["EMPLOYER_COUNTRY"].value_counts())

In [ ]:
# EXPLORING THE DISTRIBUTION OF NUMBER OF EMPLOYEES PER EMPLOYER
avgempnum = df.groupby("EMPLOYER_NAME")["EMPLOYER_NUM_EMPLOYEES"].agg(lambda x: st.mode(x)[0][0])
# WHAT IS THE MOST EMPLOYEES PER EMPLOYER
avgempnum.sort_values(ascending = False)

In [ ]:
# THE MODE LOOKS FISHY, LETS BRING IN THE TOTAL APPLICANTS
countemp = pd.DataFrame(df.groupby("EMPLOYER_NAME")["CASE_STATUS"].count()).reset_index().sort_values(by = "EMPLOYER_NAME").reset_index(drop = True)
empdf = pd.DataFrame(avgempnum).reset_index().sort_values(by = "EMPLOYER_NAME").reset_index(drop = True)
empdf["COUNT"] = np.where(empdf["EMPLOYER_NAME"]==countemp["EMPLOYER_NAME"],countemp["CASE_STATUS"],np.nan)
empdf = empdf.sort_values(by = "EMPLOYER_NUM_EMPLOYEES", ascending = False)
empdf

In [ ]:
# PLOT THE DIST OF EMPLOYEE COUNT
btmemp = avgempnum[avgempnum<=avgempnum.median()]
topemp = avgempnum[avgempnum>avgempnum.median()]
fig, ax = plt.subplots(1,2, figsize = (12,10))
sns.histplot(x = btmemp, ax = ax[0])
topend = sns.histplot(x = topemp, ax = ax[1])
topend.set_xlim([50, 10000])
plt.tight_layout()

In [ ]:
# CORRELATION ANALYSIS: NUMBER OF EMPLOYEEs WITH CASE STATUS: CASE BY CASE
st.linregress(x = df.dropna()["EMPLOYER_NUM_EMPLOYEES"], y = df.dropna()["CASE_STATUS"])
# DOES NOT SHOW MUCH, AGGREGATE MAY SHOW A DIFFERENT STORY

In [ ]:
# CORRELATION ANALYSIS: AGGREGATE BY SIZE BRACKETS AND YEARLY
# PLOTTING SUCCESS RATES OVER TIME BY SIZE OF COMPANY

small = df[df["EMPLOYER_NUM_EMPLOYEES"]<1000]
medium = df[(df["EMPLOYER_NUM_EMPLOYEES"]>=1000)&(df["EMPLOYER_NUM_EMPLOYEES"]<10000)]
large = df[df["EMPLOYER_NUM_EMPLOYEES"]>=10000]


smalldf = pd.DataFrame()
meddf = pd.DataFrame()
largedf = pd.DataFrame()
regressdf = pd.DataFrame()

for x in range(len(yr)):
    j = pd.DataFrame([len(small[(small["YEAR"]==yr[x])&(small["CASE_STATUS"]==1)])/len(small[small["YEAR"]==yr[x]])])
    med = pd.DataFrame([len(medium[(medium["YEAR"]==yr[x])&(medium["CASE_STATUS"]==1)])/len(medium[medium["YEAR"]==yr[x]])])
    lar = pd.DataFrame([len(large[(large["YEAR"]==yr[x])&(large["CASE_STATUS"]==1)])/len(large[large["YEAR"]==yr[x]])])
    smalldf = pd.concat([smalldf,j])
    meddf = pd.concat([meddf,med])
    largedf = pd.concat([largedf,lar])
    regressdf = pd.concat([smalldf,meddf,largedf], axis = 1)
    

regressdf.index = yr
regressdf.columns = ["Small Success %", "Medium Success %", "Large Success %"]
plt.figure(figsize = (12,10), dpi = 100)
regressdf.plot(kind = "bar")
plt.tight_layout()
plt.legend(bbox_to_anchor=(1.05, 1), loc=2, borderaxespad=0.)
plt.title("Success Rates by Company Size, Over Time")

In [ ]:
# PREPARING DATA FOR LIN REGRESS AND GETTING P VALUE
smallapps = []
mediumapps = []
largeapps = []

for x in range(len(yr)):
    s = (small["YEAR"]==yr[x]).sum()
    ssd = st.tstd(small[small["YEAR"]==yr[x]]["CASE_STATUS"])
    m = (medium["YEAR"]==yr[x]).sum()
    msd = st.tstd(medium[medium["YEAR"]==yr[x]]["CASE_STATUS"])
    l = (large["YEAR"]==yr[x]).sum()
    lsd = st.tstd(large[large["YEAR"]==yr[x]]["CASE_STATUS"])
    smallapps.append(s)
    mediumapps.append(m)
    largeapps.append(l)
regressdf = regressdf.reset_index()
regressdf["Small Apps"] = pd.DataFrame(smallapps)
regressdf["Medium Apps"] = pd.DataFrame(mediumapps)
regressdf["Large Apps"] = pd.DataFrame(largeapps)
regressdf = regressdf.set_index("index")
# PLOTTING APPLICATION RATES OVER TIME BY SIZE OF COMPANY
plt.figure(figsize = (12,10), dpi = 100)
regressdf.reset_index().plot(x = "index", y = ["Small Apps","Medium Apps","Large Apps"], kind = "bar")
plt.tight_layout()
plt.legend(bbox_to_anchor=(1.05, 1), loc=2, borderaxespad=0.)
plt.title("Application Rates by Company Size, Over Time")

In [ ]:
# P VALUE PER YEAR
small_med = []
for x in range(len(yr)):
    sample1_phat = regressdf.loc[yr[x]]["Small Success %"]
    sample2_phat = regressdf.loc[yr[x]]["Medium Success %"]
    sample1_size = regressdf.loc[yr[x]]["Small Apps"]
    sample2_size = regressdf.loc[yr[x]]["Medium Apps"]
    phat = (sample1_phat*sample1_size+sample2_phat*sample2_size)/(sample1_size+sample2_size)
    zscore = (sample1_phat - sample2_phat)/np.sqrt(phat*(1-phat)*((1/sample1_size)+(1/sample2_size)))    
    pval = st.norm.sf(abs(zscore))*2
    small_med.append(pval)

pvaldf = pd.DataFrame(small_med)

small_large = []
for x in range(len(yr)):
    sample1_phat = regressdf.loc[yr[x]]["Small Success %"]
    sample2_phat = regressdf.loc[yr[x]]["Large Success %"]
    sample1_size = regressdf.loc[yr[x]]["Small Apps"]
    sample2_size = regressdf.loc[yr[x]]["Large Apps"]
    phat = (sample1_phat*sample1_size+sample2_phat*sample2_size)/(sample1_size+sample2_size)
    zscore = (sample1_phat - sample2_phat)/np.sqrt(phat*(1-phat)*((1/sample1_size)+(1/sample2_size)))    
    pval = st.norm.sf(abs(zscore))*2
    small_large.append(pval)
pvaldf = pd.concat([pvaldf,pd.DataFrame(small_large)], axis = 1)

med_large = []
for x in range(len(yr)):
    sample1_phat = regressdf.loc[yr[x]]["Medium Success %"]
    sample2_phat = regressdf.loc[yr[x]]["Large Success %"]
    sample1_size = regressdf.loc[yr[x]]["Medium Apps"]
    sample2_size = regressdf.loc[yr[x]]["Large Apps"]
    phat = (sample1_phat*sample1_size+sample2_phat*sample2_size)/(sample1_size+sample2_size)
    zscore = (sample1_phat - sample2_phat)/np.sqrt(phat*(1-phat)*((1/sample1_size)+(1/sample2_size)))    
    pval = st.norm.sf(abs(zscore))*2
    med_large.append(pval)
pvaldf = pd.concat([pvaldf,pd.DataFrame(med_large)], axis = 1)
pvaldf.columns = ["Small_Med","Small_Large","Med_Large"]
pvaldf.index = yr
regressdf[["Small_Med","Small_Large","Med_Large"]] = pvaldf[["Small_Med","Small_Large","Med_Large"]]

In [ ]:
# LINEAR REGRESSION BETWEEN COMPANY SIZE AND SUCCESS RATE OVER TIME: SMALL COMPANY
print("Small Company Success Rate Over Time Slope:",round(st.linregress(y = regressdf["Small Success %"], x = yr).slope,6),", P Val:",round(st.linregress(y = regressdf["Small Success %"], x = yr).pvalue,4),", R Val:",round(st.linregress(y = regressdf["Small Success %"], x = yr).rvalue,4),"\nWeak growth, .19% growth in success rate a year, and not a statistically significant slope.")
# LINEAR REGRESSION BETWEEN COMPANY SIZE AND APPLICATIONS OVER TIME: SMALL COMPANY
print("\nSmall Company Application Rate Over Time Slope:",round(st.linregress(y = regressdf["Small Apps"], x = yr).slope,3),", P Val:",round(st.linregress(y = regressdf["Small Apps"], x = yr).pvalue,4),", R Val:",round(st.linregress(y = regressdf["Small Apps"], x = yr).rvalue,4),"\nWeak growth, 1204 applications a year, and not a statistically significant slope.")

In [ ]:
# LINEAR REGRESSION BETWEEN COMPANY SIZE AND SUCCESS RATE OVER TIME: MEDIUM COMPANY
print("Medium Company Success Rate Over Time Slope:",round(st.linregress(y = regressdf["Medium Success %"], x = yr).slope,6),", P Val:",round(st.linregress(y = regressdf["Medium Success %"], x = yr).pvalue,4),", R Val:",round(st.linregress(y = regressdf["Medium Success %"], x = yr).rvalue,4),"\nStrong growth, .57% growth in success rate a year, with a statistically significant slope.")
# LINEAR REGRESSION BETWEEN COMPANY SIZE AND APPLICATIONS OVER TIME: MEDIUM COMPANY
print("\nMedium Company Application Rate Over Time Slope:",round(st.linregress(y = regressdf["Medium Apps"], x = yr).slope,3),", P Val:",round(st.linregress(y = regressdf["Medium Apps"], x = yr).pvalue,4),", R Val:",round(st.linregress(y = regressdf["Medium Apps"], x = yr).rvalue,4),"\nModerate growth, 1122 applications a year, and not a statistically significant slope.")

In [ ]:
# LINEAR REGRESSION BETWEEN COMPANY SIZE AND SUCCESS RATE OVER TIME: LARGE COMPANY
print("Large Company Success Rate Over Time Slope:",round(st.linregress(y = regressdf["Large Success %"], x = yr).slope,6),", P Val:",round(st.linregress(y = regressdf["Large Success %"], x = yr).pvalue,4),", R Val:",round(st.linregress(y = regressdf["Large Success %"], x = yr).rvalue,4),"\nStrong growth, .83% growth in success rate a year, and a statistically significant slope.")
# LINEAR REGRESSION BETWEEN COMPANY SIZE AND APPLICATIONS OVER TIME: LARGE COMPANY
print("\nLarge Company Application Rate Over Time Slope:",round(st.linregress(y = regressdf["Large Apps"], x = yr).slope,3),", P Val:",round(st.linregress(y = regressdf["Large Apps"], x = yr).pvalue,4),", R Val:",round(st.linregress(y = regressdf["Large Apps"], x = yr).rvalue,4),"\nModerate growth, 2569 applications a year, and not a statistically significant slope.")

In [ ]:
# GETTING THE TOTAL PVAL
regressdf.loc["ALL YRS"] = regressdf.sum()
smallsuc = []
medsuc = []
largesuc = []
for x in range(len(yr)):
    s = regressdf.loc[yr[x]].iloc[0]*regressdf.loc[yr[x]].iloc[3]
    m = regressdf.loc[yr[x]].iloc[1]*regressdf.loc[yr[x]].iloc[4]
    l = regressdf.loc[yr[x]].iloc[2]*regressdf.loc[yr[x]].iloc[5]
    smallsuc.append(s)
    medsuc.append(m)
    largesuc.append(l)
regressdf.loc["ALL YRS"].iloc[0] = sum(smallsuc)/regressdf.loc["ALL YRS"].iloc[3]
regressdf.loc["ALL YRS"].iloc[1] = sum(medsuc)/regressdf.loc["ALL YRS"].iloc[4]
regressdf.loc["ALL YRS"].iloc[2] = sum(largesuc)/regressdf.loc["ALL YRS"].iloc[5]


for x in range(len(yr)):
    sample1_phat = regressdf.loc["ALL YRS"].iloc[0]
    sample2_phat = regressdf.loc["ALL YRS"].iloc[1]
    sample1_size = regressdf.loc["ALL YRS"].iloc[3]
    sample2_size = regressdf.loc["ALL YRS"].iloc[4]
    phat = (sample1_phat*sample1_size+sample2_phat*sample2_size)/(sample1_size+sample2_size)
    zscore = (sample1_phat - sample2_phat)/np.sqrt(phat*(1-phat)*((1/sample1_size)+(1/sample2_size)))    
    pval = st.norm.sf(abs(zscore))*2
regressdf.loc["ALL YRS"].iloc[6] = pval

for x in range(len(yr)):
    sample1_phat = regressdf.loc["ALL YRS"].iloc[0]
    sample2_phat = regressdf.loc["ALL YRS"].iloc[2]
    sample1_size = regressdf.loc["ALL YRS"].iloc[3]
    sample2_size = regressdf.loc["ALL YRS"].iloc[5]
    phat = (sample1_phat*sample1_size+sample2_phat*sample2_size)/(sample1_size+sample2_size)
    zscore = (sample1_phat - sample2_phat)/np.sqrt(phat*(1-phat)*((1/sample1_size)+(1/sample2_size)))    
    pval = st.norm.sf(abs(zscore))*2
regressdf.loc["ALL YRS"].iloc[7] = pval

for x in range(len(yr)):
    sample1_phat = regressdf.loc["ALL YRS"].iloc[1]
    sample2_phat = regressdf.loc["ALL YRS"].iloc[2]
    sample1_size = regressdf.loc["ALL YRS"].iloc[4]
    sample2_size = regressdf.loc["ALL YRS"].iloc[5]
    phat = (sample1_phat*sample1_size+sample2_phat*sample2_size)/(sample1_size+sample2_size)
    zscore = (sample1_phat - sample2_phat)/np.sqrt(phat*(1-phat)*((1/sample1_size)+(1/sample2_size)))    
    pval = st.norm.sf(abs(zscore))*2
regressdf.loc["ALL YRS"].iloc[8] = pval
regressdf = regressdf.head(9).style.format("{:,.2f}")
regressdf

In [ ]:
# CREATING A NEW COLUMN: SIZE OF COMPANY
df["EMPLOYER_SIZE"] = np.nan
df["EMPLOYER_SIZE"] = np.where(df["EMPLOYER_NUM_EMPLOYEES"]<1000,"SMALL",df["EMPLOYER_SIZE"])
df["EMPLOYER_SIZE"] = np.where((df["EMPLOYER_NUM_EMPLOYEES"]>=1000)&(df["EMPLOYER_NUM_EMPLOYEES"]<10000),"MEDIUM",df["EMPLOYER_SIZE"])
df["EMPLOYER_SIZE"] = np.where(df["EMPLOYER_NUM_EMPLOYEES"]>=10000,"LARGE",df["EMPLOYER_SIZE"])

In [ ]:
# DISTRIBUTION OF YEAR COMMENCED
# PLOT THE DIST OF YEAR COMMENCED
plt.figure(figsize = (12,10))
empcount = sns.histplot(x = df["EMPLOYER_YEAR_COMMENCED_BUSINESS"], bins = 100)
plt.tight_layout()

In [ ]:
# CORRELATION ANALYSIS: EMPLOYER AGE WITH CASE STATUS: CASE BY CASE
st.linregress(x = df.dropna()["EMPLOYER_YEAR_COMMENCED_BUSINESS"], y = df.dropna()["CASE_STATUS"])
# DOES NOT SHOW MUCH, AGGREGATE MAY SHOW A DIFFERENT STORY

In [ ]:
# NEW COLUMN: BREAKING UP THE COMPANIES BY AGE
df["EMPLOYER_AGE"] = np.nan
df["EMPLOYER_AGE"] = np.where((df["YEAR"]-df["EMPLOYER_YEAR_COMMENCED_BUSINESS"])<15,"YOUNG",df["EMPLOYER_AGE"])
df["EMPLOYER_AGE"] = np.where(((df["YEAR"]-df["EMPLOYER_YEAR_COMMENCED_BUSINESS"])>=15)&((df["YEAR"]-df["EMPLOYER_YEAR_COMMENCED_BUSINESS"])<30),"MEDIUM",df["EMPLOYER_AGE"])
df["EMPLOYER_AGE"] = np.where((df["YEAR"]-df["EMPLOYER_YEAR_COMMENCED_BUSINESS"])>=30,"OLD",df["EMPLOYER_AGE"])

In [ ]:
# CORRELATION ANALYSIS: AGGREGATE BY SIZE BRACKETS AND YEARLY
# PLOTTING SUCCESS RATES OVER TIME BY SIZE OF COMPANY

young = df[df["EMPLOYER_AGE"]=="YOUNG"]
mid = df[df["EMPLOYER_AGE"]=="MEDIUM"]
old = df[df["EMPLOYER_AGE"]=="OLD"]


youndf = pd.DataFrame()
middf = pd.DataFrame()
olddf = pd.DataFrame()
linregdf = pd.DataFrame()

for x in range(len(yr)):
    j = pd.DataFrame([len(young[(young["YEAR"]==yr[x])&(young["CASE_STATUS"]==1)])/len(young[young["YEAR"]==yr[x]])])
    med = pd.DataFrame([len(mid[(mid["YEAR"]==yr[x])&(mid["CASE_STATUS"]==1)])/len(mid[mid["YEAR"]==yr[x]])])
    oldy = pd.DataFrame([len(old[(old["YEAR"]==yr[x])&(old["CASE_STATUS"]==1)])/len(old[old["YEAR"]==yr[x]])])
    youndf = pd.concat([youndf,j])
    middf = pd.concat([middf,med])
    olddf = pd.concat([olddf,oldy])
    linregdf = pd.concat([youndf,middf,olddf], axis = 1)
    

linregdf.index = yr
linregdf.columns = ["Young Success %", "Medium Success %", "Old Success %"]
plt.figure(figsize = (12,10), dpi = 100)
linregdf.plot(kind = "bar")
plt.tight_layout()
plt.legend(bbox_to_anchor=(1.05, 1), loc=2, borderaxespad=0.)
plt.title("Success Rates by Company Age, Over Time")

In [ ]:
# PREPARING DATA FOR LIN REGRESS AND GETTING P VALUE
youngapps = []
midapps = []
oldapps = []

for x in range(len(yr)):
    y = (young["YEAR"]==yr[x]).sum()
    m = (mid["YEAR"]==yr[x]).sum()
    o = (old["YEAR"]==yr[x]).sum()
    youngapps.append(y)
    midapps.append(m)
    oldapps.append(o)
linregdf = linregdf.reset_index()
linregdf["Young Apps"] = pd.DataFrame(youngapps)
linregdf["Medium Apps"] = pd.DataFrame(midapps)
linregdf["Old Apps"] = pd.DataFrame(oldapps)
linregdf = linregdf.set_index("index")
# PLOTTING APPLICATION RATES OVER TIME BY SIZE OF COMPANY
plt.figure(figsize = (12,10), dpi = 100)
linregdf.reset_index().plot(x = "index", y = ["Young Apps","Medium Apps","Old Apps"], kind = "bar")
plt.tight_layout()
plt.legend(bbox_to_anchor=(1.05, 1), loc=2, borderaxespad=0.)
plt.title("Application Rates by Company Age, Over Time")

In [ ]:
# P VALUE PER YEAR
young_med = []
for x in range(len(yr)):
    sample1_phat = linregdf.loc[yr[x]]["Young Success %"]
    sample2_phat = linregdf.loc[yr[x]]["Medium Success %"]
    sample1_size = linregdf.loc[yr[x]]["Young Apps"]
    sample2_size = linregdf.loc[yr[x]]["Medium Apps"]
    phat = (sample1_phat*sample1_size+sample2_phat*sample2_size)/(sample1_size+sample2_size)
    zscore = (sample1_phat - sample2_phat)/np.sqrt(phat*(1-phat)*((1/sample1_size)+(1/sample2_size)))    
    pval = st.norm.sf(abs(zscore))*2
    young_med.append(pval)

pvaldf = pd.DataFrame(young_med)

young_old = []
for x in range(len(yr)):
    sample1_phat = linregdf.loc[yr[x]]["Young Success %"]
    sample2_phat = linregdf.loc[yr[x]]["Old Success %"]
    sample1_size = linregdf.loc[yr[x]]["Young Apps"]
    sample2_size = linregdf.loc[yr[x]]["Old Apps"]
    phat = (sample1_phat*sample1_size+sample2_phat*sample2_size)/(sample1_size+sample2_size)
    zscore = (sample1_phat - sample2_phat)/np.sqrt(phat*(1-phat)*((1/sample1_size)+(1/sample2_size)))    
    pval = st.norm.sf(abs(zscore))*2
    young_old.append(pval)
pvaldf = pd.concat([pvaldf,pd.DataFrame(young_old)], axis = 1)

med_old = []
for x in range(len(yr)):
    sample1_phat = linregdf.loc[yr[x]]["Medium Success %"]
    sample2_phat = linregdf.loc[yr[x]]["Old Success %"]
    sample1_size = linregdf.loc[yr[x]]["Medium Apps"]
    sample2_size = linregdf.loc[yr[x]]["Old Apps"]
    phat = (sample1_phat*sample1_size+sample2_phat*sample2_size)/(sample1_size+sample2_size)
    zscore = (sample1_phat - sample2_phat)/np.sqrt(phat*(1-phat)*((1/sample1_size)+(1/sample2_size)))    
    pval = st.norm.sf(abs(zscore))*2
    med_old.append(pval)
pvaldf = pd.concat([pvaldf,pd.DataFrame(med_old)], axis = 1)
pvaldf.columns = ["Young_Med","Young_Old","Med_Old"]
pvaldf.index = yr
linregdf[["Young_Med","Young_Old","Med_Old"]] = pvaldf[["Young_Med","Young_Old","Med_Old"]]
linregdf

In [ ]:
# LINEAR REGRESSION BETWEEN COMPANY AGE AND SUCCESS RATE OVER TIME: YOUNG COMPANY
print("Young Company Success Rate Over Time Slope:",round(st.linregress(y = linregdf["Young Success %"], x = yr).slope,6),", P Val:",round(st.linregress(y = linregdf["Young Success %"], x = yr).pvalue,4),", R Val:",round(st.linregress(y = linregdf["Young Success %"], x = yr).rvalue,4),"\nWeak growth, .25% growth in success rate a year, and not a statistically significant slope.")
# LINEAR REGRESSION BETWEEN COMPANY AGE AND APPLICATIONS OVER TIME: YOUNG COMPANY
print("\nYoung Company Application Rate Over Time Slope:",round(st.linregress(y = linregdf["Young Apps"], x = yr).slope,3),", P Val:",round(st.linregress(y = linregdf["Young Apps"], x = yr).pvalue,4),", R Val:",round(st.linregress(y = linregdf["Young Apps"], x = yr).rvalue,4),"\nWeak growth, 531 applications a year, and not a statistically significant slope.")

In [ ]:
# LINEAR REGRESSION BETWEEN COMPANY AGE AND SUCCESS RATE OVER TIME: MEDIUM COMPANY
print("Medium Company Success Rate Over Time Slope:",round(st.linregress(y = linregdf["Medium Success %"], x = yr).slope,6),", P Val:",round(st.linregress(y = linregdf["Medium Success %"], x = yr).pvalue,4),", R Val:",round(st.linregress(y = linregdf["Medium Success %"], x = yr).rvalue,4),"\Moderate growth, .29% growth in success rate a year, and not a statistically significant slope.")
# LINEAR REGRESSION BETWEEN COMPANY AGE AND APPLICATIONS OVER TIME: MEDIUM COMPANY
print("\nMedium Company Application Rate Over Time Slope:",round(st.linregress(y = linregdf["Medium Apps"], x = yr).slope,3),", P Val:",round(st.linregress(y = linregdf["Medium Apps"], x = yr).pvalue,4),", R Val:",round(st.linregress(y = linregdf["Medium Apps"], x = yr).rvalue,4),"\Moderate growth, 1786 applications a year, and not a statistically significant slope.")

In [ ]:
# LINEAR REGRESSION BETWEEN COMPANY AGE AND SUCCESS RATE OVER TIME: OLD COMPANY
print("Old Company Success Rate Over Time Slope:",round(st.linregress(y = linregdf["Old Success %"], x = yr).slope,6),", P Val:",round(st.linregress(y = linregdf["Old Success %"], x = yr).pvalue,4),", R Val:",round(st.linregress(y = linregdf["Old Success %"], x = yr).rvalue,4),"\nVery strong growth, 1% growth in success rate a year, and a statistically significant slope.")
# LINEAR REGRESSION BETWEEN COMPANY AGE AND APPLICATIONS OVER TIME: OLD COMPANY
print("\nOld Company Application Rate Over Time Slope:",round(st.linregress(y = linregdf["Old Apps"], x = yr).slope,3),", P Val:",round(st.linregress(y = linregdf["Old Apps"], x = yr).pvalue,4),", R Val:",round(st.linregress(y = linregdf["Old Apps"], x = yr).rvalue,4),"\nModerate growth, 2578 applications a year, and not a statistically significant slope.")

In [ ]:
# GETTING THE TOTAL PVAL
linregdf.loc["ALL YRS"] = linregdf.sum()
youngsuc = []
medsuc = []
oldsuc = []
for x in range(len(yr)):
    y = linregdf.loc[yr[x]].iloc[0]*linregdf.loc[yr[x]].iloc[3]
    m = linregdf.loc[yr[x]].iloc[1]*linregdf.loc[yr[x]].iloc[4]
    o = linregdf.loc[yr[x]].iloc[2]*linregdf.loc[yr[x]].iloc[5]
    youngsuc.append(y)
    medsuc.append(m)
    oldsuc.append(o)
linregdf.loc["ALL YRS"].iloc[0] = sum(youngsuc)/linregdf.loc["ALL YRS"].iloc[3]
linregdf.loc["ALL YRS"].iloc[1] = sum(medsuc)/linregdf.loc["ALL YRS"].iloc[4]
linregdf.loc["ALL YRS"].iloc[2] = sum(oldsuc)/linregdf.loc["ALL YRS"].iloc[5]


for x in range(len(yr)):
    sample1_phat = linregdf.loc["ALL YRS"].iloc[0]
    sample2_phat = linregdf.loc["ALL YRS"].iloc[1]
    sample1_size = linregdf.loc["ALL YRS"].iloc[3]
    sample2_size = linregdf.loc["ALL YRS"].iloc[4]
    phat = (sample1_phat*sample1_size+sample2_phat*sample2_size)/(sample1_size+sample2_size)
    zscore = (sample1_phat - sample2_phat)/np.sqrt(phat*(1-phat)*((1/sample1_size)+(1/sample2_size)))    
    pval = st.norm.sf(abs(zscore))*2
linregdf.loc["ALL YRS"].iloc[6] = pval

for x in range(len(yr)):
    sample1_phat = linregdf.loc["ALL YRS"].iloc[0]
    sample2_phat = linregdf.loc["ALL YRS"].iloc[2]
    sample1_size = linregdf.loc["ALL YRS"].iloc[3]
    sample2_size = linregdf.loc["ALL YRS"].iloc[5]
    phat = (sample1_phat*sample1_size+sample2_phat*sample2_size)/(sample1_size+sample2_size)
    zscore = (sample1_phat - sample2_phat)/np.sqrt(phat*(1-phat)*((1/sample1_size)+(1/sample2_size)))    
    pval = st.norm.sf(abs(zscore))*2
linregdf.loc["ALL YRS"].iloc[7] = pval

for x in range(len(yr)):
    sample1_phat = linregdf.loc["ALL YRS"].iloc[1]
    sample2_phat = linregdf.loc["ALL YRS"].iloc[2]
    sample1_size = linregdf.loc["ALL YRS"].iloc[4]
    sample2_size = linregdf.loc["ALL YRS"].iloc[5]
    phat = (sample1_phat*sample1_size+sample2_phat*sample2_size)/(sample1_size+sample2_size)
    zscore = (sample1_phat - sample2_phat)/np.sqrt(phat*(1-phat)*((1/sample1_size)+(1/sample2_size)))    
    pval = st.norm.sf(abs(zscore))*2
linregdf.loc["ALL YRS"].iloc[8] = pval
linregdf = linregdf.head(9).style.format("{:,.2f}")
linregdf

In [ ]:
# Top 15 NAICS of all time 2014-2021
allcode = pd.DataFrame(df["NAICS_CODE"].value_counts()[:15]).reset_index()
allcode.columns = ["ACode","COUNT ACode"]
succode = pd.DataFrame(df[df["CASE_STATUS"]==1]["NAICS_CODE"].value_counts()[:15]).reset_index()
succode.columns = ["SCode","COUNT SCode"]
pd.concat([allcode,succode], axis = 1)

In [ ]:
# Grabbing the top 15 for each year codes: only succesful applications
top15scode = pd.DataFrame()
def grab (year):
    return df[(df["YEAR"]==year)&(df["CASE_STATUS"]==1)]["NAICS_CODE"].value_counts()[:15]
for x in range(len(yr)):
    j = pd.DataFrame(grab(yr[x]))
    top15scode = pd.concat([top15scode,j], axis = 1)
top15scode.columns= yr
# King of Consistency: HOW MANY YEARS IS THIS CODE NOT IN THE TOP 15?: TOP 10 CONSISTENT CODES...
kingcons = top15scode.isnull().replace(False,0).replace(True,1)
kingcons["Yrs not in Top 15"] = top15scode.isnull().replace(False,0).replace(True,1).sum(axis = 1)
kingcons["Yrs not in Top 15"].sort_values()[:10]

In [ ]:
# Grabbing the top 15 for each year codes: total applications
top15acode = pd.DataFrame()
def grab (year):
    return df[df["YEAR"]==year]["NAICS_CODE"].value_counts()[:15]
for x in range(len(yr)):
    j = pd.DataFrame(grab(yr[x]))
    top15acode = pd.concat([top15acode,j], axis = 1)
top15acode.columns= yr
# King of Consistency: HOW MANY YEARS IS THIS CODE NOT IN THE TOP 15?: TOP 10 CONSISTENT CODES...
kingconsA = top15acode.isnull().replace(False,0).replace(True,1)
kingconsA["Yrs not in Top 15"] = top15acode.isnull().replace(False,0).replace(True,1).sum(axis = 1)
kingconsA["Yrs not in Top 15"].sort_values()[:10]

In [ ]:
# DIVING DEEPER INTO THE TOP CODES!
#ANCHORED BY STATE: PW SKILL,APPLICATIONS, MEDIAN WAGE, YRS POST GRADE, MEDIAN DAYS

code = "541511"
# CHANGE THIS FOR ANY CODE

# MEDIAN WAGE, MEDIAN YEARS AFTER GRADUATING, MEDIAN DAYS TO DECIDE : FOR EACH STATE IN THE TOP 15 OF THE COMPANY AND BY PW LEVEL
topstate = df[df["NAICS_CODE"] == code]["WORKSITE_STATE"].value_counts()[:15].index.tolist()
pwskill = df["PW_SKILL_LEVEL"].dropna().unique().tolist()
pwskill.sort()


statelist = []
pwlist = pwskill*len(topstate)
apps = []
medwage = []
yrsgrad = []
meddays = []


for x in range(len(topstate)):
    s = (topstate[x]+",")*len(pwskill)
    s = "".join(s).split(",")
    s = list(filter(("").__ne__, s))
    statelist.append(s) 
for x in range(len(topstate)):
    for j in range(len(pwskill)):
        a = len(df[(df["NAICS_CODE"] == code)&(df["WORKSITE_STATE"] == topstate[x])][df[(df["NAICS_CODE"] == code)&(df["WORKSITE_STATE"] == topstate[x])]["PW_SKILL_LEVEL"]==pwskill[j]])
        apps.append(a)
        b = df[(df["NAICS_CODE"] == code)&(df["WORKSITE_STATE"] == topstate[x])][df[(df["NAICS_CODE"] == code)&(df["WORKSITE_STATE"] == topstate[x])]["PW_SKILL_LEVEL"]==pwskill[j]]["WO_A"].median()
        medwage.append(b)
        c = df[(df["NAICS_CODE"] == code)&(df["WORKSITE_STATE"] == topstate[x])][df[(df["NAICS_CODE"] == code)&(df["WORKSITE_STATE"] == topstate[x])]["PW_SKILL_LEVEL"]==pwskill[j]]["YRSPOSTGRAD"].median()
        yrsgrad.append(c)
        d = df[(df["NAICS_CODE"] == code)&(df["WORKSITE_STATE"] == topstate[x])][df[(df["NAICS_CODE"] == code)&(df["WORKSITE_STATE"] == topstate[x])]["PW_SKILL_LEVEL"]==pwskill[j]]["DAYS_TO_DECIDE"].median()
        meddays.append(d)
codestatedf = pd.DataFrame([sum(statelist, []),pwlist,apps,medwage,yrsgrad,meddays]).transpose()
codestatedf.columns = ["STATE","PW LEVEL", "TOTAL APPLICATIONS","MEDIAN WAGE","MEDIAN YRS POST GRAD","MEDIAN DAYS TO DECIDE"]
codestatedf = codestatedf.set_index(["STATE","PW LEVEL"])
codestatedf

In [ ]:
# ANCHORED BY STATE: EDU LEVEL,APPLICATIONS, MEDIAN WAGE, YRS POST GRADE, MEDIAN DAYS
# MEDIAN WAGE, MEDIAN YEARS AFTER GRADUATING, MEDIAN DAYS TO DECIDE : FOR EACH STATE IN THE TOP 15 OF THE COMPANY AND BY EDU LEVEL
edu = df["FOREIGN_WORKER_EDUCATION"].dropna().unique().tolist()
edu[0],edu[1],edu[2],edu[3],edu[4],edu[5] = edu[3],edu[5],edu[0],edu[2],edu[1],edu[4]

statelist = []
edulist = edu*len(topstate)
apps = []
medwage = []
yrsgrad = []
meddays = []


for x in range(len(topstate)):
    s = (topstate[x]+",")*len(edu)
    s = "".join(s).split(",")
    s = list(filter(("").__ne__, s))
    statelist.append(s) 
for x in range(len(topstate)):
    for j in range(len(edu)):
        a = len(df[(df["NAICS_CODE"] == code)&(df["WORKSITE_STATE"] == topstate[x])][df[(df["NAICS_CODE"] == code)&(df["WORKSITE_STATE"] == topstate[x])]["FOREIGN_WORKER_EDUCATION"]==edu[j]])
        apps.append(a)
        b = df[(df["NAICS_CODE"] == code)&(df["WORKSITE_STATE"] == topstate[x])][df[(df["NAICS_CODE"] == code)&(df["WORKSITE_STATE"] == topstate[x])]["FOREIGN_WORKER_EDUCATION"]==edu[j]]["WO_A"].median()
        medwage.append(b)
        c = df[(df["NAICS_CODE"] == code)&(df["WORKSITE_STATE"] == topstate[x])][df[(df["NAICS_CODE"] == code)&(df["WORKSITE_STATE"] == topstate[x])]["FOREIGN_WORKER_EDUCATION"]==edu[j]]["YRSPOSTGRAD"].median()
        yrsgrad.append(c)
        d = df[(df["NAICS_CODE"] == code)&(df["WORKSITE_STATE"] == topstate[x])][df[(df["NAICS_CODE"] == code)&(df["WORKSITE_STATE"] == topstate[x])]["FOREIGN_WORKER_EDUCATION"]==edu[j]]["DAYS_TO_DECIDE"].median()
        meddays.append(d)
codestatedf1 = pd.DataFrame([sum(statelist, []),edulist,apps,medwage,yrsgrad,meddays]).transpose()
codestatedf1.columns = ["STATE","EDU LEVEL", "TOTAL APPLICATIONS","MEDIAN WAGE","MEDIAN YRS POST GRAD","MEDIAN DAYS TO DECIDE"]
codestatedf1 = codestatedf1.set_index(["STATE","EDU LEVEL"])
codestatedf1

In [ ]:
# NEW COLUMN: 1 is in the top 15 of the YEAR, 0 is not: Applications
filler = pd.DataFrame()
for x in range(len(yr)):
    listy = top15acode[yr[x]].dropna().index.tolist()
    filler = pd.concat([filler,pd.DataFrame((df[df["YEAR"]==yr[x]]["NAICS_CODE"].isin(listy)).replace(True,1).replace(False,0))])

filler = filler.reset_index().sort_values(by = "index").set_index("index")
filler.columns = ["TOP15CODE"]
df["TOP15CODE"] = filler

In [ ]:
# CORRELATION ANALYSIS: NUMBER OF EMPLOYEEs WITH CASE STATUS: CASE BY CASE
st.linregress(x = df.dropna()["TOP15CODE"], y = df.dropna()["CASE_STATUS"])
# DOES NOT SHOW MUCH, AGGREGATE MAY SHOW A DIFFERENT STORY

In [ ]:
# PLOTTING THE DIFFERENT SUCCESS RATES OVER TIME BY TOP 15 COMPANY (IN TERMS OF ANNUAL APPLICATIONS)
top15 = df[df["TOP15CODE"]==1]
not15 = df[df["TOP15CODE"]==0]

topdf = pd.DataFrame()
notdf = pd.DataFrame()
lanalysisdf = pd.DataFrame()

for x in range(len(yr)):
    j = pd.DataFrame([len(top15[(top15["YEAR"]==yr[x])&(top15["CASE_STATUS"]==1)])/len(top15[top15["YEAR"]==yr[x]])])
    b = pd.DataFrame([len(not15[(not15["YEAR"]==yr[x])&(not15["CASE_STATUS"]==1)])/len(not15[not15["YEAR"]==yr[x]])])
    topdf = pd.concat([topdf,j])
    notdf = pd.concat([notdf,b])
    lanalysisdf = pd.concat([topdf,notdf], axis = 1)

lanalysisdf.index = yr
lanalysisdf.columns = ["Top 15 Success %", "Not 15 Success %"]
plt.figure(figsize = (12,10), dpi = 100)
lanalysisdf.plot(kind = "bar")
plt.tight_layout()
plt.legend(bbox_to_anchor=(1.05, 1), loc=2, borderaxespad=0.)
plt.title("Success Rates by Companies Codes in the Top 15 and not in the Top 15, Over Time")


In [ ]:
# PREPARING DATA FOR LIN REGRESS AND GETTING P VALUE
top15apps = []
not15apps = []

for x in range(len(yr)):
    t = (top15["YEAR"]==yr[x]).sum()
    n = (not15["YEAR"]==yr[x]).sum()
    top15apps.append(t)
    not15apps.append(n)
lanalysisdf = lanalysisdf.reset_index()
lanalysisdf["Top 15 Apps"] = pd.DataFrame(top15apps)
lanalysisdf["Not 15 Apps"] = pd.DataFrame(not15apps)
lanalysisdf = lanalysisdf.set_index("index")
# PLOTTING APPLICATION RATES OVER TIME BY TOP 15 or NOT TOP 15
plt.figure(figsize = (12,10), dpi = 100)
lanalysisdf.reset_index().plot(x = "index", y = ["Top 15 Apps","Not 15 Apps"], kind = "bar")
plt.tight_layout()
plt.legend(bbox_to_anchor=(1.05, 1), loc=2, borderaxespad=0.)
plt.title("Application Rates by Companies Codes in the Top 15 and not in the Top 15, Over Time")


In [ ]:
# P VALUE PER YEAR: ON THE SUCCESS RATE
top_not = []
for x in range(len(yr)):
    sample1_phat = lanalysisdf.loc[yr[x]]["Top 15 Success %"]
    sample2_phat = lanalysisdf.loc[yr[x]]["Not 15 Success %"]
    sample1_size = lanalysisdf.loc[yr[x]]["Top 15 Apps"]
    sample2_size = lanalysisdf.loc[yr[x]]["Not 15 Apps"]
    phat = (sample1_phat*sample1_size+sample2_phat*sample2_size)/(sample1_size+sample2_size)
    zscore = (sample1_phat - sample2_phat)/np.sqrt(phat*(1-phat)*((1/sample1_size)+(1/sample2_size)))    
    pval = st.norm.sf(abs(zscore))*2
    top_not.append(pval)

pvaldf = pd.DataFrame(top_not)
pvaldf.columns = ["Top 15 Vs. Not Top 15"]
pvaldf.index = yr
lanalysisdf["PVAL Success %"] = pvaldf["Top 15 Vs. Not Top 15"]

In [ ]:
# LINEAR REGRESSION BETWEEN COMPANY SIZE AND SUCCESS RATE OVER TIME: TOP 15 COMPANY
print("Top 15 Company Codes Success Rate Over Time Slope:",round(st.linregress(y = lanalysisdf["Top 15 Success %"], x = yr).slope,6),", P Val:",round(st.linregress(y = lanalysisdf["Top 15 Success %"], x = yr).pvalue,4),", R Val:",round(st.linregress(y = lanalysisdf["Top 15 Success %"], x = yr).rvalue,4),"\nStrong growth, .75% growth in success rate a year, and a statistically significant slope.")
# LINEAR REGRESSION BETWEEN COMPANY SIZE AND APPLICATIONS OVER TIME: TOP 15 COMPANY
print("\nTop 15 Company Codes Application Rate Over Time Slope:",round(st.linregress(y = lanalysisdf["Top 15 Apps"], x = yr).slope,3),", P Val:",round(st.linregress(y = lanalysisdf["Top 15 Apps"], x = yr).pvalue,4),", R Val:",round(st.linregress(y = lanalysisdf["Top 15 Apps"], x = yr).rvalue,4),"\nModerate growth, 2068 applications a year, and not a statistically significant slope.")

# LINEAR REGRESSION BETWEEN COMPANY SIZE AND SUCCESS RATE OVER TIME: NOT TOP 15 COMPANY
print("\n\nNot Top 15 Company Codes Success Rate Over Time Slope:",round(st.linregress(y = lanalysisdf["Not 15 Success %"], x = yr).slope,6),", P Val:",round(st.linregress(y = lanalysisdf["Not 15 Success %"], x = yr).pvalue,4),", R Val:",round(st.linregress(y = lanalysisdf["Not 15 Success %"], x = yr).rvalue,4),"\nStrong growth, .47% growth in success rate a year, and a statistically significant slope.")
# LINEAR REGRESSION BETWEEN COMPANY SIZE AND APPLICATIONS OVER TIME: NOT TOP 15 COMPANY
print("\nNot Top 15 Company Codes Application Rate Over Time Slope:",round(st.linregress(y = lanalysisdf["Not 15 Apps"], x = yr).slope,3),", P Val:",round(st.linregress(y = lanalysisdf["Not 15 Apps"], x = yr).pvalue,4),", R Val:",round(st.linregress(y = lanalysisdf["Not 15 Apps"], x = yr).rvalue,4),"\nWeak growth, 2828 applications a year, but not a statistically significant slope.")


In [ ]:
# GETTING THE TOTAL PVAL
lanalysisdf.loc["ALL YRS"] = lanalysisdf.sum()
top15suc = []
not15suc = []

for x in range(len(yr)):
    t15 = lanalysisdf.loc[yr[x]].iloc[0]*lanalysisdf.loc[yr[x]].iloc[2]
    n15 = lanalysisdf.loc[yr[x]].iloc[1]*lanalysisdf.loc[yr[x]].iloc[3]
    top15suc.append(t15)
    not15suc.append(n15)
lanalysisdf.loc["ALL YRS"].iloc[0] = sum(top15suc)/lanalysisdf.loc["ALL YRS"].iloc[2]
lanalysisdf.loc["ALL YRS"].iloc[1] = sum(not15suc)/lanalysisdf.loc["ALL YRS"].iloc[3]


for x in range(len(yr)):
    sample1_phat = lanalysisdf.loc["ALL YRS"].iloc[0]
    sample2_phat = lanalysisdf.loc["ALL YRS"].iloc[1]
    sample1_size = lanalysisdf.loc["ALL YRS"].iloc[2]
    sample2_size = lanalysisdf.loc["ALL YRS"].iloc[3]
    phat = (sample1_phat*sample1_size+sample2_phat*sample2_size)/(sample1_size+sample2_size)
    zscore = (sample1_phat - sample2_phat)/np.sqrt(phat*(1-phat)*((1/sample1_size)+(1/sample2_size)))    
    pval = st.norm.sf(abs(zscore))*2
lanalysisdf.loc["ALL YRS"].iloc[4] = pval

lanalysisdf = lanalysisdf.head(9).style.format("{:,.2f}")
lanalysisdf

In [ ]:
#### APPLICATION / JOB RELATED #####

In [ ]:
# GROUPBY YEAR AND PLOT THE .25, .5, and .75 WO_A
wages = pd.DataFrame([df.groupby("YEAR")["WO_A"].quantile(.25),df.groupby("YEAR")["WO_A"].quantile(.5),
 df.groupby("YEAR")["WO_A"].quantile(.75)]).transpose()
wages.columns = [".25",".5",".75"]
wages.plot(kind = "bar")
plt.tight_layout()
plt.legend(bbox_to_anchor=(1.05, 1), loc=2, borderaxespad=0.)
plt.title("Annual Wages by Percentiles, Over Time")

In [ ]:
# CORRELATION ANALYSIS: Wage offered and success
st.linregress(x = df.dropna()["WO_A"], y = df.dropna()["CASE_STATUS"])
# DOES NOT SHOW MUCH, AGGREGATE MAY SHOW A DIFFERENT STORY

In [ ]:
# CREATING A NEW COLUMN: WAGE GROUP, SPLITS WAGES INTO THIRDS
high = df.groupby("YEAR")["WO_A"].quantile(2/3).tolist()
low = df.groupby("YEAR")["WO_A"].quantile(1/3).tolist()
df["WAGE_GROUP"] = np.nan
for x in range(len(yr)):
    df["WAGE_GROUP"] = np.where((df["YEAR"]==yr[x])&(df["WO_A"]<low[x]),"LOW",df["WAGE_GROUP"])
    df["WAGE_GROUP"] = np.where((df["YEAR"]==yr[x])&(df["WO_A"]>=low[x])&(df["WO_A"]<high[x]),"MED",df["WAGE_GROUP"])
    df["WAGE_GROUP"] = np.where((df["YEAR"]==yr[x])&(df["WO_A"]>=high[x]),"HIGH",df["WAGE_GROUP"])

In [ ]:
# CORRELATION ANALYSIS: AGGREGATE BY WAGE GROUP
# PLOTTING SUCCESS RATES OVER TIME BY WAGE GROUP

young = df[df["WAGE_GROUP"]=="LOW"]
mid = df[df["WAGE_GROUP"]=="MED"]
old = df[df["WAGE_GROUP"]=="HIGH"]


youndf = pd.DataFrame()
middf = pd.DataFrame()
olddf = pd.DataFrame()
linregdf = pd.DataFrame()

for x in range(len(yr)):
    j = pd.DataFrame([len(young[(young["YEAR"]==yr[x])&(young["CASE_STATUS"]==1)])/len(young[young["YEAR"]==yr[x]])])
    med = pd.DataFrame([len(mid[(mid["YEAR"]==yr[x])&(mid["CASE_STATUS"]==1)])/len(mid[mid["YEAR"]==yr[x]])])
    oldy = pd.DataFrame([len(old[(old["YEAR"]==yr[x])&(old["CASE_STATUS"]==1)])/len(old[old["YEAR"]==yr[x]])])
    youndf = pd.concat([youndf,j])
    middf = pd.concat([middf,med])
    olddf = pd.concat([olddf,oldy])
    linregdf = pd.concat([youndf,middf,olddf], axis = 1)
    

linregdf.index = yr
linregdf.columns = ["Low Success %", "Medium Success %", "High Success %"]
plt.figure(figsize = (12,10), dpi = 100)
linregdf.plot(kind = "bar")
plt.tight_layout()
plt.legend(bbox_to_anchor=(1.05, 1), loc=2, borderaxespad=0.)
plt.title("Success Rates by Wage Bracket, Over Time")

In [ ]:
# PREPARING DATA FOR LIN REGRESS AND GETTING P VALUE
youngapps = []
midapps = []
oldapps = []

for x in range(len(yr)):
    y = (young["YEAR"]==yr[x]).sum()
    m = (mid["YEAR"]==yr[x]).sum()
    o = (old["YEAR"]==yr[x]).sum()
    youngapps.append(y)
    midapps.append(m)
    oldapps.append(o)
linregdf = linregdf.reset_index()
linregdf["Low Apps"] = pd.DataFrame(youngapps)
linregdf["Medium Apps"] = pd.DataFrame(midapps)
linregdf["High Apps"] = pd.DataFrame(oldapps)
linregdf = linregdf.set_index("index")

# P VALUE PER YEAR
young_med = []
for x in range(len(yr)):
    sample1_phat = linregdf.loc[yr[x]]["Low Success %"]
    sample2_phat = linregdf.loc[yr[x]]["Medium Success %"]
    sample1_size = linregdf.loc[yr[x]]["Low Apps"]
    sample2_size = linregdf.loc[yr[x]]["Medium Apps"]
    phat = (sample1_phat*sample1_size+sample2_phat*sample2_size)/(sample1_size+sample2_size)
    zscore = (sample1_phat - sample2_phat)/np.sqrt(phat*(1-phat)*((1/sample1_size)+(1/sample2_size)))    
    pval = st.norm.sf(abs(zscore))*2
    young_med.append(pval)

pvaldf = pd.DataFrame(young_med)

young_old = []
for x in range(len(yr)):
    sample1_phat = linregdf.loc[yr[x]]["Low Success %"]
    sample2_phat = linregdf.loc[yr[x]]["High Success %"]
    sample1_size = linregdf.loc[yr[x]]["Low Apps"]
    sample2_size = linregdf.loc[yr[x]]["High Apps"]
    phat = (sample1_phat*sample1_size+sample2_phat*sample2_size)/(sample1_size+sample2_size)
    zscore = (sample1_phat - sample2_phat)/np.sqrt(phat*(1-phat)*((1/sample1_size)+(1/sample2_size)))    
    pval = st.norm.sf(abs(zscore))*2
    young_old.append(pval)
pvaldf = pd.concat([pvaldf,pd.DataFrame(young_old)], axis = 1)

med_old = []
for x in range(len(yr)):
    sample1_phat = linregdf.loc[yr[x]]["Medium Success %"]
    sample2_phat = linregdf.loc[yr[x]]["High Success %"]
    sample1_size = linregdf.loc[yr[x]]["Medium Apps"]
    sample2_size = linregdf.loc[yr[x]]["High Apps"]
    phat = (sample1_phat*sample1_size+sample2_phat*sample2_size)/(sample1_size+sample2_size)
    zscore = (sample1_phat - sample2_phat)/np.sqrt(phat*(1-phat)*((1/sample1_size)+(1/sample2_size)))    
    pval = st.norm.sf(abs(zscore))*2
    med_old.append(pval)
pvaldf = pd.concat([pvaldf,pd.DataFrame(med_old)], axis = 1)
pvaldf.columns = ["Low_Med","Low_High","Med_High"]
pvaldf.index = yr
linregdf[["Low_Med","Low_High","Med_High"]] = pvaldf[["Low_Med","Low_High","Med_High"]]

In [ ]:
# LINEAR REGRESSION BETWEEN COMPANY AGE AND SUCCESS RATE OVER TIME: Low Wage
print("Low Wage Success Rate Over Time Slope:",round(st.linregress(y = linregdf["Low Success %"], x = yr).slope,6),", P Val:",round(st.linregress(y = linregdf["Low Success %"], x = yr).pvalue,4),", R Val:",round(st.linregress(y = linregdf["Low Success %"], x = yr).rvalue,4),"\nModerate growth, .56% growth in success rate a year, and not a statistically significant slope.")
# LINEAR REGRESSION BETWEEN COMPANY AGE AND APPLICATIONS OVER TIME: Low Wage
print("\nLow Wage Application Rate Over Time Slope:",round(st.linregress(y = linregdf["Low Apps"], x = yr).slope,3),", P Val:",round(st.linregress(y = linregdf["Low Apps"], x = yr).pvalue,4),", R Val:",round(st.linregress(y = linregdf["Low Apps"], x = yr).rvalue,4),"\nWeak growth, 1633 applications a year, and not a statistically significant slope.")

# LINEAR REGRESSION BETWEEN COMPANY AGE AND SUCCESS RATE OVER TIME: MEDIUM Wage
print("\n\nMedium Wage Success Rate Over Time Slope:",round(st.linregress(y = linregdf["Medium Success %"], x = yr).slope,6),", P Val:",round(st.linregress(y = linregdf["Medium Success %"], x = yr).pvalue,4),", R Val:",round(st.linregress(y = linregdf["Medium Success %"], x = yr).rvalue,4),"\nStrong growth, .64% growth in success rate a year, and a statistically significant slope.")
# LINEAR REGRESSION BETWEEN COMPANY AGE AND APPLICATIONS OVER TIME: MEDIUM Wage
print("\nMedium Wage Application Rate Over Time Slope:",round(st.linregress(y = linregdf["Medium Apps"], x = yr).slope,3),", P Val:",round(st.linregress(y = linregdf["Medium Apps"], x = yr).pvalue,4),", R Val:",round(st.linregress(y = linregdf["Medium Apps"], x = yr).rvalue,4),"\nModerate growth, 1688 applications a year, and not a statistically significant slope.")

# LINEAR REGRESSION BETWEEN COMPANY AGE AND SUCCESS RATE OVER TIME: OLD COMPANY
print("\n\nHigh Wage Success Rate Over Time Slope:",round(st.linregress(y = linregdf["High Success %"], x = yr).slope,6),", P Val:",round(st.linregress(y = linregdf["High Success %"], x = yr).pvalue,4),", R Val:",round(st.linregress(y = linregdf["High Success %"], x = yr).rvalue,4),"\nStrong growth, .67% growth in success rate a year, and a statistically significant slope.")
# LINEAR REGRESSION BETWEEN COMPANY AGE AND APPLICATIONS OVER TIME: OLD COMPANY
print("\nHigh Wage Application Rate Over Time Slope:",round(st.linregress(y = linregdf["High Apps"], x = yr).slope,3),", P Val:",round(st.linregress(y = linregdf["High Apps"], x = yr).pvalue,4),", R Val:",round(st.linregress(y = linregdf["High Apps"], x = yr).rvalue,4),"\nModerate growth, 1575 applications a year, and not a statistically significant slope.")

In [ ]:
# GETTING THE TOTAL PVAL
linregdf.loc["ALL YRS"] = linregdf.sum()
youngsuc = []
medsuc = []
oldsuc = []
for x in range(len(yr)):
    y = linregdf.loc[yr[x]].iloc[0]*linregdf.loc[yr[x]].iloc[3]
    m = linregdf.loc[yr[x]].iloc[1]*linregdf.loc[yr[x]].iloc[4]
    o = linregdf.loc[yr[x]].iloc[2]*linregdf.loc[yr[x]].iloc[5]
    youngsuc.append(y)
    medsuc.append(m)
    oldsuc.append(o)
linregdf.loc["ALL YRS"].iloc[0] = sum(youngsuc)/linregdf.loc["ALL YRS"].iloc[3]
linregdf.loc["ALL YRS"].iloc[1] = sum(medsuc)/linregdf.loc["ALL YRS"].iloc[4]
linregdf.loc["ALL YRS"].iloc[2] = sum(oldsuc)/linregdf.loc["ALL YRS"].iloc[5]


for x in range(len(yr)):
    sample1_phat = linregdf.loc["ALL YRS"].iloc[0]
    sample2_phat = linregdf.loc["ALL YRS"].iloc[1]
    sample1_size = linregdf.loc["ALL YRS"].iloc[3]
    sample2_size = linregdf.loc["ALL YRS"].iloc[4]
    phat = (sample1_phat*sample1_size+sample2_phat*sample2_size)/(sample1_size+sample2_size)
    zscore = (sample1_phat - sample2_phat)/np.sqrt(phat*(1-phat)*((1/sample1_size)+(1/sample2_size)))    
    pval = st.norm.sf(abs(zscore))*2
linregdf.loc["ALL YRS"].iloc[6] = pval

for x in range(len(yr)):
    sample1_phat = linregdf.loc["ALL YRS"].iloc[0]
    sample2_phat = linregdf.loc["ALL YRS"].iloc[2]
    sample1_size = linregdf.loc["ALL YRS"].iloc[3]
    sample2_size = linregdf.loc["ALL YRS"].iloc[5]
    phat = (sample1_phat*sample1_size+sample2_phat*sample2_size)/(sample1_size+sample2_size)
    zscore = (sample1_phat - sample2_phat)/np.sqrt(phat*(1-phat)*((1/sample1_size)+(1/sample2_size)))    
    pval = st.norm.sf(abs(zscore))*2
linregdf.loc["ALL YRS"].iloc[7] = pval

for x in range(len(yr)):
    sample1_phat = linregdf.loc["ALL YRS"].iloc[1]
    sample2_phat = linregdf.loc["ALL YRS"].iloc[2]
    sample1_size = linregdf.loc["ALL YRS"].iloc[4]
    sample2_size = linregdf.loc["ALL YRS"].iloc[5]
    phat = (sample1_phat*sample1_size+sample2_phat*sample2_size)/(sample1_size+sample2_size)
    zscore = (sample1_phat - sample2_phat)/np.sqrt(phat*(1-phat)*((1/sample1_size)+(1/sample2_size)))    
    pval = st.norm.sf(abs(zscore))*2
linregdf.loc["ALL YRS"].iloc[8] = pval
linregdf = linregdf.head(9).style.format("{:,.2f}")
linregdf

In [ ]:
# Distribution of people in the 66th percentile and above in Wages

joby = df[df["WAGE_GROUP"]=="HIGH"].groupby("WORKSITE_STATE")["CASE_STATUS"].value_counts()
jobyer = pd.DataFrame(joby.groupby(level=0).transform(lambda x: (x / x.sum()).round(2)))
jobyer.columns = ["Percent"]
f1 = jobyer.reset_index()["WORKSITE_STATE"].tolist()
t1 = jobyer.reset_index()["CASE_STATUS"].replace(1,"Accept").replace(0,"Reject").tolist()
idx1 = []
idx1.extend([tuple(a) for a in zip (f1,t1)])
jobyer.index = pd.MultiIndex.from_tuples(idx1)
jobyer = jobyer.reset_index()
jobyer = jobyer[jobyer["level_1"]=="Accept"]
jobyer = jobyer.drop("level_1",axis = 1)

jobycount = pd.DataFrame(df[df["WAGE_GROUP"]=="HIGH"].groupby("WORKSITE_STATE")["CASE_STATUS"].value_counts())
jobycount.columns = ["Count"]
jobycount = jobycount.reset_index()
jobycount = jobycount[jobycount["CASE_STATUS"]==1]
jobycount = jobycount["Count"]

jobyer = pd.concat([jobyer, jobycount], axis = 1)
jobyer.columns = ["State","Text","Count"]
jobyer["Text"] = jobyer["State"].astype(str)+": "+jobyer["Text"].astype(str)+" Percent Successful"
# Choropleth Map of JOB State Accept %
data = dict(type = "choropleth",
           locations = jobyer["State"],
           locationmode = "USA-states",
           z = jobyer["Count"].astype(float),
           text = jobyer["Text"],
           colorbar={"title":"Count"},
           colorscale = "sunset",
           marker = dict(line = dict(color = "rgb(255,255,255)")) )
layout = dict(title = "Distribution of Top 66 Percentile Wages",
              geo = dict(scope = "usa",showlakes = True, lakecolor = "rgb(255,255,255)"))
choro = go.Figure(data, layout)
iplot(choro)

In [ ]:
# PW_SOC_CODE ANALYSIS
# Top 5 PW_SOC of all time 2014-2021
allsoc = pd.DataFrame(df["PW_SOC_CODE"].value_counts()[:5]).reset_index()
allsoc.columns = ["ASoc","COUNT ASoc"]
sucsoc = pd.DataFrame(df[df["CASE_STATUS"]==1]["PW_SOC_CODE"].value_counts()[:5]).reset_index()
sucsoc.columns = ["SSoc","COUNT SSoc"]
pd.concat([allsoc,sucsoc], axis = 1)

In [ ]:
# Grabbing the top 5 for each year PW CODE: only succesful applications
top5semp = pd.DataFrame()
def grab (year):
    return df[(df["YEAR"]==year)&(df["CASE_STATUS"]==1)]["PW_SOC_CODE"].value_counts()[:5]
for x in range(len(yr)):
    j = pd.DataFrame(grab(yr[x]))
    top5semp = pd.concat([top5semp,j], axis = 1)
top5semp.columns= yr
# King of Consistency: HOW MANY YEARS IS THIS PW CODE NOT IN THE TOP 5?: TOP CONSISTENT COMPANIES...
kingcons = top5semp.isnull().replace(False,0).replace(True,1)
kingcons["Yrs not in Top 5"] = top5semp.isnull().replace(False,0).replace(True,1).sum(axis = 1)
kingcons["Yrs not in Top 5"].sort_values()[:10]


In [ ]:
# Grabbing the top 5 for each year PW CODE: total applications
top5aemp = pd.DataFrame()
def grab (year):
    return df[df["YEAR"]==year]["PW_SOC_CODE"].value_counts()[:5]
for x in range(len(yr)):
    j = pd.DataFrame(grab(yr[x]))
    top5aemp = pd.concat([top5aemp,j], axis = 1)
top5aemp.columns= yr
# King of Consistency: HOW MANY YEARS IS THIS PW CODE NOT IN THE TOP 5?: TOP CONSISTENT COMPANIES...
kingconsA = top5aemp.isnull().replace(False,0).replace(True,1)
kingconsA["Yrs not in Top 5"] = top5aemp.isnull().replace(False,0).replace(True,1).sum(axis = 1)
kingconsA["Yrs not in Top 5"].sort_values()[:10]


In [ ]:
# DIVING DEEPER INTO THE TOP CODES!
#ANCHORED BY STATE: PW SKILL,APPLICATIONS, MEDIAN WAGE, YRS POST GRADE, MEDIAN DAYS

code = "15-1132"
# CHANGE THIS FOR ANY CODE

# MEDIAN WAGE, MEDIAN YEARS AFTER GRADUATING, MEDIAN DAYS TO DECIDE : FOR EACH STATE IN THE TOP 15 OF THE COMPANY AND BY PW LEVEL
topstate = df[df["PW_SOC_CODE"] == code]["WORKSITE_STATE"].value_counts()[:15].index.tolist()
pwskill = df["PW_SKILL_LEVEL"].dropna().unique().tolist()
pwskill.sort()


statelist = []
pwlist = pwskill*len(topstate)
apps = []
medwage = []
yrsgrad = []
meddays = []


for x in range(len(topstate)):
    s = (topstate[x]+",")*len(pwskill)
    s = "".join(s).split(",")
    s = list(filter(("").__ne__, s))
    statelist.append(s) 
for x in range(len(topstate)):
    for j in range(len(pwskill)):
        a = len(df[(df["PW_SOC_CODE"] == code)&(df["WORKSITE_STATE"] == topstate[x])][df[(df["PW_SOC_CODE"] == code)&(df["WORKSITE_STATE"] == topstate[x])]["PW_SKILL_LEVEL"]==pwskill[j]])
        apps.append(a)
        b = df[(df["PW_SOC_CODE"] == code)&(df["WORKSITE_STATE"] == topstate[x])][df[(df["PW_SOC_CODE"] == code)&(df["WORKSITE_STATE"] == topstate[x])]["PW_SKILL_LEVEL"]==pwskill[j]]["WO_A"].median()
        medwage.append(b)
        c = df[(df["PW_SOC_CODE"] == code)&(df["WORKSITE_STATE"] == topstate[x])][df[(df["PW_SOC_CODE"] == code)&(df["WORKSITE_STATE"] == topstate[x])]["PW_SKILL_LEVEL"]==pwskill[j]]["YRSPOSTGRAD"].median()
        yrsgrad.append(c)
        d = df[(df["PW_SOC_CODE"] == code)&(df["WORKSITE_STATE"] == topstate[x])][df[(df["PW_SOC_CODE"] == code)&(df["WORKSITE_STATE"] == topstate[x])]["PW_SKILL_LEVEL"]==pwskill[j]]["DAYS_TO_DECIDE"].median()
        meddays.append(d)
codestatedf = pd.DataFrame([sum(statelist, []),pwlist,apps,medwage,yrsgrad,meddays]).transpose()
codestatedf.columns = ["STATE","PW LEVEL", "TOTAL APPLICATIONS","MEDIAN WAGE","MEDIAN YRS POST GRAD","MEDIAN DAYS TO DECIDE"]
codestatedf = codestatedf.set_index(["STATE","PW LEVEL"])
codestatedf


In [ ]:
# ANCHORED BY STATE: EDU LEVEL,APPLICATIONS, MEDIAN WAGE, YRS POST GRADE, MEDIAN DAYS
# MEDIAN WAGE, MEDIAN YEARS AFTER GRADUATING, MEDIAN DAYS TO DECIDE : FOR EACH STATE IN THE TOP 15 OF THE COMPANY AND BY EDU LEVEL
edu = df["FOREIGN_WORKER_EDUCATION"].dropna().unique().tolist()
edu[0],edu[1],edu[2],edu[3],edu[4],edu[5] = edu[3],edu[5],edu[0],edu[2],edu[1],edu[4]

statelist = []
edulist = edu*len(topstate)
apps = []
medwage = []
yrsgrad = []
meddays = []


for x in range(len(topstate)):
    s = (topstate[x]+",")*len(edu)
    s = "".join(s).split(",")
    s = list(filter(("").__ne__, s))
    statelist.append(s) 
for x in range(len(topstate)):
    for j in range(len(edu)):
        a = len(df[(df["PW_SOC_CODE"] == code)&(df["WORKSITE_STATE"] == topstate[x])][df[(df["PW_SOC_CODE"] == code)&(df["WORKSITE_STATE"] == topstate[x])]["FOREIGN_WORKER_EDUCATION"]==edu[j]])
        apps.append(a)
        b = df[(df["PW_SOC_CODE"] == code)&(df["WORKSITE_STATE"] == topstate[x])][df[(df["PW_SOC_CODE"] == code)&(df["WORKSITE_STATE"] == topstate[x])]["FOREIGN_WORKER_EDUCATION"]==edu[j]]["WO_A"].median()
        medwage.append(b)
        c = df[(df["PW_SOC_CODE"] == code)&(df["WORKSITE_STATE"] == topstate[x])][df[(df["PW_SOC_CODE"] == code)&(df["WORKSITE_STATE"] == topstate[x])]["FOREIGN_WORKER_EDUCATION"]==edu[j]]["YRSPOSTGRAD"].median()
        yrsgrad.append(c)
        d = df[(df["PW_SOC_CODE"] == code)&(df["WORKSITE_STATE"] == topstate[x])][df[(df["PW_SOC_CODE"] == code)&(df["WORKSITE_STATE"] == topstate[x])]["FOREIGN_WORKER_EDUCATION"]==edu[j]]["DAYS_TO_DECIDE"].median()
        meddays.append(d)
codestatedf1 = pd.DataFrame([sum(statelist, []),edulist,apps,medwage,yrsgrad,meddays]).transpose()
codestatedf1.columns = ["STATE","EDU LEVEL", "TOTAL APPLICATIONS","MEDIAN WAGE","MEDIAN YRS POST GRAD","MEDIAN DAYS TO DECIDE"]
codestatedf1 = codestatedf1.set_index(["STATE","EDU LEVEL"])
codestatedf1

In [ ]:
# NEW COLUMN: 1 is in the top 5 of the YEAR, 0 is not: Applications
filler = pd.DataFrame()
for x in range(len(yr)):
    listy = top5aemp[yr[x]].dropna().index.tolist()
    filler = pd.concat([filler,pd.DataFrame((df[df["YEAR"]==yr[x]]["PW_SOC_CODE"].isin(listy)).replace(True,1).replace(False,0))])

filler = filler.reset_index().sort_values(by = "index").set_index("index")
filler.columns = ["TOP5SOC"]
df["TOP5SOC"] = filler

In [ ]:
# Distribution of the states worked in if within top 5 pw code

joby = df[df["TOP5SOC"]==1].groupby("WORKSITE_STATE")["CASE_STATUS"].value_counts()
jobyer = pd.DataFrame(joby.groupby(level=0).transform(lambda x: (x / x.sum()).round(2)))
jobyer.columns = ["Percent"]
f1 = jobyer.reset_index()["WORKSITE_STATE"].tolist()
t1 = jobyer.reset_index()["CASE_STATUS"].replace(1,"Accept").replace(0,"Reject").tolist()
idx1 = []
idx1.extend([tuple(a) for a in zip (f1,t1)])
jobyer.index = pd.MultiIndex.from_tuples(idx1)
jobyer = jobyer.reset_index()
jobyer = jobyer[jobyer["level_1"]=="Accept"]
jobyer = jobyer.drop("level_1",axis = 1)

jobycount = pd.DataFrame(df[df["TOP5SOC"]==1].groupby("WORKSITE_STATE")["CASE_STATUS"].value_counts())
jobycount.columns = ["Count"]
jobycount = jobycount.reset_index()
jobycount = jobycount[jobycount["CASE_STATUS"]==1]
jobycount = jobycount["Count"]

jobyer = pd.concat([jobyer, jobycount], axis = 1)
jobyer.columns = ["State","Text","Count"]
jobyer["Text"] = jobyer["State"].astype(str)+": "+jobyer["Text"].astype(str)+" Percent Successful"
# Choropleth Map of JOB State Accept %
data = dict(type = "choropleth",
           locations = jobyer["State"],
           locationmode = "USA-states",
           z = jobyer["Count"].astype(float),
           text = jobyer["Text"],
           colorbar={"title":"Count"},
           colorscale = "jet",
           marker = dict(line = dict(color = "rgb(255,255,255)")) )
layout = dict(title = "Distribution of Top 5 SOC Code",
              geo = dict(scope = "usa",showlakes = True, lakecolor = "rgb(255,255,255)"))
choro = go.Figure(data, layout)
iplot(choro)

In [ ]:
# CORRELATION ANALYSIS: TOP 5 PW CODE WITH CASE STATUS: CASE BY CASE
st.linregress(x = df.dropna()["TOP5SOC"], y = df.dropna()["CASE_STATUS"])
# DOES NOT SHOW MUCH, AGGREGATE MAY SHOW A DIFFERENT STORY

In [ ]:
# PLOTTING THE DIFFERENT SUCCESS RATES OVER TIME BY TOP 15 COMPANY (IN TERMS OF ANNUAL APPLICATIONS)
top15 = df[df["TOP5SOC"]==1]
not15 = df[df["TOP5SOC"]==0]

topdf = pd.DataFrame()
notdf = pd.DataFrame()
analysisdf = pd.DataFrame()

for x in range(len(yr)):
    j = pd.DataFrame([len(top15[(top15["YEAR"]==yr[x])&(top15["CASE_STATUS"]==1)])/len(top15[top15["YEAR"]==yr[x]])])
    b = pd.DataFrame([len(not15[(not15["YEAR"]==yr[x])&(not15["CASE_STATUS"]==1)])/len(not15[not15["YEAR"]==yr[x]])])
    topdf = pd.concat([topdf,j])
    notdf = pd.concat([notdf,b])
    analysisdf = pd.concat([topdf,notdf], axis = 1)

analysisdf.index = yr
analysisdf.columns = ["Top 5 Success %", "Not 5 Success %"]
plt.figure(figsize = (12,10), dpi = 100)
analysisdf.plot(kind = "bar")
plt.tight_layout()
plt.legend(bbox_to_anchor=(1.05, 1), loc=2, borderaxespad=0.)
plt.title("Success Rates by Applicants in the Top 5 and not in the Top 5 Job Codes, Over Time")

In [ ]:
# PREPARING DATA FOR LIN REGRESS AND GETTING P VALUE
top15apps = []
not15apps = []

for x in range(len(yr)):
    t = (top15["YEAR"]==yr[x]).sum()
    n = (not15["YEAR"]==yr[x]).sum()
    top15apps.append(t)
    not15apps.append(n)
analysisdf = analysisdf.reset_index()
analysisdf["Top 5 Apps"] = pd.DataFrame(top15apps)
analysisdf["Not 5 Apps"] = pd.DataFrame(not15apps)
analysisdf = analysisdf.set_index("index")
# PLOTTING APPLICATION RATES OVER TIME BY TOP 15 or NOT TOP 15
plt.figure(figsize = (12,10), dpi = 100)
analysisdf.reset_index().plot(x = "index", y = ["Top 5 Apps","Not 5 Apps"], kind = "bar")
plt.tight_layout()
plt.legend(bbox_to_anchor=(1.05, 1), loc=2, borderaxespad=0.)
plt.title("Application Rates by Companies in the Top 5 and not in the Top 5 Job Codes, Over Time")

In [ ]:
# P VALUE PER YEAR: ON THE SUCCESS RATE
top_not = []
for x in range(len(yr)):
    sample1_phat = analysisdf.loc[yr[x]]["Top 5 Success %"]
    sample2_phat = analysisdf.loc[yr[x]]["Not 5 Success %"]
    sample1_size = analysisdf.loc[yr[x]]["Top 5 Apps"]
    sample2_size = analysisdf.loc[yr[x]]["Not 5 Apps"]
    phat = (sample1_phat*sample1_size+sample2_phat*sample2_size)/(sample1_size+sample2_size)
    zscore = (sample1_phat - sample2_phat)/np.sqrt(phat*(1-phat)*((1/sample1_size)+(1/sample2_size)))    
    pval = st.norm.sf(abs(zscore))*2
    top_not.append(pval)

pvaldf = pd.DataFrame(top_not)
pvaldf.columns = ["Top 5 Vs. Not Top 5"]
pvaldf.index = yr
analysisdf["PVAL Success %"] = pvaldf["Top 5 Vs. Not Top 5"]

In [ ]:
# LINEAR REGRESSION BETWEEN Code AND SUCCESS RATE OVER TIME: TOP 5 Code
print("Top 5 Code Success Rate Over Time Slope:",round(st.linregress(y = analysisdf["Top 5 Success %"], x = yr).slope,6),", P Val:",round(st.linregress(y = analysisdf["Top 5 Success %"], x = yr).pvalue,4),", R Val:",round(st.linregress(y = analysisdf["Top 5 Success %"], x = yr).rvalue,4),"\nStrong growth, .56% growth in success rate a year, and a statistically significant slope.")
# LINEAR REGRESSION BETWEEN Code AND APPLICATIONS OVER TIME: TOP 5 Code
print("\nTop 5 Code Application Rate Over Time Slope:",round(st.linregress(y = analysisdf["Top 5 Apps"], x = yr).slope,3),", P Val:",round(st.linregress(y = analysisdf["Top 5 Apps"], x = yr).pvalue,4),", R Val:",round(st.linregress(y = analysisdf["Top 5 Apps"], x = yr).rvalue,4),"\nModerate growth, 2326 applications a year, but not a statistically significant slope.")

# LINEAR REGRESSION BETWEEN Code AND SUCCESS RATE OVER TIME: NOT TOP 5 Code
print("\n\nNot Top 5 Code Success Rate Over Time Slope:",round(st.linregress(y = analysisdf["Not 5 Success %"], x = yr).slope,6),", P Val:",round(st.linregress(y = analysisdf["Not 5 Success %"], x = yr).pvalue,4),", R Val:",round(st.linregress(y = analysisdf["Not 5 Success %"], x = yr).rvalue,4),"\nStrong growth, .79% growth in success rate a year, and a statistically significant slope.")
# LINEAR REGRESSION BETWEEN Code AND APPLICATIONS OVER TIME: NOT TOP 5 Code
print("\nNot Top 5 Code Application Rate Over Time Slope:",round(st.linregress(y = analysisdf["Not 5 Apps"], x = yr).slope,3),", P Val:",round(st.linregress(y = analysisdf["Not 5 Apps"], x = yr).pvalue,4),", R Val:",round(st.linregress(y = analysisdf["Not 5 Apps"], x = yr).rvalue,4),"\nModerate growth, 2570 applications a year, but not a statistically significant slope.")


In [ ]:
# GETTING THE TOTAL PVAL
analysisdf.loc["ALL YRS"] = analysisdf.sum()
top15suc = []
not15suc = []

for x in range(len(yr)):
    t15 = analysisdf.loc[yr[x]].iloc[0]*analysisdf.loc[yr[x]].iloc[2]
    n15 = analysisdf.loc[yr[x]].iloc[1]*analysisdf.loc[yr[x]].iloc[3]
    top15suc.append(t15)
    not15suc.append(n15)
analysisdf.loc["ALL YRS"].iloc[0] = sum(top15suc)/analysisdf.loc["ALL YRS"].iloc[2]
analysisdf.loc["ALL YRS"].iloc[1] = sum(not15suc)/analysisdf.loc["ALL YRS"].iloc[3]


for x in range(len(yr)):
    sample1_phat = analysisdf.loc["ALL YRS"].iloc[0]
    sample2_phat = analysisdf.loc["ALL YRS"].iloc[1]
    sample1_size = analysisdf.loc["ALL YRS"].iloc[2]
    sample2_size = analysisdf.loc["ALL YRS"].iloc[3]
    phat = (sample1_phat*sample1_size+sample2_phat*sample2_size)/(sample1_size+sample2_size)
    zscore = (sample1_phat - sample2_phat)/np.sqrt(phat*(1-phat)*((1/sample1_size)+(1/sample2_size)))    
    pval = st.norm.sf(abs(zscore))*2
analysisdf.loc["ALL YRS"].iloc[4] = pval

analysisdf = analysisdf.head(9).style.format("{:,.2f}")
analysisdf

In [ ]:
# Top 10 WORK STATE of all time 2014-2021
allsoc = pd.DataFrame(df["WORKSITE_STATE"].value_counts()[:10]).reset_index()
allsoc.columns = ["AState","COUNT AState"]
sucsoc = pd.DataFrame(df[df["CASE_STATUS"]==1]["WORKSITE_STATE"].value_counts()[:10]).reset_index()
sucsoc.columns = ["SState","COUNT SState"]
pd.concat([allsoc,sucsoc], axis = 1)

In [ ]:
# ACCEPT % BASED ON STATE WORKED IN

joby = df.groupby("WORKSITE_STATE")["CASE_STATUS"].value_counts()
jobyer = pd.DataFrame(joby.groupby(level=0).transform(lambda x: (x / x.sum()).round(2)))
jobyer.columns = ["Percent"]
f1 = jobyer.reset_index()["WORKSITE_STATE"].tolist()
t1 = jobyer.reset_index()["CASE_STATUS"].replace(1,"Accept").replace(0,"Reject").tolist()
idx1 = []
idx1.extend([tuple(a) for a in zip (f1,t1)])
jobyer.index = pd.MultiIndex.from_tuples(idx1)
jobyer = jobyer.reset_index()
jobyer = jobyer[jobyer["level_1"]=="Accept"]
jobyer = jobyer.drop("level_1",axis = 1)

jobycount = pd.DataFrame(df.groupby("WORKSITE_STATE")["CASE_STATUS"].value_counts())
jobycount.columns = ["Count"]
jobycount = jobycount.reset_index()
jobycount = jobycount[jobycount["CASE_STATUS"]==1]
jobycount = jobycount["Count"]

jobyer = pd.concat([jobyer, jobycount], axis = 1)
jobyer.columns = ["State","Percent","Count"]
jobyer["Count"] = jobyer["State"].astype(str)+": "+jobyer["Count"].astype(str)+" Successful Applications"
# Choropleth Map of JOB State Accept %
data = dict(type = "choropleth",
           locations = jobyer["State"],
           locationmode = "USA-states",
           z = jobyer["Percent"].astype(float),
           text = jobyer["Count"],
           colorbar={"title":"Percent Accepted"},
           colorscale = "earth",
           marker = dict(line = dict(color = "rgb(255,255,255)")) )
layout = dict(title = "Percent Accepted in States Worked In",
              geo = dict(scope = "usa",showlakes = True, lakecolor = "rgb(255,255,255)"))
choro = go.Figure(data, layout)
iplot(choro)

In [ ]:
# Grabbing the top 5 for each year state worked in: total applications
top5aemp = pd.DataFrame()
def grab (year):
    return df[df["YEAR"]==year]["WORKSITE_STATE"].value_counts()[:5]
for x in range(len(yr)):
    j = pd.DataFrame(grab(yr[x]))
    top5aemp = pd.concat([top5aemp,j], axis = 1)
top5aemp.columns= yr
# King of Consistency: HOW MANY YEARS IS THIS state NOT IN THE TOP 5?: TOP CONSISTENT states...
kingconsA = top5aemp.isnull().replace(False,0).replace(True,1)
kingconsA["Yrs not in Top 5"] = top5aemp.isnull().replace(False,0).replace(True,1).sum(axis = 1)
kingconsA["Yrs not in Top 5"].sort_values()[:10]


In [ ]:
# NEW COLUMN: 1 is in the top 5 of the YEAR, 0 is not: Applications
filler = pd.DataFrame()
for x in range(len(yr)):
    listy = top5aemp[yr[x]].dropna().index.tolist()
    filler = pd.concat([filler,pd.DataFrame((df[df["YEAR"]==yr[x]]["WORKSITE_STATE"].isin(listy)).replace(True,1).replace(False,0))])

filler = filler.reset_index().sort_values(by = "index").set_index("index")
filler.columns = ["TOP5WORKSTATE"]
df["TOP5WORKSTATE"] = filler

In [ ]:
# CORRELATION ANALYSIS: TOP 5 WORK STATE WITH CASE STATUS: CASE BY CASE
st.linregress(x = df.dropna()["TOP5WORKSTATE"], y = df.dropna()["CASE_STATUS"])
# DOES NOT SHOW MUCH, AGGREGATE MAY SHOW A DIFFERENT STORY

In [ ]:
# PLOTTING THE DIFFERENT SUCCESS RATES OVER TIME BY TOP 5 COMPANY (IN TERMS OF ANNUAL APPLICATIONS)
top15 = df[df["TOP5WORKSTATE"]==1]
not15 = df[df["TOP5WORKSTATE"]==0]

topdf = pd.DataFrame()
notdf = pd.DataFrame()
analysisdf = pd.DataFrame()

for x in range(len(yr)):
    j = pd.DataFrame([len(top15[(top15["YEAR"]==yr[x])&(top15["CASE_STATUS"]==1)])/len(top15[top15["YEAR"]==yr[x]])])
    b = pd.DataFrame([len(not15[(not15["YEAR"]==yr[x])&(not15["CASE_STATUS"]==1)])/len(not15[not15["YEAR"]==yr[x]])])
    topdf = pd.concat([topdf,j])
    notdf = pd.concat([notdf,b])
    analysisdf = pd.concat([topdf,notdf], axis = 1)

analysisdf.index = yr
analysisdf.columns = ["Top 5 Success %", "Not 5 Success %"]
plt.figure(figsize = (12,10), dpi = 100)
analysisdf.plot(kind = "bar")
plt.tight_layout()
plt.legend(bbox_to_anchor=(1.05, 1), loc=2, borderaxespad=0.)
plt.title("Success Rates by Applicants in the Top 5 and not in the Top 5 States, Over Time")


In [ ]:
# PREPARING DATA FOR LIN REGRESS AND GETTING P VALUE
top15apps = []
not15apps = []

for x in range(len(yr)):
    t = (top15["YEAR"]==yr[x]).sum()
    n = (not15["YEAR"]==yr[x]).sum()
    top15apps.append(t)
    not15apps.append(n)
analysisdf = analysisdf.reset_index()
analysisdf["Top 5 Apps"] = pd.DataFrame(top15apps)
analysisdf["Not 5 Apps"] = pd.DataFrame(not15apps)
analysisdf = analysisdf.set_index("index")
# PLOTTING APPLICATION RATES OVER TIME BY TOP 5 or NOT TOP 5
plt.figure(figsize = (12,10), dpi = 100)
analysisdf.reset_index().plot(x = "index", y = ["Top 5 Apps","Not 5 Apps"], kind = "bar")
plt.tight_layout()
plt.legend(bbox_to_anchor=(1.05, 1), loc=2, borderaxespad=0.)
plt.title("Application Rates by Companies in the Top 5 and not in the Top 5 States, Over Time")

In [ ]:
# P VALUE PER YEAR: ON THE SUCCESS RATE
top_not = []
for x in range(len(yr)):
    sample1_phat = analysisdf.loc[yr[x]]["Top 5 Success %"]
    sample2_phat = analysisdf.loc[yr[x]]["Not 5 Success %"]
    sample1_size = analysisdf.loc[yr[x]]["Top 5 Apps"]
    sample2_size = analysisdf.loc[yr[x]]["Not 5 Apps"]
    phat = (sample1_phat*sample1_size+sample2_phat*sample2_size)/(sample1_size+sample2_size)
    zscore = (sample1_phat - sample2_phat)/np.sqrt(phat*(1-phat)*((1/sample1_size)+(1/sample2_size)))    
    pval = st.norm.sf(abs(zscore))*2
    top_not.append(pval)

pvaldf = pd.DataFrame(top_not)
pvaldf.columns = ["Top 5 Vs. Not Top 5"]
pvaldf.index = yr
analysisdf["PVAL Success %"] = pvaldf["Top 5 Vs. Not Top 5"]

In [ ]:
# LINEAR REGRESSION BETWEEN TOP STATES AND SUCCESS RATE OVER TIME: TOP 5 STATES
print("top 5 success rate over time:",st.linregress(y = analysisdf["Top 5 Success %"], x = yr))
print("\ntop 5 applications over time:",st.linregress(y = analysisdf["Top 5 Apps"], x = yr))

print("\n\nnottop 5 success rate over time:",st.linregress(y = analysisdf["Not 5 Success %"], x = yr))
print("\nnot top 5 applications over time:",st.linregress(y = analysisdf["Not 5 Apps"], x = yr))

In [ ]:
# GETTING THE TOTAL PVAL
analysisdf.loc["ALL YRS"] = analysisdf.sum()
top15suc = []
not15suc = []

for x in range(len(yr)):
    t15 = analysisdf.loc[yr[x]].iloc[0]*analysisdf.loc[yr[x]].iloc[2]
    n15 = analysisdf.loc[yr[x]].iloc[1]*analysisdf.loc[yr[x]].iloc[3]
    top15suc.append(t15)
    not15suc.append(n15)
analysisdf.loc["ALL YRS"].iloc[0] = sum(top15suc)/analysisdf.loc["ALL YRS"].iloc[2]
analysisdf.loc["ALL YRS"].iloc[1] = sum(not15suc)/analysisdf.loc["ALL YRS"].iloc[3]


for x in range(len(yr)):
    sample1_phat = analysisdf.loc["ALL YRS"].iloc[0]
    sample2_phat = analysisdf.loc["ALL YRS"].iloc[1]
    sample1_size = analysisdf.loc["ALL YRS"].iloc[2]
    sample2_size = analysisdf.loc["ALL YRS"].iloc[3]
    phat = (sample1_phat*sample1_size+sample2_phat*sample2_size)/(sample1_size+sample2_size)
    zscore = (sample1_phat - sample2_phat)/np.sqrt(phat*(1-phat)*((1/sample1_size)+(1/sample2_size)))    
    pval = st.norm.sf(abs(zscore))*2
analysisdf.loc["ALL YRS"].iloc[4] = pval

analysisdf = analysisdf.head(9).style.format("{:,.2f}")
analysisdf

In [ ]:
# Top 10 WORK Cities of all time 2014-2021
allsoc = pd.DataFrame(df["WORKSITE_CITY"].value_counts()[:10]).reset_index()
allsoc.columns = ["ACity","COUNT ACity"]
sucsoc = pd.DataFrame(df[df["CASE_STATUS"]==1]["WORKSITE_CITY"].value_counts()[:10]).reset_index()
sucsoc.columns = ["SCity","COUNT SCity"]
pd.concat([allsoc,sucsoc], axis = 1)

In [ ]:
# Grabbing the top 5 for each year state worked in: total applications
top5aemp = pd.DataFrame()
def grab (year):
    return df[df["YEAR"]==year]["WORKSITE_CITY"].value_counts()[:10]
for x in range(len(yr)):
    j = pd.DataFrame(grab(yr[x]))
    top5aemp = pd.concat([top5aemp,j], axis = 1)
top5aemp.columns= yr
# King of Consistency: HOW MANY YEARS IS THIS state NOT IN THE TOP 5?: TOP CONSISTENT states...
kingconsA = top5aemp.isnull().replace(False,0).replace(True,1)
kingconsA["Yrs not in Top 10"] = top5aemp.isnull().replace(False,0).replace(True,1).sum(axis = 1)
kingconsA["Yrs not in Top 10"].sort_values()[:10]


In [ ]:
# NEW COLUMN: 1 is in the top 5 of the YEAR, 0 is not: Applications
filler = pd.DataFrame()
for x in range(len(yr)):
    listy = top5aemp[yr[x]].dropna().index.tolist()
    filler = pd.concat([filler,pd.DataFrame((df[df["YEAR"]==yr[x]]["WORKSITE_CITY"].isin(listy)).replace(True,1).replace(False,0))])

filler = filler.reset_index().sort_values(by = "index").set_index("index")
filler.columns = ["TOP10WORKCITY"]
df["TOP10WORKCITY"] = filler

In [ ]:
# CORRELATION ANALYSIS: TOP 10 WORK CITY WITH CASE STATUS: CASE BY CASE
st.linregress(x = df.dropna()["TOP10WORKCITY"], y = df.dropna()["CASE_STATUS"])
# DOES NOT SHOW MUCH, AGGREGATE MAY SHOW A DIFFERENT STORY

In [ ]:
# PLOTTING THE DIFFERENT SUCCESS RATES OVER TIME BY TOP 10 COMPANY (IN TERMS OF ANNUAL APPLICATIONS)
top15 = df[df["TOP10WORKCITY"]==1]
not15 = df[df["TOP10WORKCITY"]==0]

topdf = pd.DataFrame()
notdf = pd.DataFrame()
analysisdf = pd.DataFrame()

for x in range(len(yr)):
    j = pd.DataFrame([len(top15[(top15["YEAR"]==yr[x])&(top15["CASE_STATUS"]==1)])/len(top15[top15["YEAR"]==yr[x]])])
    b = pd.DataFrame([len(not15[(not15["YEAR"]==yr[x])&(not15["CASE_STATUS"]==1)])/len(not15[not15["YEAR"]==yr[x]])])
    topdf = pd.concat([topdf,j])
    notdf = pd.concat([notdf,b])
    analysisdf = pd.concat([topdf,notdf], axis = 1)

analysisdf.index = yr
analysisdf.columns = ["Top 10 Success %", "Not 10 Success %"]
plt.figure(figsize = (12,10), dpi = 100)
analysisdf.plot(kind = "bar")
plt.tight_layout()
plt.legend(bbox_to_anchor=(1.05, 1), loc=2, borderaxespad=0.)
plt.title("Success Rates by Applicants in the Top 10 and not in the Top 10 Cities, Over Time")

In [ ]:
# PREPARING DATA FOR LIN REGRESS AND GETTING P VALUE
top15apps = []
not15apps = []

for x in range(len(yr)):
    t = (top15["YEAR"]==yr[x]).sum()
    n = (not15["YEAR"]==yr[x]).sum()
    top15apps.append(t)
    not15apps.append(n)
analysisdf = analysisdf.reset_index()
analysisdf["Top 10 Apps"] = pd.DataFrame(top15apps)
analysisdf["Not 10 Apps"] = pd.DataFrame(not15apps)
analysisdf = analysisdf.set_index("index")
# PLOTTING APPLICATION RATES OVER TIME BY TOP 10 or NOT TOP 10
plt.figure(figsize = (12,10), dpi = 100)
analysisdf.reset_index().plot(x = "index", y = ["Top 10 Apps","Not 10 Apps"], kind = "bar")
plt.tight_layout()
plt.legend(bbox_to_anchor=(1.05, 1), loc=2, borderaxespad=0.)
plt.title("Application Rates by Companies in the Top 10 and not in the Top 10 Cities, Over Time")

In [ ]:
# P VALUE PER YEAR: ON THE SUCCESS RATE
top_not = []
for x in range(len(yr)):
    sample1_phat = analysisdf.loc[yr[x]]["Top 10 Success %"]
    sample2_phat = analysisdf.loc[yr[x]]["Not 10 Success %"]
    sample1_size = analysisdf.loc[yr[x]]["Top 10 Apps"]
    sample2_size = analysisdf.loc[yr[x]]["Not 10 Apps"]
    phat = (sample1_phat*sample1_size+sample2_phat*sample2_size)/(sample1_size+sample2_size)
    zscore = (sample1_phat - sample2_phat)/np.sqrt(phat*(1-phat)*((1/sample1_size)+(1/sample2_size)))    
    pval = st.norm.sf(abs(zscore))*2
    top_not.append(pval)

pvaldf = pd.DataFrame(top_not)
pvaldf.columns = ["Top 10 Vs. Not Top 10"]
pvaldf.index = yr
analysisdf["PVAL Success %"] = pvaldf["Top 10 Vs. Not Top 10"]


In [ ]:
# LINEAR REGRESSION BETWEEN TOP CITIES AND SUCCESS RATE OVER TIME: TOP 10 CITIES
print("top 10 success rate over time:",st.linregress(y = analysisdf["Top 10 Success %"], x = yr))
print("\ntop 10 applications over time:",st.linregress(y = analysisdf["Top 10 Apps"], x = yr))

print("\n\nnot top 10 success rate over time:",st.linregress(y = analysisdf["Not 10 Success %"], x = yr))
print("\nnot top 10 applications over time:",st.linregress(y = analysisdf["Not 10 Apps"], x = yr))

In [ ]:
# GETTING THE TOTAL PVAL
analysisdf.loc["ALL YRS"] = analysisdf.sum()
top15suc = []
not15suc = []

for x in range(len(yr)):
    t15 = analysisdf.loc[yr[x]].iloc[0]*analysisdf.loc[yr[x]].iloc[2]
    n15 = analysisdf.loc[yr[x]].iloc[1]*analysisdf.loc[yr[x]].iloc[3]
    top15suc.append(t15)
    not15suc.append(n15)
analysisdf.loc["ALL YRS"].iloc[0] = sum(top15suc)/analysisdf.loc["ALL YRS"].iloc[2]
analysisdf.loc["ALL YRS"].iloc[1] = sum(not15suc)/analysisdf.loc["ALL YRS"].iloc[3]


for x in range(len(yr)):
    sample1_phat = analysisdf.loc["ALL YRS"].iloc[0]
    sample2_phat = analysisdf.loc["ALL YRS"].iloc[1]
    sample1_size = analysisdf.loc["ALL YRS"].iloc[2]
    sample2_size = analysisdf.loc["ALL YRS"].iloc[3]
    phat = (sample1_phat*sample1_size+sample2_phat*sample2_size)/(sample1_size+sample2_size)
    zscore = (sample1_phat - sample2_phat)/np.sqrt(phat*(1-phat)*((1/sample1_size)+(1/sample2_size)))    
    pval = st.norm.sf(abs(zscore))*2
analysisdf.loc["ALL YRS"].iloc[4] = pval

analysisdf = analysisdf.head(9).style.format("{:,.2f}")
analysisdf

In [ ]:
# CREATING A NEW COLUMN: SPLITTING THE REQUIRED EDUCATION LEVEL INTO 4 GROUPS
df["REQ_EDU_GROUP"] = np.nan
for x in range(len(yr)):
    df["REQ_EDU_GROUP"] = np.where((df["MINIMUM_EDUCATION"]=="HIGH SCHOOL")|(df["MINIMUM_EDUCATION"]=="ASSOCIATE'S"),"LOWER",df["REQ_EDU_GROUP"])
    df["REQ_EDU_GROUP"] = np.where(df["MINIMUM_EDUCATION"]=="BACHELOR'S","COLLEGE",df["REQ_EDU_GROUP"])
    df["REQ_EDU_GROUP"] = np.where((df["MINIMUM_EDUCATION"]=="MASTER'S")|(df["MINIMUM_EDUCATION"]=="DOCTORATE"),"UPPER",df["REQ_EDU_GROUP"])
    df["REQ_EDU_GROUP"] = np.where((df["MINIMUM_EDUCATION"].isnull())|(df["MINIMUM_EDUCATION"]=="OTHER"),"OTHER",df["REQ_EDU_GROUP"])

In [ ]:
# MINIMUM EDUCATION REQUIRED ANALYSIS
# PLOTTING SUCCESS RATES OVER TIME BY MINIMUM EDUCATION REQUIRED

lower = df[df["REQ_EDU_GROUP"]=="LOWER"]
college = df[df["REQ_EDU_GROUP"]=="COLLEGE"]
upper = df[df["REQ_EDU_GROUP"]=="UPPER"]
other = df[df["REQ_EDU_GROUP"]=="OTHER"]



lowerdf = pd.DataFrame()
collegedf = pd.DataFrame()
upperdf = pd.DataFrame()
otherdf = pd.DataFrame()
linregdf = pd.DataFrame()

for x in range(len(yr)):
    lowe = pd.DataFrame([len(lower[(lower["YEAR"]==yr[x])&(lower["CASE_STATUS"]==1)])/len(lower[lower["YEAR"]==yr[x]])])
    coll = pd.DataFrame([len(college[(college["YEAR"]==yr[x])&(college["CASE_STATUS"]==1)])/len(college[college["YEAR"]==yr[x]])])
    upp = pd.DataFrame([len(upper[(upper["YEAR"]==yr[x])&(upper["CASE_STATUS"]==1)])/len(upper[upper["YEAR"]==yr[x]])])
    ott = pd.DataFrame([len(other[(other["YEAR"]==yr[x])&(other["CASE_STATUS"]==1)])/len(other[other["YEAR"]==yr[x]])])
    lowerdf = pd.concat([lowerdf,lowe])
    collegedf = pd.concat([collegedf,coll])
    upperdf = pd.concat([upperdf,upp])
    otherdf = pd.concat([otherdf, ott])
    linregdf = pd.concat([lowerdf,collegedf,upperdf,otherdf], axis = 1)
    

linregdf.index = yr
linregdf.columns = ["Lower Success %", "College Success %", "Upper Success %","Other Success %"]
plt.figure(figsize = (12,10), dpi = 100)
linregdf.plot(kind = "bar")
plt.tight_layout()
plt.legend(bbox_to_anchor=(1.05, 1), loc=2, borderaxespad=0.)
plt.title("Success Rates by Minimum Required Education, Over Time")

In [ ]:
# PREPARING DATA FOR LIN REGRESS AND GETTING P VALUE
lowerapps = []
collegeapps = []
upperapps = []
otherapps = []

for x in range(len(yr)):
    l = (lower["YEAR"]==yr[x]).sum()
    c = (college["YEAR"]==yr[x]).sum()
    u = (upper["YEAR"]==yr[x]).sum()
    o = (other["YEAR"]==yr[x]).sum()
    lowerapps.append(l)
    collegeapps.append(c)
    upperapps.append(u)
    otherapps.append(o)
linregdf = linregdf.reset_index()
linregdf["Lower Apps"] = pd.DataFrame(lowerapps)
linregdf["College Apps"] = pd.DataFrame(collegeapps)
linregdf["Upper Apps"] = pd.DataFrame(upperapps)
linregdf["Other Apps"] = pd.DataFrame(otherapps)
linregdf = linregdf.set_index("index")
# PLOTTING APPLICATION RATES OVER TIME BY EDUCATION LEVEL
plt.figure(figsize = (12,10), dpi = 100)
linregdf.reset_index().plot(x = "index", y = ["Lower Apps","College Apps","Upper Apps","Other Apps"], kind = "bar")
plt.tight_layout()
plt.legend(bbox_to_anchor=(1.05, 1), loc=2, borderaxespad=0.)
plt.title("Application Rates by Minimum Required Education, Over Time")

In [ ]:
# P VALUE PER YEAR
lower_college = []
for x in range(len(yr)):
    sample1_phat = linregdf.loc[yr[x]]["Lower Success %"]
    sample2_phat = linregdf.loc[yr[x]]["College Success %"]
    sample1_size = linregdf.loc[yr[x]]["Lower Apps"]
    sample2_size = linregdf.loc[yr[x]]["College Apps"]
    phat = (sample1_phat*sample1_size+sample2_phat*sample2_size)/(sample1_size+sample2_size)
    zscore = (sample1_phat - sample2_phat)/np.sqrt(phat*(1-phat)*((1/sample1_size)+(1/sample2_size)))    
    pval = st.norm.sf(abs(zscore))*2
    lower_college.append(pval)

pvaldf = pd.DataFrame(lower_college)

lower_upper = []
for x in range(len(yr)):
    sample1_phat = linregdf.loc[yr[x]]["Lower Success %"]
    sample2_phat = linregdf.loc[yr[x]]["Upper Success %"]
    sample1_size = linregdf.loc[yr[x]]["Lower Apps"]
    sample2_size = linregdf.loc[yr[x]]["Upper Apps"]
    phat = (sample1_phat*sample1_size+sample2_phat*sample2_size)/(sample1_size+sample2_size)
    zscore = (sample1_phat - sample2_phat)/np.sqrt(phat*(1-phat)*((1/sample1_size)+(1/sample2_size)))    
    pval = st.norm.sf(abs(zscore))*2
    lower_upper.append(pval)
pvaldf = pd.concat([pvaldf,pd.DataFrame(lower_upper)], axis = 1)

lower_other = []
for x in range(len(yr)):
    sample1_phat = linregdf.loc[yr[x]]["Lower Success %"]
    sample2_phat = linregdf.loc[yr[x]]["Other Success %"]
    sample1_size = linregdf.loc[yr[x]]["Lower Apps"]
    sample2_size = linregdf.loc[yr[x]]["Other Apps"]
    phat = (sample1_phat*sample1_size+sample2_phat*sample2_size)/(sample1_size+sample2_size)
    zscore = (sample1_phat - sample2_phat)/np.sqrt(phat*(1-phat)*((1/sample1_size)+(1/sample2_size)))    
    pval = st.norm.sf(abs(zscore))*2
    lower_other.append(pval)
pvaldf = pd.concat([pvaldf,pd.DataFrame(lower_other)], axis = 1)

college_upper = []
for x in range(len(yr)):
    sample1_phat = linregdf.loc[yr[x]]["College Success %"]
    sample2_phat = linregdf.loc[yr[x]]["Upper Success %"]
    sample1_size = linregdf.loc[yr[x]]["College Apps"]
    sample2_size = linregdf.loc[yr[x]]["Upper Apps"]
    phat = (sample1_phat*sample1_size+sample2_phat*sample2_size)/(sample1_size+sample2_size)
    zscore = (sample1_phat - sample2_phat)/np.sqrt(phat*(1-phat)*((1/sample1_size)+(1/sample2_size)))    
    pval = st.norm.sf(abs(zscore))*2
    college_upper.append(pval)
pvaldf = pd.concat([pvaldf,pd.DataFrame(college_upper)], axis = 1)

college_other = []
for x in range(len(yr)):
    sample1_phat = linregdf.loc[yr[x]]["College Success %"]
    sample2_phat = linregdf.loc[yr[x]]["Other Success %"]
    sample1_size = linregdf.loc[yr[x]]["College Apps"]
    sample2_size = linregdf.loc[yr[x]]["Other Apps"]
    phat = (sample1_phat*sample1_size+sample2_phat*sample2_size)/(sample1_size+sample2_size)
    zscore = (sample1_phat - sample2_phat)/np.sqrt(phat*(1-phat)*((1/sample1_size)+(1/sample2_size)))    
    pval = st.norm.sf(abs(zscore))*2
    college_other.append(pval)
pvaldf = pd.concat([pvaldf,pd.DataFrame(college_other)], axis = 1)

upper_other = []
for x in range(len(yr)):
    sample1_phat = linregdf.loc[yr[x]]["Upper Success %"]
    sample2_phat = linregdf.loc[yr[x]]["Other Success %"]
    sample1_size = linregdf.loc[yr[x]]["Upper Apps"]
    sample2_size = linregdf.loc[yr[x]]["Other Apps"]
    phat = (sample1_phat*sample1_size+sample2_phat*sample2_size)/(sample1_size+sample2_size)
    zscore = (sample1_phat - sample2_phat)/np.sqrt(phat*(1-phat)*((1/sample1_size)+(1/sample2_size)))    
    pval = st.norm.sf(abs(zscore))*2
    upper_other.append(pval)
pvaldf = pd.concat([pvaldf,pd.DataFrame(upper_other)], axis = 1)

pvaldf.columns = ["Lower_College","Lower_Upper","Lower_Other","College_Upper","College_Other","Upper_Other"]
pvaldf.index = yr
linregdf[["Lower_College","Lower_Upper","Lower_Other","College_Upper","College_Other","Upper_Other"]] = pvaldf[["Lower_College","Lower_Upper","Lower_Other","College_Upper","College_Other","Upper_Other"]]

In [ ]:
# LINEAR REGRESSION BETWEEN Minimum education required and success
print("Lower success rate over time:",st.linregress(y = linregdf["Lower Success %"], x = yr))
print("\nLower applications over time:",st.linregress(y = linregdf["Lower Apps"], x = yr))

print("\n\nCollege success rate over time:",st.linregress(y = linregdf["College Success %"], x = yr))
print("\nCollege applications over time:",st.linregress(y = linregdf["College Apps"], x = yr))

print("\n\nUpper success rate over time:",st.linregress(y = linregdf["Upper Success %"], x = yr))
print("\nUpper applications over time:",st.linregress(y = linregdf["Upper Apps"], x = yr))

print("\n\nOther success rate over time:",st.linregress(y = linregdf["Other Success %"], x = yr))
print("\nOther applications over time:",st.linregress(y = linregdf["Other Apps"], x = yr))

In [ ]:
# GETTING THE TOTAL PVAL
linregdf.loc["ALL YRS"] = linregdf.sum()
lowersuc = []
collegesuc = []
uppersuc = []
othersuc = []

for x in range(len(yr)):
    l = linregdf.loc[yr[x]].iloc[0]*linregdf.loc[yr[x]].iloc[4]
    c = linregdf.loc[yr[x]].iloc[1]*linregdf.loc[yr[x]].iloc[5]
    u = linregdf.loc[yr[x]].iloc[2]*linregdf.loc[yr[x]].iloc[6]
    o = linregdf.loc[yr[x]].iloc[3]*linregdf.loc[yr[x]].iloc[7]
    lowersuc.append(l)
    collegesuc.append(c)
    uppersuc.append(u)
    othersuc.append(o)
    
linregdf.loc["ALL YRS"].iloc[0] = sum(lowersuc)/linregdf.loc["ALL YRS"].iloc[4]
linregdf.loc["ALL YRS"].iloc[1] = sum(collegesuc)/linregdf.loc["ALL YRS"].iloc[5]
linregdf.loc["ALL YRS"].iloc[2] = sum(uppersuc)/linregdf.loc["ALL YRS"].iloc[6]
linregdf.loc["ALL YRS"].iloc[3] = sum(othersuc)/linregdf.loc["ALL YRS"].iloc[7]

for x in range(len(yr)):
    sample1_phat = linregdf.loc["ALL YRS"].iloc[0]
    sample2_phat = linregdf.loc["ALL YRS"].iloc[1]
    sample1_size = linregdf.loc["ALL YRS"].iloc[4]
    sample2_size = linregdf.loc["ALL YRS"].iloc[5]
    phat = (sample1_phat*sample1_size+sample2_phat*sample2_size)/(sample1_size+sample2_size)
    zscore = (sample1_phat - sample2_phat)/np.sqrt(phat*(1-phat)*((1/sample1_size)+(1/sample2_size)))    
    pval = st.norm.sf(abs(zscore))*2
linregdf.loc["ALL YRS"].iloc[8] = pval

for x in range(len(yr)):
    sample1_phat = linregdf.loc["ALL YRS"].iloc[0]
    sample2_phat = linregdf.loc["ALL YRS"].iloc[2]
    sample1_size = linregdf.loc["ALL YRS"].iloc[4]
    sample2_size = linregdf.loc["ALL YRS"].iloc[6]
    phat = (sample1_phat*sample1_size+sample2_phat*sample2_size)/(sample1_size+sample2_size)
    zscore = (sample1_phat - sample2_phat)/np.sqrt(phat*(1-phat)*((1/sample1_size)+(1/sample2_size)))    
    pval = st.norm.sf(abs(zscore))*2
linregdf.loc["ALL YRS"].iloc[9] = pval

for x in range(len(yr)):
    sample1_phat = linregdf.loc["ALL YRS"].iloc[0]
    sample2_phat = linregdf.loc["ALL YRS"].iloc[3]
    sample1_size = linregdf.loc["ALL YRS"].iloc[4]
    sample2_size = linregdf.loc["ALL YRS"].iloc[7]
    phat = (sample1_phat*sample1_size+sample2_phat*sample2_size)/(sample1_size+sample2_size)
    zscore = (sample1_phat - sample2_phat)/np.sqrt(phat*(1-phat)*((1/sample1_size)+(1/sample2_size)))    
    pval = st.norm.sf(abs(zscore))*2
linregdf.loc["ALL YRS"].iloc[10] = pval

for x in range(len(yr)):
    sample1_phat = linregdf.loc["ALL YRS"].iloc[1]
    sample2_phat = linregdf.loc["ALL YRS"].iloc[2]
    sample1_size = linregdf.loc["ALL YRS"].iloc[5]
    sample2_size = linregdf.loc["ALL YRS"].iloc[6]
    phat = (sample1_phat*sample1_size+sample2_phat*sample2_size)/(sample1_size+sample2_size)
    zscore = (sample1_phat - sample2_phat)/np.sqrt(phat*(1-phat)*((1/sample1_size)+(1/sample2_size)))    
    pval = st.norm.sf(abs(zscore))*2
linregdf.loc["ALL YRS"].iloc[11] = pval

for x in range(len(yr)):
    sample1_phat = linregdf.loc["ALL YRS"].iloc[1]
    sample2_phat = linregdf.loc["ALL YRS"].iloc[3]
    sample1_size = linregdf.loc["ALL YRS"].iloc[5]
    sample2_size = linregdf.loc["ALL YRS"].iloc[7]
    phat = (sample1_phat*sample1_size+sample2_phat*sample2_size)/(sample1_size+sample2_size)
    zscore = (sample1_phat - sample2_phat)/np.sqrt(phat*(1-phat)*((1/sample1_size)+(1/sample2_size)))    
    pval = st.norm.sf(abs(zscore))*2
linregdf.loc["ALL YRS"].iloc[12] = pval

for x in range(len(yr)):
    sample1_phat = linregdf.loc["ALL YRS"].iloc[2]
    sample2_phat = linregdf.loc["ALL YRS"].iloc[3]
    sample1_size = linregdf.loc["ALL YRS"].iloc[6]
    sample2_size = linregdf.loc["ALL YRS"].iloc[7]
    phat = (sample1_phat*sample1_size+sample2_phat*sample2_size)/(sample1_size+sample2_size)
    zscore = (sample1_phat - sample2_phat)/np.sqrt(phat*(1-phat)*((1/sample1_size)+(1/sample2_size)))    
    pval = st.norm.sf(abs(zscore))*2
linregdf.loc["ALL YRS"].iloc[13] = pval

linregdf = linregdf.head(9).style.format("{:,.2f}")
linregdf

In [ ]:
# DIVING DEEPER INTO THE MINIMUM REQUIRED EDUCATION LEVEL
#ANCHORED BY STATE: PW SKILL,APPLICATIONS, MEDIAN WAGE, YRS POST GRADE, MEDIAN DAYS

code = "LOWER"
# CHANGE THIS FOR ANY CODE

# MEDIAN WAGE, MEDIAN YEARS AFTER GRADUATING, MEDIAN DAYS TO DECIDE : BY PW LEVEL
topstate = df[df["REQ_EDU_GROUP"] == code]["WORKSITE_STATE"].value_counts()[:15].index.tolist()
pwskill = df["PW_SKILL_LEVEL"].dropna().unique().tolist()
pwskill.sort()


statelist = []
pwlist = pwskill*len(topstate)
apps = []
medwage = []
yrsgrad = []
meddays = []


for x in range(len(topstate)):
    s = (topstate[x]+",")*len(pwskill)
    s = "".join(s).split(",")
    s = list(filter(("").__ne__, s))
    statelist.append(s) 
for x in range(len(topstate)):
    for j in range(len(pwskill)):
        a = len(df[(df["REQ_EDU_GROUP"] == code)&(df["WORKSITE_STATE"] == topstate[x])][df[(df["REQ_EDU_GROUP"] == code)&(df["WORKSITE_STATE"] == topstate[x])]["PW_SKILL_LEVEL"]==pwskill[j]])
        apps.append(a)
        b = df[(df["REQ_EDU_GROUP"] == code)&(df["WORKSITE_STATE"] == topstate[x])][df[(df["REQ_EDU_GROUP"] == code)&(df["WORKSITE_STATE"] == topstate[x])]["PW_SKILL_LEVEL"]==pwskill[j]]["WO_A"].median()
        medwage.append(b)
        c = df[(df["REQ_EDU_GROUP"] == code)&(df["WORKSITE_STATE"] == topstate[x])][df[(df["REQ_EDU_GROUP"] == code)&(df["WORKSITE_STATE"] == topstate[x])]["PW_SKILL_LEVEL"]==pwskill[j]]["YRSPOSTGRAD"].median()
        yrsgrad.append(c)
        d = df[(df["REQ_EDU_GROUP"] == code)&(df["WORKSITE_STATE"] == topstate[x])][df[(df["REQ_EDU_GROUP"] == code)&(df["WORKSITE_STATE"] == topstate[x])]["PW_SKILL_LEVEL"]==pwskill[j]]["DAYS_TO_DECIDE"].median()
        meddays.append(d)
codestatedf = pd.DataFrame([sum(statelist, []),pwlist,apps,medwage,yrsgrad,meddays]).transpose()
codestatedf.columns = ["STATE","PW LEVEL", "TOTAL APPLICATIONS","MEDIAN WAGE","MEDIAN YRS POST GRAD","MEDIAN DAYS TO DECIDE"]
codestatedf = codestatedf.set_index(["STATE","PW LEVEL"])
codestatedf

In [ ]:
# PW LEVEL DISTRIBUTION ANALYSIS
# PLOTTING SUCCESS RATES OVER TIME BY PW LEVEL SKILL

lower = df[df["PW_SKILL_LEVEL"]=="LEVEL I"]
college = df[df["PW_SKILL_LEVEL"]=="LEVEL II"]
upper = df[df["PW_SKILL_LEVEL"]=="LEVEL III"]
other = df[df["PW_SKILL_LEVEL"]=="LEVEL IV"]



lowerdf = pd.DataFrame()
collegedf = pd.DataFrame()
upperdf = pd.DataFrame()
otherdf = pd.DataFrame()
linregdf = pd.DataFrame()

for x in range(len(yr)):
    lowe = pd.DataFrame([len(lower[(lower["YEAR"]==yr[x])&(lower["CASE_STATUS"]==1)])/len(lower[lower["YEAR"]==yr[x]])])
    coll = pd.DataFrame([len(college[(college["YEAR"]==yr[x])&(college["CASE_STATUS"]==1)])/len(college[college["YEAR"]==yr[x]])])
    upp = pd.DataFrame([len(upper[(upper["YEAR"]==yr[x])&(upper["CASE_STATUS"]==1)])/len(upper[upper["YEAR"]==yr[x]])])
    ott = pd.DataFrame([len(other[(other["YEAR"]==yr[x])&(other["CASE_STATUS"]==1)])/len(other[other["YEAR"]==yr[x]])])
    lowerdf = pd.concat([lowerdf,lowe])
    collegedf = pd.concat([collegedf,coll])
    upperdf = pd.concat([upperdf,upp])
    otherdf = pd.concat([otherdf, ott])
    linregdf = pd.concat([lowerdf,collegedf,upperdf,otherdf], axis = 1)
    

linregdf.index = yr
linregdf.columns = ["Level 1 Success %", "Level 2 Success %", "Level 3 Success %","Level 4 Success %"]
plt.figure(figsize = (12,10), dpi = 100)
linregdf.plot(kind = "bar")
plt.tight_layout()
plt.legend(bbox_to_anchor=(1.05, 1), loc=2, borderaxespad=0.)
plt.title("Success Rates by PW Skill Level, Over Time")

In [ ]:
# PREPARING DATA FOR LIN REGRESS AND GETTING P VALUE
lowerapps = []
collegeapps = []
upperapps = []
otherapps = []

for x in range(len(yr)):
    l = (lower["YEAR"]==yr[x]).sum()
    c = (college["YEAR"]==yr[x]).sum()
    u = (upper["YEAR"]==yr[x]).sum()
    o = (other["YEAR"]==yr[x]).sum()
    lowerapps.append(l)
    collegeapps.append(c)
    upperapps.append(u)
    otherapps.append(o)
linregdf = linregdf.reset_index()
linregdf["Level 1 Apps"] = pd.DataFrame(lowerapps)
linregdf["Level 2 Apps"] = pd.DataFrame(collegeapps)
linregdf["Level 3 Apps"] = pd.DataFrame(upperapps)
linregdf["Level 4 Apps"] = pd.DataFrame(otherapps)
linregdf = linregdf.set_index("index")
# PLOTTING APPLICATION RATES OVER TIME BY SKILL LEVEL
plt.figure(figsize = (12,10), dpi = 100)
linregdf.reset_index().plot(x = "index", y = ["Level 1 Apps","Level 2 Apps","Level 3 Apps","Level 4 Apps"], kind = "bar")
plt.tight_layout()
plt.legend(bbox_to_anchor=(1.05, 1), loc=2, borderaxespad=0.)
plt.title("Application Rates by PW Skill Level, Over Time")

In [ ]:
# P VALUE PER YEAR
lower_college = []
for x in range(len(yr)):
    sample1_phat = linregdf.loc[yr[x]]["Level 1 Success %"]
    sample2_phat = linregdf.loc[yr[x]]["Level 2 Success %"]
    sample1_size = linregdf.loc[yr[x]]["Level 1 Apps"]
    sample2_size = linregdf.loc[yr[x]]["Level 2 Apps"]
    phat = (sample1_phat*sample1_size+sample2_phat*sample2_size)/(sample1_size+sample2_size)
    zscore = (sample1_phat - sample2_phat)/np.sqrt(phat*(1-phat)*((1/sample1_size)+(1/sample2_size)))    
    pval = st.norm.sf(abs(zscore))*2
    lower_college.append(pval)

pvaldf = pd.DataFrame(lower_college)

lower_upper = []
for x in range(len(yr)):
    sample1_phat = linregdf.loc[yr[x]]["Level 1 Success %"]
    sample2_phat = linregdf.loc[yr[x]]["Level 3 Success %"]
    sample1_size = linregdf.loc[yr[x]]["Level 1 Apps"]
    sample2_size = linregdf.loc[yr[x]]["Level 3 Apps"]
    phat = (sample1_phat*sample1_size+sample2_phat*sample2_size)/(sample1_size+sample2_size)
    zscore = (sample1_phat - sample2_phat)/np.sqrt(phat*(1-phat)*((1/sample1_size)+(1/sample2_size)))    
    pval = st.norm.sf(abs(zscore))*2
    lower_upper.append(pval)
pvaldf = pd.concat([pvaldf,pd.DataFrame(lower_upper)], axis = 1)

lower_other = []
for x in range(len(yr)):
    sample1_phat = linregdf.loc[yr[x]]["Level 1 Success %"]
    sample2_phat = linregdf.loc[yr[x]]["Level 4 Success %"]
    sample1_size = linregdf.loc[yr[x]]["Level 1 Apps"]
    sample2_size = linregdf.loc[yr[x]]["Level 4 Apps"]
    phat = (sample1_phat*sample1_size+sample2_phat*sample2_size)/(sample1_size+sample2_size)
    zscore = (sample1_phat - sample2_phat)/np.sqrt(phat*(1-phat)*((1/sample1_size)+(1/sample2_size)))    
    pval = st.norm.sf(abs(zscore))*2
    lower_other.append(pval)
pvaldf = pd.concat([pvaldf,pd.DataFrame(lower_other)], axis = 1)

college_upper = []
for x in range(len(yr)):
    sample1_phat = linregdf.loc[yr[x]]["Level 2 Success %"]
    sample2_phat = linregdf.loc[yr[x]]["Level 3 Success %"]
    sample1_size = linregdf.loc[yr[x]]["Level 2 Apps"]
    sample2_size = linregdf.loc[yr[x]]["Level 3 Apps"]
    phat = (sample1_phat*sample1_size+sample2_phat*sample2_size)/(sample1_size+sample2_size)
    zscore = (sample1_phat - sample2_phat)/np.sqrt(phat*(1-phat)*((1/sample1_size)+(1/sample2_size)))    
    pval = st.norm.sf(abs(zscore))*2
    college_upper.append(pval)
pvaldf = pd.concat([pvaldf,pd.DataFrame(college_upper)], axis = 1)

college_other = []
for x in range(len(yr)):
    sample1_phat = linregdf.loc[yr[x]]["Level 2 Success %"]
    sample2_phat = linregdf.loc[yr[x]]["Level 4 Success %"]
    sample1_size = linregdf.loc[yr[x]]["Level 2 Apps"]
    sample2_size = linregdf.loc[yr[x]]["Level 4 Apps"]
    phat = (sample1_phat*sample1_size+sample2_phat*sample2_size)/(sample1_size+sample2_size)
    zscore = (sample1_phat - sample2_phat)/np.sqrt(phat*(1-phat)*((1/sample1_size)+(1/sample2_size)))    
    pval = st.norm.sf(abs(zscore))*2
    college_other.append(pval)
pvaldf = pd.concat([pvaldf,pd.DataFrame(college_other)], axis = 1)

upper_other = []
for x in range(len(yr)):
    sample1_phat = linregdf.loc[yr[x]]["Level 3 Success %"]
    sample2_phat = linregdf.loc[yr[x]]["Level 4 Success %"]
    sample1_size = linregdf.loc[yr[x]]["Level 3 Apps"]
    sample2_size = linregdf.loc[yr[x]]["Level 4 Apps"]
    phat = (sample1_phat*sample1_size+sample2_phat*sample2_size)/(sample1_size+sample2_size)
    zscore = (sample1_phat - sample2_phat)/np.sqrt(phat*(1-phat)*((1/sample1_size)+(1/sample2_size)))    
    pval = st.norm.sf(abs(zscore))*2
    upper_other.append(pval)
pvaldf = pd.concat([pvaldf,pd.DataFrame(upper_other)], axis = 1)

pvaldf.columns = ["L1_L2","L1_L3","L1_L4","L2_L3","L2_L4","L3_L4"]
pvaldf.index = yr
linregdf[["L1_L2","L1_L3","L1_L4","L2_L3","L2_L4","L3_L4"]] = pvaldf[["L1_L2","L1_L3","L1_L4","L2_L3","L2_L4","L3_L4"]]

In [ ]:
# LINEAR REGRESSION BETWEEN PW SKILL LEVEL and success
print("Level 1 success rate over time:",st.linregress(y = linregdf["Level 1 Success %"], x = yr))
print("\nLevel 1 applications over time:",st.linregress(y = linregdf["Level 1 Apps"], x = yr))

print("\n\nLevel 2 success rate over time:",st.linregress(y = linregdf["Level 2 Success %"], x = yr))
print("\nLevel 2 applications over time:",st.linregress(y = linregdf["Level 2 Apps"], x = yr))

print("\n\nLevel 3 success rate over time:",st.linregress(y = linregdf["Level 3 Success %"], x = yr))
print("\nLevel 3 applications over time:",st.linregress(y = linregdf["Level 3 Apps"], x = yr))

print("\n\nLevel 4 success rate over time:",st.linregress(y = linregdf["Level 4 Success %"], x = yr))
print("\nLevel 4 applications over time:",st.linregress(y = linregdf["Level 4 Apps"], x = yr))

In [ ]:
# GETTING THE TOTAL PVAL
linregdf.loc["ALL YRS"] = linregdf.sum()
lowersuc = []
collegesuc = []
uppersuc = []
othersuc = []

for x in range(len(yr)):
    l = linregdf.loc[yr[x]].iloc[0]*linregdf.loc[yr[x]].iloc[4]
    c = linregdf.loc[yr[x]].iloc[1]*linregdf.loc[yr[x]].iloc[5]
    u = linregdf.loc[yr[x]].iloc[2]*linregdf.loc[yr[x]].iloc[6]
    o = linregdf.loc[yr[x]].iloc[3]*linregdf.loc[yr[x]].iloc[7]
    lowersuc.append(l)
    collegesuc.append(c)
    uppersuc.append(u)
    othersuc.append(o)
    
linregdf.loc["ALL YRS"].iloc[0] = sum(lowersuc)/linregdf.loc["ALL YRS"].iloc[4]
linregdf.loc["ALL YRS"].iloc[1] = sum(collegesuc)/linregdf.loc["ALL YRS"].iloc[5]
linregdf.loc["ALL YRS"].iloc[2] = sum(uppersuc)/linregdf.loc["ALL YRS"].iloc[6]
linregdf.loc["ALL YRS"].iloc[3] = sum(othersuc)/linregdf.loc["ALL YRS"].iloc[7]

for x in range(len(yr)):
    sample1_phat = linregdf.loc["ALL YRS"].iloc[0]
    sample2_phat = linregdf.loc["ALL YRS"].iloc[1]
    sample1_size = linregdf.loc["ALL YRS"].iloc[4]
    sample2_size = linregdf.loc["ALL YRS"].iloc[5]
    phat = (sample1_phat*sample1_size+sample2_phat*sample2_size)/(sample1_size+sample2_size)
    zscore = (sample1_phat - sample2_phat)/np.sqrt(phat*(1-phat)*((1/sample1_size)+(1/sample2_size)))    
    pval = st.norm.sf(abs(zscore))*2
linregdf.loc["ALL YRS"].iloc[8] = pval

for x in range(len(yr)):
    sample1_phat = linregdf.loc["ALL YRS"].iloc[0]
    sample2_phat = linregdf.loc["ALL YRS"].iloc[2]
    sample1_size = linregdf.loc["ALL YRS"].iloc[4]
    sample2_size = linregdf.loc["ALL YRS"].iloc[6]
    phat = (sample1_phat*sample1_size+sample2_phat*sample2_size)/(sample1_size+sample2_size)
    zscore = (sample1_phat - sample2_phat)/np.sqrt(phat*(1-phat)*((1/sample1_size)+(1/sample2_size)))    
    pval = st.norm.sf(abs(zscore))*2
linregdf.loc["ALL YRS"].iloc[9] = pval

for x in range(len(yr)):
    sample1_phat = linregdf.loc["ALL YRS"].iloc[0]
    sample2_phat = linregdf.loc["ALL YRS"].iloc[3]
    sample1_size = linregdf.loc["ALL YRS"].iloc[4]
    sample2_size = linregdf.loc["ALL YRS"].iloc[7]
    phat = (sample1_phat*sample1_size+sample2_phat*sample2_size)/(sample1_size+sample2_size)
    zscore = (sample1_phat - sample2_phat)/np.sqrt(phat*(1-phat)*((1/sample1_size)+(1/sample2_size)))    
    pval = st.norm.sf(abs(zscore))*2
linregdf.loc["ALL YRS"].iloc[10] = pval

for x in range(len(yr)):
    sample1_phat = linregdf.loc["ALL YRS"].iloc[1]
    sample2_phat = linregdf.loc["ALL YRS"].iloc[2]
    sample1_size = linregdf.loc["ALL YRS"].iloc[5]
    sample2_size = linregdf.loc["ALL YRS"].iloc[6]
    phat = (sample1_phat*sample1_size+sample2_phat*sample2_size)/(sample1_size+sample2_size)
    zscore = (sample1_phat - sample2_phat)/np.sqrt(phat*(1-phat)*((1/sample1_size)+(1/sample2_size)))    
    pval = st.norm.sf(abs(zscore))*2
linregdf.loc["ALL YRS"].iloc[11] = pval

for x in range(len(yr)):
    sample1_phat = linregdf.loc["ALL YRS"].iloc[1]
    sample2_phat = linregdf.loc["ALL YRS"].iloc[3]
    sample1_size = linregdf.loc["ALL YRS"].iloc[5]
    sample2_size = linregdf.loc["ALL YRS"].iloc[7]
    phat = (sample1_phat*sample1_size+sample2_phat*sample2_size)/(sample1_size+sample2_size)
    zscore = (sample1_phat - sample2_phat)/np.sqrt(phat*(1-phat)*((1/sample1_size)+(1/sample2_size)))    
    pval = st.norm.sf(abs(zscore))*2
linregdf.loc["ALL YRS"].iloc[12] = pval

for x in range(len(yr)):
    sample1_phat = linregdf.loc["ALL YRS"].iloc[2]
    sample2_phat = linregdf.loc["ALL YRS"].iloc[3]
    sample1_size = linregdf.loc["ALL YRS"].iloc[6]
    sample2_size = linregdf.loc["ALL YRS"].iloc[7]
    phat = (sample1_phat*sample1_size+sample2_phat*sample2_size)/(sample1_size+sample2_size)
    zscore = (sample1_phat - sample2_phat)/np.sqrt(phat*(1-phat)*((1/sample1_size)+(1/sample2_size)))    
    pval = st.norm.sf(abs(zscore))*2
linregdf.loc["ALL YRS"].iloc[13] = pval

linregdf = linregdf.head(9).style.format("{:,.2f}")
linregdf

In [ ]:
# DEEPER DIVE INTO ALL PW SKILL LEVELS!
# ANCHORED BY STATE: EDU LEVEL,APPLICATIONS, MEDIAN WAGE, YRS POST GRADE, MEDIAN DAYS
# MEDIAN WAGE, MEDIAN YEARS AFTER GRADUATING, MEDIAN DAYS TO DECIDE : BY EDU LEVEL
code = "LEVEL IV"
# CHANGE THIS FOR ANY CODE
topstate = df[df["PW_SKILL_LEVEL"] == code]["WORKSITE_STATE"].value_counts()[:15].index.tolist()
edu = df["FOREIGN_WORKER_EDUCATION"].dropna().unique().tolist()
edu[0],edu[1],edu[2],edu[3],edu[4],edu[5] = edu[3],edu[5],edu[0],edu[2],edu[1],edu[4]

statelist = []
edulist = edu*len(topstate)
apps = []
medwage = []
yrsgrad = []
meddays = []


for x in range(len(topstate)):
    s = (topstate[x]+",")*len(edu)
    s = "".join(s).split(",")
    s = list(filter(("").__ne__, s))
    statelist.append(s) 
for x in range(len(topstate)):
    for j in range(len(edu)):
        a = len(df[(df["PW_SKILL_LEVEL"] == code)&(df["WORKSITE_STATE"] == topstate[x])][df[(df["PW_SKILL_LEVEL"] == code)&(df["WORKSITE_STATE"] == topstate[x])]["FOREIGN_WORKER_EDUCATION"]==edu[j]])
        apps.append(a)
        b = df[(df["PW_SKILL_LEVEL"] == code)&(df["WORKSITE_STATE"] == topstate[x])][df[(df["PW_SKILL_LEVEL"] == code)&(df["WORKSITE_STATE"] == topstate[x])]["FOREIGN_WORKER_EDUCATION"]==edu[j]]["WO_A"].median()
        medwage.append(b)
        c = df[(df["PW_SKILL_LEVEL"] == code)&(df["WORKSITE_STATE"] == topstate[x])][df[(df["PW_SKILL_LEVEL"] == code)&(df["WORKSITE_STATE"] == topstate[x])]["FOREIGN_WORKER_EDUCATION"]==edu[j]]["YRSPOSTGRAD"].median()
        yrsgrad.append(c)
        d = df[(df["PW_SKILL_LEVEL"] == code)&(df["WORKSITE_STATE"] == topstate[x])][df[(df["PW_SKILL_LEVEL"] == code)&(df["WORKSITE_STATE"] == topstate[x])]["FOREIGN_WORKER_EDUCATION"]==edu[j]]["DAYS_TO_DECIDE"].median()
        meddays.append(d)
codestatedf1 = pd.DataFrame([sum(statelist, []),edulist,apps,medwage,yrsgrad,meddays]).transpose()
codestatedf1.columns = ["STATE","EDU LEVEL", "TOTAL APPLICATIONS","MEDIAN WAGE","MEDIAN YRS POST GRAD","MEDIAN DAYS TO DECIDE"]
codestatedf1 = codestatedf1.set_index(["STATE","EDU LEVEL"])
codestatedf1

In [ ]:
# PROFESSIONAL OCCUPATION?
# DIVING DEEPER INTO IF PROFESSIONAL OCCUPATION!
#ANCHORED BY STATE: PW SKILL,APPLICATIONS, MEDIAN WAGE, YRS POST GRADE, MEDIAN DAYS

code = 1
# CHANGE THIS FOR ANY CODE

# MEDIAN WAGE, MEDIAN YEARS AFTER GRADUATING, MEDIAN DAYS TO DECIDE : FOR EACH STATE IN THE TOP 15 OF THE COMPANY AND BY PW LEVEL
topstate = df[df["PROFESSIONAL_OCCUPATION"] == code]["WORKSITE_STATE"].value_counts()[:15].index.tolist()
pwskill = df["PW_SKILL_LEVEL"].dropna().unique().tolist()
pwskill.sort()


statelist = []
pwlist = pwskill*len(topstate)
apps = []
medwage = []
yrsgrad = []
meddays = []


for x in range(len(topstate)):
    s = (topstate[x]+",")*len(pwskill)
    s = "".join(s).split(",")
    s = list(filter(("").__ne__, s))
    statelist.append(s) 
for x in range(len(topstate)):
    for j in range(len(pwskill)):
        a = len(df[(df["PROFESSIONAL_OCCUPATION"] == code)&(df["WORKSITE_STATE"] == topstate[x])][df[(df["PROFESSIONAL_OCCUPATION"] == code)&(df["WORKSITE_STATE"] == topstate[x])]["PW_SKILL_LEVEL"]==pwskill[j]])
        apps.append(a)
        b = df[(df["PROFESSIONAL_OCCUPATION"] == code)&(df["WORKSITE_STATE"] == topstate[x])][df[(df["PROFESSIONAL_OCCUPATION"] == code)&(df["WORKSITE_STATE"] == topstate[x])]["PW_SKILL_LEVEL"]==pwskill[j]]["WO_A"].median()
        medwage.append(b)
        c = df[(df["PROFESSIONAL_OCCUPATION"] == code)&(df["WORKSITE_STATE"] == topstate[x])][df[(df["PROFESSIONAL_OCCUPATION"] == code)&(df["WORKSITE_STATE"] == topstate[x])]["PW_SKILL_LEVEL"]==pwskill[j]]["YRSPOSTGRAD"].median()
        yrsgrad.append(c)
        d = df[(df["PROFESSIONAL_OCCUPATION"] == code)&(df["WORKSITE_STATE"] == topstate[x])][df[(df["PROFESSIONAL_OCCUPATION"] == code)&(df["WORKSITE_STATE"] == topstate[x])]["PW_SKILL_LEVEL"]==pwskill[j]]["DAYS_TO_DECIDE"].median()
        meddays.append(d)
codestatedf = pd.DataFrame([sum(statelist, []),pwlist,apps,medwage,yrsgrad,meddays]).transpose()
codestatedf.columns = ["STATE","PW LEVEL", "TOTAL APPLICATIONS","MEDIAN WAGE","MEDIAN YRS POST GRAD","MEDIAN DAYS TO DECIDE"]
codestatedf = codestatedf.set_index(["STATE","PW LEVEL"])
codestatedf

In [ ]:
# ANCHORED BY STATE: EDU LEVEL,APPLICATIONS, MEDIAN WAGE, YRS POST GRADE, MEDIAN DAYS
# MEDIAN WAGE, MEDIAN YEARS AFTER GRADUATING, MEDIAN DAYS TO DECIDE : FOR EACH STATE IN THE TOP 15 OF THE COMPANY AND BY EDU LEVEL
edu = df["FOREIGN_WORKER_EDUCATION"].dropna().unique().tolist()
edu[0],edu[1],edu[2],edu[3],edu[4],edu[5] = edu[3],edu[5],edu[0],edu[2],edu[1],edu[4]

statelist = []
edulist = edu*len(topstate)
apps = []
medwage = []
yrsgrad = []
meddays = []


for x in range(len(topstate)):
    s = (topstate[x]+",")*len(edu)
    s = "".join(s).split(",")
    s = list(filter(("").__ne__, s))
    statelist.append(s) 
for x in range(len(topstate)):
    for j in range(len(edu)):
        a = len(df[(df["PROFESSIONAL_OCCUPATION"] == code)&(df["WORKSITE_STATE"] == topstate[x])][df[(df["PROFESSIONAL_OCCUPATION"] == code)&(df["WORKSITE_STATE"] == topstate[x])]["FOREIGN_WORKER_EDUCATION"]==edu[j]])
        apps.append(a)
        b = df[(df["PROFESSIONAL_OCCUPATION"] == code)&(df["WORKSITE_STATE"] == topstate[x])][df[(df["PROFESSIONAL_OCCUPATION"] == code)&(df["WORKSITE_STATE"] == topstate[x])]["FOREIGN_WORKER_EDUCATION"]==edu[j]]["WO_A"].median()
        medwage.append(b)
        c = df[(df["PROFESSIONAL_OCCUPATION"] == code)&(df["WORKSITE_STATE"] == topstate[x])][df[(df["PROFESSIONAL_OCCUPATION"] == code)&(df["WORKSITE_STATE"] == topstate[x])]["FOREIGN_WORKER_EDUCATION"]==edu[j]]["YRSPOSTGRAD"].median()
        yrsgrad.append(c)
        d = df[(df["PROFESSIONAL_OCCUPATION"] == code)&(df["WORKSITE_STATE"] == topstate[x])][df[(df["PROFESSIONAL_OCCUPATION"] == code)&(df["WORKSITE_STATE"] == topstate[x])]["FOREIGN_WORKER_EDUCATION"]==edu[j]]["DAYS_TO_DECIDE"].median()
        meddays.append(d)
codestatedf1 = pd.DataFrame([sum(statelist, []),edulist,apps,medwage,yrsgrad,meddays]).transpose()
codestatedf1.columns = ["STATE","EDU LEVEL", "TOTAL APPLICATIONS","MEDIAN WAGE","MEDIAN YRS POST GRAD","MEDIAN DAYS TO DECIDE"]
codestatedf1 = codestatedf1.set_index(["STATE","EDU LEVEL"])
codestatedf1

In [ ]:
# PLOTTING THE DIFFERENT SUCCESS RATES OVER TIME BY Professional Occupation or not (IN TERMS OF ANNUAL APPLICATIONS)

top15 = df[df["PROFESSIONAL_OCCUPATION"]==1]
not15 = df[df["PROFESSIONAL_OCCUPATION"]==0]

topdf = pd.DataFrame()
notdf = pd.DataFrame()
analysisdf = pd.DataFrame()

for x in range(len(yr)):
    j = pd.DataFrame([len(top15[(top15["YEAR"]==yr[x])&(top15["CASE_STATUS"]==1)])/len(top15[top15["YEAR"]==yr[x]])])
    b = pd.DataFrame([len(not15[(not15["YEAR"]==yr[x])&(not15["CASE_STATUS"]==1)])/len(not15[not15["YEAR"]==yr[x]])])
    topdf = pd.concat([topdf,j])
    notdf = pd.concat([notdf,b])
    analysisdf = pd.concat([topdf,notdf], axis = 1)

analysisdf.index = yr
analysisdf.columns = ["Professional Success %", "Not Professional Success %"]
plt.figure(figsize = (12,10), dpi = 100)
analysisdf.plot(kind = "bar")
plt.tight_layout()
plt.legend(bbox_to_anchor=(1.05, 1), loc=2, borderaxespad=0.)
plt.title("Success Rates by Professional and not Professional Jobs, Over Time")

In [ ]:
# PREPARING DATA FOR LIN REGRESS AND GETTING P VALUE
top15apps = []
not15apps = []

for x in range(len(yr)):
    t = (top15["YEAR"]==yr[x]).sum()
    n = (not15["YEAR"]==yr[x]).sum()
    top15apps.append(t)
    not15apps.append(n)
analysisdf = analysisdf.reset_index()
analysisdf["Professional Apps"] = pd.DataFrame(top15apps)
analysisdf["Not Professional Apps"] = pd.DataFrame(not15apps)
analysisdf = analysisdf.set_index("index")
# PLOTTING APPLICATION RATES OVER TIME BY Professional or not
plt.figure(figsize = (12,10), dpi = 100)
analysisdf.reset_index().plot(x = "index", y = ["Professional Apps","Not Professional Apps"], kind = "bar")
plt.tight_layout()
plt.legend(bbox_to_anchor=(1.05, 1), loc=2, borderaxespad=0.)
plt.title("Application Rates by Professional and not Professional Jobs, Over Time")

In [ ]:
# P VALUE PER YEAR: ON THE SUCCESS RATE
top_not = []
for x in range(len(yr)):
    sample1_phat = analysisdf.loc[yr[x]]["Professional Success %"]
    sample2_phat = analysisdf.loc[yr[x]]["Not Professional Success %"]
    sample1_size = analysisdf.loc[yr[x]]["Professional Apps"]
    sample2_size = analysisdf.loc[yr[x]]["Not Professional Apps"]
    phat = (sample1_phat*sample1_size+sample2_phat*sample2_size)/(sample1_size+sample2_size)
    zscore = (sample1_phat - sample2_phat)/np.sqrt(phat*(1-phat)*((1/sample1_size)+(1/sample2_size)))    
    pval = st.norm.sf(abs(zscore))*2
    top_not.append(pval)

pvaldf = pd.DataFrame(top_not)
pvaldf.columns = ["Professional Vs. Not Professional"]
pvaldf.index = yr
analysisdf["PVAL Success %"] = pvaldf["Professional Vs. Not Professional"]

In [ ]:
# LINEAR REGRESSION BETWEEN Professional Occupation AND SUCCESS RATE OVER TIME
print("Professional success rate over time:",st.linregress(y = analysisdf["Professional Success %"], x = yr))
print("\nProfessional applications over time:",st.linregress(y = analysisdf["Professional Apps"], x = yr))

print("\n\nNot professional success rate over time:",st.linregress(y = analysisdf["Not Professional Success %"], x = yr))
print("\nNot Professional applications over time:",st.linregress(y = analysisdf["Not Professional Apps"], x = yr))

In [ ]:
# GETTING THE TOTAL PVAL
analysisdf.loc["ALL YRS"] = analysisdf.sum()
top15suc = []
not15suc = []

for x in range(len(yr)):
    t15 = analysisdf.loc[yr[x]].iloc[0]*analysisdf.loc[yr[x]].iloc[2]
    n15 = analysisdf.loc[yr[x]].iloc[1]*analysisdf.loc[yr[x]].iloc[3]
    top15suc.append(t15)
    not15suc.append(n15)
analysisdf.loc["ALL YRS"].iloc[0] = sum(top15suc)/analysisdf.loc["ALL YRS"].iloc[2]
analysisdf.loc["ALL YRS"].iloc[1] = sum(not15suc)/analysisdf.loc["ALL YRS"].iloc[3]


for x in range(len(yr)):
    sample1_phat = analysisdf.loc["ALL YRS"].iloc[0]
    sample2_phat = analysisdf.loc["ALL YRS"].iloc[1]
    sample1_size = analysisdf.loc["ALL YRS"].iloc[2]
    sample2_size = analysisdf.loc["ALL YRS"].iloc[3]
    phat = (sample1_phat*sample1_size+sample2_phat*sample2_size)/(sample1_size+sample2_size)
    zscore = (sample1_phat - sample2_phat)/np.sqrt(phat*(1-phat)*((1/sample1_size)+(1/sample2_size)))    
    pval = st.norm.sf(abs(zscore))*2
analysisdf.loc["ALL YRS"].iloc[4] = pval

analysisdf = analysisdf.head(9).style.format("{:,.2f}")
analysisdf

In [ ]:
# MOST COMMON NEWSPAPER (FIRST) NAMES
print(df['FIRST_NEWSPAPER_NAME'].nunique(),"unique newspapers.")
print("After cleaning,",df['FIRST_NEWSPAPER_NAME'].str.replace('[^\w\s]','', regex = True).str.replace(" ","").str.replace("THE","").nunique(),"newspapers.")

In [ ]:
# NEW COLUMN, CLEANED UP VERSION OF NEWSPAPER NAMES: TOP 10 OR NOT
df["CLEANED_FNEWS"] = df['FIRST_NEWSPAPER_NAME'].str.replace('[^\w\s]','', regex = True).str.replace(" ","").str.replace("THE","")

# Top 10 WORK Cities of all time 2014-2021
allsoc = pd.DataFrame(df["CLEANED_FNEWS"].value_counts()[:10]).reset_index()
allsoc.columns = ["APaper","COUNT APaper"]
sucsoc = pd.DataFrame(df[df["CASE_STATUS"]==1]["CLEANED_FNEWS"].value_counts()[:10]).reset_index()
sucsoc.columns = ["SPaper","COUNT SPaper"]
pd.concat([allsoc,sucsoc], axis = 1)
# Grabbing the top 5 for each year state worked in: total applications
top5aemp = pd.DataFrame()
def grab (year):
    return df[df["YEAR"]==year]["CLEANED_FNEWS"].value_counts()[:10]
for x in range(len(yr)):
    j = pd.DataFrame(grab(yr[x]))
    top5aemp = pd.concat([top5aemp,j], axis = 1)
top5aemp.columns= yr
# King of Consistency: HOW MANY YEARS IS THIS state NOT IN THE TOP 5?: TOP CONSISTENT states...
kingconsA = top5aemp.isnull().replace(False,0).replace(True,1)
kingconsA["Yrs not in Top 10"] = top5aemp.isnull().replace(False,0).replace(True,1).sum(axis = 1)
kingconsA["Yrs not in Top 10"].sort_values()[:10]

In [ ]:
# NEW COLUMN: 1 is in the top 10 of the YEAR, 0 is not: Applications
filler = pd.DataFrame()
for x in range(len(yr)):
    listy = top5aemp[yr[x]].dropna().index.tolist()
    filler = pd.concat([filler,pd.DataFrame((df[df["YEAR"]==yr[x]]["CLEANED_FNEWS"].isin(listy)).replace(True,1).replace(False,0))])

filler = filler.reset_index().sort_values(by = "index").set_index("index")
filler.columns = ["TOP10NEWS"]
df["TOP10NEWS"] = filler

In [ ]:
# PLOTTING THE DIFFERENT SUCCESS RATES OVER TIME BY TOP 10 Newspaper (IN TERMS OF ANNUAL APPLICATIONS)

top15 = df[df["TOP10NEWS"]==1]
not15 = df[df["TOP10NEWS"]==0]

topdf = pd.DataFrame()
notdf = pd.DataFrame()
analysisdf = pd.DataFrame()

for x in range(len(yr)):
    j = pd.DataFrame([len(top15[(top15["YEAR"]==yr[x])&(top15["CASE_STATUS"]==1)])/len(top15[top15["YEAR"]==yr[x]])])
    b = pd.DataFrame([len(not15[(not15["YEAR"]==yr[x])&(not15["CASE_STATUS"]==1)])/len(not15[not15["YEAR"]==yr[x]])])
    topdf = pd.concat([topdf,j])
    notdf = pd.concat([notdf,b])
    analysisdf = pd.concat([topdf,notdf], axis = 1)

analysisdf.index = yr
analysisdf.columns = ["Top 10 Success %", "Not 10 Success %"]
plt.figure(figsize = (12,10), dpi = 100)
analysisdf.plot(kind = "bar")
plt.tight_layout()
plt.legend(bbox_to_anchor=(1.05, 1), loc=2, borderaxespad=0.)
plt.title("Success Rates by Top 10 and not Top 10 Newspapers, Over Time")

In [ ]:
# PREPARING DATA FOR LIN REGRESS AND GETTING P VALUE
top15apps = []
not15apps = []

for x in range(len(yr)):
    t = (top15["YEAR"]==yr[x]).sum()
    n = (not15["YEAR"]==yr[x]).sum()
    top15apps.append(t)
    not15apps.append(n)
analysisdf = analysisdf.reset_index()
analysisdf["Top 10 Apps"] = pd.DataFrame(top15apps)
analysisdf["Not 10 Apps"] = pd.DataFrame(not15apps)
analysisdf = analysisdf.set_index("index")
# PLOTTING APPLICATION RATES OVER TIME BY TOP 15 or NOT TOP 15
plt.figure(figsize = (12,10), dpi = 100)
analysisdf.reset_index().plot(x = "index", y = ["Top 10 Apps","Not 10 Apps"], kind = "bar")
plt.tight_layout()
plt.legend(bbox_to_anchor=(1.05, 1), loc=2, borderaxespad=0.)
plt.title("Application Rates by Top 10 or not Top 10 Newspapers, Over Time")

In [ ]:
# P VALUE PER YEAR: ON THE SUCCESS RATE
top_not = []
for x in range(len(yr)):
    sample1_phat = analysisdf.loc[yr[x]]["Top 10 Success %"]
    sample2_phat = analysisdf.loc[yr[x]]["Not 10 Success %"]
    sample1_size = analysisdf.loc[yr[x]]["Top 10 Apps"]
    sample2_size = analysisdf.loc[yr[x]]["Not 10 Apps"]
    phat = (sample1_phat*sample1_size+sample2_phat*sample2_size)/(sample1_size+sample2_size)
    zscore = (sample1_phat - sample2_phat)/np.sqrt(phat*(1-phat)*((1/sample1_size)+(1/sample2_size)))    
    pval = st.norm.sf(abs(zscore))*2
    top_not.append(pval)

pvaldf = pd.DataFrame(top_not)
pvaldf.columns = ["Top 10 Vs. Not 10"]
pvaldf.index = yr
analysisdf["PVAL Success %"] = pvaldf["Top 10 Vs. Not 10"]

In [ ]:
# LINEAR REGRESSION BETWEEN Professional Occupation AND SUCCESS RATE OVER TIME
print("Top 10 success rate over time:",st.linregress(y = analysisdf["Top 10 Success %"], x = yr))
print("\nTop 10 applications over time:",st.linregress(y = analysisdf["Top 10 Apps"], x = yr))

print("\n\nNot 10 success rate over time:",st.linregress(y = analysisdf["Not 10 Success %"], x = yr))
print("\nNot 10 applications over time:",st.linregress(y = analysisdf["Not 10 Apps"], x = yr))

In [ ]:
# GETTING THE TOTAL PVAL
analysisdf.loc["ALL YRS"] = analysisdf.sum()
top15suc = []
not15suc = []

for x in range(len(yr)):
    t15 = analysisdf.loc[yr[x]].iloc[0]*analysisdf.loc[yr[x]].iloc[2]
    n15 = analysisdf.loc[yr[x]].iloc[1]*analysisdf.loc[yr[x]].iloc[3]
    top15suc.append(t15)
    not15suc.append(n15)
analysisdf.loc["ALL YRS"].iloc[0] = sum(top15suc)/analysisdf.loc["ALL YRS"].iloc[2]
analysisdf.loc["ALL YRS"].iloc[1] = sum(not15suc)/analysisdf.loc["ALL YRS"].iloc[3]


for x in range(len(yr)):
    sample1_phat = analysisdf.loc["ALL YRS"].iloc[0]
    sample2_phat = analysisdf.loc["ALL YRS"].iloc[1]
    sample1_size = analysisdf.loc["ALL YRS"].iloc[2]
    sample2_size = analysisdf.loc["ALL YRS"].iloc[3]
    phat = (sample1_phat*sample1_size+sample2_phat*sample2_size)/(sample1_size+sample2_size)
    zscore = (sample1_phat - sample2_phat)/np.sqrt(phat*(1-phat)*((1/sample1_size)+(1/sample2_size)))    
    pval = st.norm.sf(abs(zscore))*2
analysisdf.loc["ALL YRS"].iloc[4] = pval

analysisdf = analysisdf.head(9).style.format("{:,.2f}")
analysisdf

In [ ]:
# FIRST AD TO DAYS
# CREATING A NEW COLUMN: AD GROUP, SPLITS DAYS INTO THIRDS
df["FIRST_AD_TO_APP_DAYS"] = df["FIRST_AD_TO_APP_DAYS"].replace("#VALUE!",np.nan).astype(float)
high = df.groupby("YEAR")["FIRST_AD_TO_APP_DAYS"].quantile(2/3).tolist()
low = df.groupby("YEAR")["FIRST_AD_TO_APP_DAYS"].quantile(1/3).tolist()
df["AD_GROUP"] = np.nan
for x in range(len(yr)):
    df["AD_GROUP"] = np.where((df["YEAR"]==yr[x])&(df["FIRST_AD_TO_APP_DAYS"]<low[x]),"FAST",df["AD_GROUP"])
    df["AD_GROUP"] = np.where((df["YEAR"]==yr[x])&(df["FIRST_AD_TO_APP_DAYS"]>=low[x])&(df["FIRST_AD_TO_APP_DAYS"]<high[x]),"MEDIUM",df["AD_GROUP"])
    df["AD_GROUP"] = np.where((df["YEAR"]==yr[x])&(df["FIRST_AD_TO_APP_DAYS"]>=high[x]),"SLOW",df["AD_GROUP"])

In [ ]:
# DIVING DEEPER INTO THE TOP CODES!
#ANCHORED BY STATE: PW SKILL,APPLICATIONS, MEDIAN WAGE, YRS POST GRADE, MEDIAN DAYS

code = "FAST"
# CHANGE THIS FOR ANY CODE

# MEDIAN WAGE, MEDIAN YEARS AFTER GRADUATING, MEDIAN DAYS TO DECIDE : BY PW LEVEL WITHIN AD GROUP
topstate = df[df["AD_GROUP"] == code]["WORKSITE_STATE"].value_counts()[:15].index.tolist()
pwskill = df["PW_SKILL_LEVEL"].dropna().unique().tolist()
pwskill.sort()


statelist = []
pwlist = pwskill*len(topstate)
apps = []
medwage = []
yrsgrad = []
meddays = []


for x in range(len(topstate)):
    s = (topstate[x]+",")*len(pwskill)
    s = "".join(s).split(",")
    s = list(filter(("").__ne__, s))
    statelist.append(s) 
for x in range(len(topstate)):
    for j in range(len(pwskill)):
        a = len(df[(df["AD_GROUP"] == code)&(df["WORKSITE_STATE"] == topstate[x])][df[(df["AD_GROUP"] == code)&(df["WORKSITE_STATE"] == topstate[x])]["PW_SKILL_LEVEL"]==pwskill[j]])
        apps.append(a)
        b = df[(df["AD_GROUP"] == code)&(df["WORKSITE_STATE"] == topstate[x])][df[(df["AD_GROUP"] == code)&(df["WORKSITE_STATE"] == topstate[x])]["PW_SKILL_LEVEL"]==pwskill[j]]["WO_A"].median()
        medwage.append(b)
        c = df[(df["AD_GROUP"] == code)&(df["WORKSITE_STATE"] == topstate[x])][df[(df["AD_GROUP"] == code)&(df["WORKSITE_STATE"] == topstate[x])]["PW_SKILL_LEVEL"]==pwskill[j]]["YRSPOSTGRAD"].median()
        yrsgrad.append(c)
        d = df[(df["AD_GROUP"] == code)&(df["WORKSITE_STATE"] == topstate[x])][df[(df["AD_GROUP"] == code)&(df["WORKSITE_STATE"] == topstate[x])]["PW_SKILL_LEVEL"]==pwskill[j]]["DAYS_TO_DECIDE"].median()
        meddays.append(d)
codestatedf = pd.DataFrame([sum(statelist, []),pwlist,apps,medwage,yrsgrad,meddays]).transpose()
codestatedf.columns = ["STATE","PW LEVEL", "TOTAL APPLICATIONS","MEDIAN WAGE","MEDIAN YRS POST GRAD","MEDIAN DAYS TO DECIDE"]
codestatedf = codestatedf.set_index(["STATE","PW LEVEL"])
codestatedf

In [ ]:
# ANCHORED BY STATE: EDU LEVEL,APPLICATIONS, MEDIAN WAGE, YRS POST GRADE, MEDIAN DAYS
# MEDIAN WAGE, MEDIAN YEARS AFTER GRADUATING, MEDIAN DAYS TO DECIDE : BY EDU LEVEL WITHIN AD GROUP
edu = df["FOREIGN_WORKER_EDUCATION"].dropna().unique().tolist()
edu[0],edu[1],edu[2],edu[3],edu[4],edu[5] = edu[3],edu[5],edu[0],edu[2],edu[1],edu[4]

statelist = []
edulist = edu*len(topstate)
apps = []
medwage = []
yrsgrad = []
meddays = []


for x in range(len(topstate)):
    s = (topstate[x]+",")*len(edu)
    s = "".join(s).split(",")
    s = list(filter(("").__ne__, s))
    statelist.append(s) 
for x in range(len(topstate)):
    for j in range(len(edu)):
        a = len(df[(df["AD_GROUP"] == code)&(df["WORKSITE_STATE"] == topstate[x])][df[(df["AD_GROUP"] == code)&(df["WORKSITE_STATE"] == topstate[x])]["FOREIGN_WORKER_EDUCATION"]==edu[j]])
        apps.append(a)
        b = df[(df["AD_GROUP"] == code)&(df["WORKSITE_STATE"] == topstate[x])][df[(df["AD_GROUP"] == code)&(df["WORKSITE_STATE"] == topstate[x])]["FOREIGN_WORKER_EDUCATION"]==edu[j]]["WO_A"].median()
        medwage.append(b)
        c = df[(df["AD_GROUP"] == code)&(df["WORKSITE_STATE"] == topstate[x])][df[(df["AD_GROUP"] == code)&(df["WORKSITE_STATE"] == topstate[x])]["FOREIGN_WORKER_EDUCATION"]==edu[j]]["YRSPOSTGRAD"].median()
        yrsgrad.append(c)
        d = df[(df["AD_GROUP"] == code)&(df["WORKSITE_STATE"] == topstate[x])][df[(df["AD_GROUP"] == code)&(df["WORKSITE_STATE"] == topstate[x])]["FOREIGN_WORKER_EDUCATION"]==edu[j]]["DAYS_TO_DECIDE"].median()
        meddays.append(d)
codestatedf1 = pd.DataFrame([sum(statelist, []),edulist,apps,medwage,yrsgrad,meddays]).transpose()
codestatedf1.columns = ["STATE","EDU LEVEL", "TOTAL APPLICATIONS","MEDIAN WAGE","MEDIAN YRS POST GRAD","MEDIAN DAYS TO DECIDE"]
codestatedf1 = codestatedf1.set_index(["STATE","EDU LEVEL"])
codestatedf1

In [ ]:
# CORRELATION ANALYSIS: AGGREGATE BY SIZE BRACKETS AND YEARLY
# PLOTTING SUCCESS RATES OVER TIME BY SIZE OF COMPANY

young = df[df["AD_GROUP"]=="FAST"]
mid = df[df["AD_GROUP"]=="MEDIUM"]
old = df[df["AD_GROUP"]=="SLOW"]


youndf = pd.DataFrame()
middf = pd.DataFrame()
olddf = pd.DataFrame()
linregdf = pd.DataFrame()

for x in range(len(yr)):
    j = pd.DataFrame([len(young[(young["YEAR"]==yr[x])&(young["CASE_STATUS"]==1)])/len(young[young["YEAR"]==yr[x]])])
    med = pd.DataFrame([len(mid[(mid["YEAR"]==yr[x])&(mid["CASE_STATUS"]==1)])/len(mid[mid["YEAR"]==yr[x]])])
    oldy = pd.DataFrame([len(old[(old["YEAR"]==yr[x])&(old["CASE_STATUS"]==1)])/len(old[old["YEAR"]==yr[x]])])
    youndf = pd.concat([youndf,j])
    middf = pd.concat([middf,med])
    olddf = pd.concat([olddf,oldy])
    linregdf = pd.concat([youndf,middf,olddf], axis = 1)
    

linregdf.index = yr
linregdf.columns = ["Fast Success %", "Medium Success %", "Slow Success %"]
plt.figure(figsize = (12,10), dpi = 100)
linregdf.plot(kind = "bar")
plt.tight_layout()
plt.legend(bbox_to_anchor=(1.05, 1), loc=2, borderaxespad=0.)
plt.title("Success Rates by Time To Apply Since First Ad, Over Time")


In [ ]:
# PREPARING DATA FOR LIN REGRESS AND GETTING P VALUE
youngapps = []
midapps = []
oldapps = []

for x in range(len(yr)):
    y = (young["YEAR"]==yr[x]).sum()
    m = (mid["YEAR"]==yr[x]).sum()
    o = (old["YEAR"]==yr[x]).sum()
    youngapps.append(y)
    midapps.append(m)
    oldapps.append(o)
linregdf = linregdf.reset_index()
linregdf["Fast Apps"] = pd.DataFrame(youngapps)
linregdf["Medium Apps"] = pd.DataFrame(midapps)
linregdf["Slow Apps"] = pd.DataFrame(oldapps)
linregdf = linregdf.set_index("index")
# PLOTTING APPLICATION RATES OVER TIME BY SIZE OF COMPANY
plt.figure(figsize = (12,10), dpi = 100)
linregdf.reset_index().plot(x = "index", y = ["Fast Apps","Medium Apps","Slow Apps"], kind = "bar")
plt.tight_layout()
plt.legend(bbox_to_anchor=(1.05, 1), loc=2, borderaxespad=0.)
plt.title("Application Rates by Time To Apply Since First Ad, Over Time")

In [ ]:
# P VALUE PER YEAR
young_med = []
for x in range(len(yr)):
    sample1_phat = linregdf.loc[yr[x]]["Fast Success %"]
    sample2_phat = linregdf.loc[yr[x]]["Medium Success %"]
    sample1_size = linregdf.loc[yr[x]]["Fast Apps"]
    sample2_size = linregdf.loc[yr[x]]["Medium Apps"]
    phat = (sample1_phat*sample1_size+sample2_phat*sample2_size)/(sample1_size+sample2_size)
    zscore = (sample1_phat - sample2_phat)/np.sqrt(phat*(1-phat)*((1/sample1_size)+(1/sample2_size)))    
    pval = st.norm.sf(abs(zscore))*2
    young_med.append(pval)

pvaldf = pd.DataFrame(young_med)

young_old = []
for x in range(len(yr)):
    sample1_phat = linregdf.loc[yr[x]]["Fast Success %"]
    sample2_phat = linregdf.loc[yr[x]]["Slow Success %"]
    sample1_size = linregdf.loc[yr[x]]["Fast Apps"]
    sample2_size = linregdf.loc[yr[x]]["Slow Apps"]
    phat = (sample1_phat*sample1_size+sample2_phat*sample2_size)/(sample1_size+sample2_size)
    zscore = (sample1_phat - sample2_phat)/np.sqrt(phat*(1-phat)*((1/sample1_size)+(1/sample2_size)))    
    pval = st.norm.sf(abs(zscore))*2
    young_old.append(pval)
pvaldf = pd.concat([pvaldf,pd.DataFrame(young_old)], axis = 1)

med_old = []
for x in range(len(yr)):
    sample1_phat = linregdf.loc[yr[x]]["Medium Success %"]
    sample2_phat = linregdf.loc[yr[x]]["Slow Success %"]
    sample1_size = linregdf.loc[yr[x]]["Medium Apps"]
    sample2_size = linregdf.loc[yr[x]]["Slow Apps"]
    phat = (sample1_phat*sample1_size+sample2_phat*sample2_size)/(sample1_size+sample2_size)
    zscore = (sample1_phat - sample2_phat)/np.sqrt(phat*(1-phat)*((1/sample1_size)+(1/sample2_size)))    
    pval = st.norm.sf(abs(zscore))*2
    med_old.append(pval)
pvaldf = pd.concat([pvaldf,pd.DataFrame(med_old)], axis = 1)
pvaldf.columns = ["Fast_Med","Fast_Slow","Med_Slow"]
pvaldf.index = yr
linregdf[["Fast_Med","Fast_Slow","Med_Slow"]] = pvaldf[["Fast_Med","Fast_Slow","Med_Slow"]]


In [ ]:
# LINEAR REGRESSION BETWEEN Speed to Apply and success
print("Fast success rate over time:",st.linregress(y = linregdf["Fast Success %"], x = yr))
print("\nFast applications over time:",st.linregress(y = linregdf["Fast Apps"], x = yr))

print("\n\nMedium success rate over time:",st.linregress(y = linregdf["Medium Success %"], x = yr))
print("\nMedium applications over time:",st.linregress(y = linregdf["Medium Apps"], x = yr))

print("\n\nSlow success rate over time:",st.linregress(y = linregdf["Slow Success %"], x = yr))
print("\nSlow applications over time:",st.linregress(y = linregdf["Slow Apps"], x = yr))

In [ ]:
# GETTING THE TOTAL PVAL
linregdf.loc["ALL YRS"] = linregdf.sum()
youngsuc = []
medsuc = []
oldsuc = []
for x in range(len(yr)):
    y = linregdf.loc[yr[x]].iloc[0]*linregdf.loc[yr[x]].iloc[3]
    m = linregdf.loc[yr[x]].iloc[1]*linregdf.loc[yr[x]].iloc[4]
    o = linregdf.loc[yr[x]].iloc[2]*linregdf.loc[yr[x]].iloc[5]
    youngsuc.append(y)
    medsuc.append(m)
    oldsuc.append(o)
linregdf.loc["ALL YRS"].iloc[0] = sum(youngsuc)/linregdf.loc["ALL YRS"].iloc[3]
linregdf.loc["ALL YRS"].iloc[1] = sum(medsuc)/linregdf.loc["ALL YRS"].iloc[4]
linregdf.loc["ALL YRS"].iloc[2] = sum(oldsuc)/linregdf.loc["ALL YRS"].iloc[5]


for x in range(len(yr)):
    sample1_phat = linregdf.loc["ALL YRS"].iloc[0]
    sample2_phat = linregdf.loc["ALL YRS"].iloc[1]
    sample1_size = linregdf.loc["ALL YRS"].iloc[3]
    sample2_size = linregdf.loc["ALL YRS"].iloc[4]
    phat = (sample1_phat*sample1_size+sample2_phat*sample2_size)/(sample1_size+sample2_size)
    zscore = (sample1_phat - sample2_phat)/np.sqrt(phat*(1-phat)*((1/sample1_size)+(1/sample2_size)))    
    pval = st.norm.sf(abs(zscore))*2
linregdf.loc["ALL YRS"].iloc[6] = pval

for x in range(len(yr)):
    sample1_phat = linregdf.loc["ALL YRS"].iloc[0]
    sample2_phat = linregdf.loc["ALL YRS"].iloc[2]
    sample1_size = linregdf.loc["ALL YRS"].iloc[3]
    sample2_size = linregdf.loc["ALL YRS"].iloc[5]
    phat = (sample1_phat*sample1_size+sample2_phat*sample2_size)/(sample1_size+sample2_size)
    zscore = (sample1_phat - sample2_phat)/np.sqrt(phat*(1-phat)*((1/sample1_size)+(1/sample2_size)))    
    pval = st.norm.sf(abs(zscore))*2
linregdf.loc["ALL YRS"].iloc[7] = pval

for x in range(len(yr)):
    sample1_phat = linregdf.loc["ALL YRS"].iloc[1]
    sample2_phat = linregdf.loc["ALL YRS"].iloc[2]
    sample1_size = linregdf.loc["ALL YRS"].iloc[4]
    sample2_size = linregdf.loc["ALL YRS"].iloc[5]
    phat = (sample1_phat*sample1_size+sample2_phat*sample2_size)/(sample1_size+sample2_size)
    zscore = (sample1_phat - sample2_phat)/np.sqrt(phat*(1-phat)*((1/sample1_size)+(1/sample2_size)))    
    pval = st.norm.sf(abs(zscore))*2
linregdf.loc["ALL YRS"].iloc[8] = pval
linregdf = linregdf.head(9).style.format("{:,.2f}")
linregdf

In [ ]:
# EMPLOYEE REF PROGRAM
# PLOTTING THE DIFFERENT SUCCESS RATES OVER TIME BY employee ref or not (IN TERMS OF ANNUAL APPLICATIONS)

top15 = df[df["EMPLOYEE_REF_PROG"]==1]
not15 = df[df["EMPLOYEE_REF_PROG"]==0]

topdf = pd.DataFrame()
notdf = pd.DataFrame()
analysisdf = pd.DataFrame()

for x in range(len(yr)):
    j = pd.DataFrame([len(top15[(top15["YEAR"]==yr[x])&(top15["CASE_STATUS"]==1)])/len(top15[top15["YEAR"]==yr[x]])])
    b = pd.DataFrame([len(not15[(not15["YEAR"]==yr[x])&(not15["CASE_STATUS"]==1)])/len(not15[not15["YEAR"]==yr[x]])])
    topdf = pd.concat([topdf,j])
    notdf = pd.concat([notdf,b])
    analysisdf = pd.concat([topdf,notdf], axis = 1)

analysisdf.index = yr
analysisdf.columns = ["Ref Success %", "Not Ref Success %"]
plt.figure(figsize = (12,10), dpi = 100)
analysisdf.plot(kind = "bar")
plt.tight_layout()
plt.legend(bbox_to_anchor=(1.05, 1), loc=2, borderaxespad=0.)
plt.title("Success Rates by advertisement with Referral Program or not, Over Time")

In [ ]:
# PREPARING DATA FOR LIN REGRESS AND GETTING P VALUE
top15apps = []
not15apps = []

for x in range(len(yr)):
    t = (top15["YEAR"]==yr[x]).sum()
    n = (not15["YEAR"]==yr[x]).sum()
    top15apps.append(t)
    not15apps.append(n)
analysisdf = analysisdf.reset_index()
analysisdf["Ref Apps"] = pd.DataFrame(top15apps)
analysisdf["Not Ref Apps"] = pd.DataFrame(not15apps)
analysisdf = analysisdf.set_index("index")
# PLOTTING APPLICATION RATES OVER TIME BY TOP 15 or NOT TOP 15
plt.figure(figsize = (12,10), dpi = 100)
analysisdf.reset_index().plot(x = "index", y = ["Ref Apps","Not Ref Apps"], kind = "bar")
plt.tight_layout()
plt.legend(bbox_to_anchor=(1.05, 1), loc=2, borderaxespad=0.)
plt.title("Application Rates by advertisement with Referral Program or not, Over Time")

In [ ]:
# P VALUE PER YEAR: ON THE SUCCESS RATE
top_not = []
for x in range(len(yr)):
    sample1_phat = analysisdf.loc[yr[x]]["Ref Success %"]
    sample2_phat = analysisdf.loc[yr[x]]["Not Ref Success %"]
    sample1_size = analysisdf.loc[yr[x]]["Ref Apps"]
    sample2_size = analysisdf.loc[yr[x]]["Not Ref Apps"]
    phat = (sample1_phat*sample1_size+sample2_phat*sample2_size)/(sample1_size+sample2_size)
    zscore = (sample1_phat - sample2_phat)/np.sqrt(phat*(1-phat)*((1/sample1_size)+(1/sample2_size)))    
    pval = st.norm.sf(abs(zscore))*2
    top_not.append(pval)

pvaldf = pd.DataFrame(top_not)
pvaldf.columns = ["Ref Vs. Not Ref"]
pvaldf.index = yr
analysisdf["PVAL Success %"] = pvaldf["Ref Vs. Not Ref"]

In [ ]:
# LINEAR REGRESSION BETWEEN Professional Occupation AND SUCCESS RATE OVER TIME
print("Ref success rate over time:",st.linregress(y = analysisdf["Ref Success %"], x = yr))
print("\nRef applications over time:",st.linregress(y = analysisdf["Ref Apps"], x = yr))

print("\n\nNot Ref success rate over time:",st.linregress(y = analysisdf["Not Ref Success %"], x = yr))
print("\nNot Ref applications over time:",st.linregress(y = analysisdf["Not Ref Apps"], x = yr))

In [ ]:
# GETTING THE TOTAL PVAL
analysisdf.loc["ALL YRS"] = analysisdf.sum()
top15suc = []
not15suc = []

for x in range(len(yr)):
    t15 = analysisdf.loc[yr[x]].iloc[0]*analysisdf.loc[yr[x]].iloc[2]
    n15 = analysisdf.loc[yr[x]].iloc[1]*analysisdf.loc[yr[x]].iloc[3]
    top15suc.append(t15)
    not15suc.append(n15)
analysisdf.loc["ALL YRS"].iloc[0] = sum(top15suc)/analysisdf.loc["ALL YRS"].iloc[2]
analysisdf.loc["ALL YRS"].iloc[1] = sum(not15suc)/analysisdf.loc["ALL YRS"].iloc[3]


for x in range(len(yr)):
    sample1_phat = analysisdf.loc["ALL YRS"].iloc[0]
    sample2_phat = analysisdf.loc["ALL YRS"].iloc[1]
    sample1_size = analysisdf.loc["ALL YRS"].iloc[2]
    sample2_size = analysisdf.loc["ALL YRS"].iloc[3]
    phat = (sample1_phat*sample1_size+sample2_phat*sample2_size)/(sample1_size+sample2_size)
    zscore = (sample1_phat - sample2_phat)/np.sqrt(phat*(1-phat)*((1/sample1_size)+(1/sample2_size)))    
    pval = st.norm.sf(abs(zscore))*2
analysisdf.loc["ALL YRS"].iloc[4] = pval

analysisdf = analysisdf.head(9).style.format("{:,.2f}")
analysisdf

In [ ]:
# DIVING DEEPER INTO THE REF PROGRAM!
#ANCHORED BY STATE: PW SKILL,APPLICATIONS, MEDIAN WAGE, YRS POST GRADE, MEDIAN DAYS

code = 1
# CHANGE THIS FOR ANY CODE

# MEDIAN WAGE, MEDIAN YEARS AFTER GRADUATING, MEDIAN DAYS TO DECIDE : BY PW LEVEL
topstate = df[df["EMPLOYEE_REF_PROG"] == code]["WORKSITE_STATE"].value_counts()[:15].index.tolist()
pwskill = df["PW_SKILL_LEVEL"].dropna().unique().tolist()
pwskill.sort()


statelist = []
pwlist = pwskill*len(topstate)
apps = []
medwage = []
yrsgrad = []
meddays = []


for x in range(len(topstate)):
    s = (topstate[x]+",")*len(pwskill)
    s = "".join(s).split(",")
    s = list(filter(("").__ne__, s))
    statelist.append(s) 
for x in range(len(topstate)):
    for j in range(len(pwskill)):
        a = len(df[(df["EMPLOYEE_REF_PROG"] == code)&(df["WORKSITE_STATE"] == topstate[x])][df[(df["EMPLOYEE_REF_PROG"] == code)&(df["WORKSITE_STATE"] == topstate[x])]["PW_SKILL_LEVEL"]==pwskill[j]])
        apps.append(a)
        b = df[(df["EMPLOYEE_REF_PROG"] == code)&(df["WORKSITE_STATE"] == topstate[x])][df[(df["EMPLOYEE_REF_PROG"] == code)&(df["WORKSITE_STATE"] == topstate[x])]["PW_SKILL_LEVEL"]==pwskill[j]]["WO_A"].median()
        medwage.append(b)
        c = df[(df["EMPLOYEE_REF_PROG"] == code)&(df["WORKSITE_STATE"] == topstate[x])][df[(df["EMPLOYEE_REF_PROG"] == code)&(df["WORKSITE_STATE"] == topstate[x])]["PW_SKILL_LEVEL"]==pwskill[j]]["YRSPOSTGRAD"].median()
        yrsgrad.append(c)
        d = df[(df["EMPLOYEE_REF_PROG"] == code)&(df["WORKSITE_STATE"] == topstate[x])][df[(df["EMPLOYEE_REF_PROG"] == code)&(df["WORKSITE_STATE"] == topstate[x])]["PW_SKILL_LEVEL"]==pwskill[j]]["DAYS_TO_DECIDE"].median()
        meddays.append(d)
codestatedf = pd.DataFrame([sum(statelist, []),pwlist,apps,medwage,yrsgrad,meddays]).transpose()
codestatedf.columns = ["STATE","PW LEVEL", "TOTAL APPLICATIONS","MEDIAN WAGE","MEDIAN YRS POST GRAD","MEDIAN DAYS TO DECIDE"]
codestatedf = codestatedf.set_index(["STATE","PW LEVEL"])
codestatedf

In [ ]:
# ANCHORED BY STATE: EDU LEVEL,APPLICATIONS, MEDIAN WAGE, YRS POST GRADE, MEDIAN DAYS
# MEDIAN WAGE, MEDIAN YEARS AFTER GRADUATING, MEDIAN DAYS TO DECIDE : BY EDU LEVEL
edu = df["FOREIGN_WORKER_EDUCATION"].dropna().unique().tolist()
edu[0],edu[1],edu[2],edu[3],edu[4],edu[5] = edu[3],edu[5],edu[0],edu[2],edu[1],edu[4]

statelist = []
edulist = edu*len(topstate)
apps = []
medwage = []
yrsgrad = []
meddays = []


for x in range(len(topstate)):
    s = (topstate[x]+",")*len(edu)
    s = "".join(s).split(",")
    s = list(filter(("").__ne__, s))
    statelist.append(s) 
for x in range(len(topstate)):
    for j in range(len(edu)):
        a = len(df[(df["EMPLOYEE_REF_PROG"] == code)&(df["WORKSITE_STATE"] == topstate[x])][df[(df["EMPLOYEE_REF_PROG"] == code)&(df["WORKSITE_STATE"] == topstate[x])]["FOREIGN_WORKER_EDUCATION"]==edu[j]])
        apps.append(a)
        b = df[(df["EMPLOYEE_REF_PROG"] == code)&(df["WORKSITE_STATE"] == topstate[x])][df[(df["EMPLOYEE_REF_PROG"] == code)&(df["WORKSITE_STATE"] == topstate[x])]["FOREIGN_WORKER_EDUCATION"]==edu[j]]["WO_A"].median()
        medwage.append(b)
        c = df[(df["EMPLOYEE_REF_PROG"] == code)&(df["WORKSITE_STATE"] == topstate[x])][df[(df["EMPLOYEE_REF_PROG"] == code)&(df["WORKSITE_STATE"] == topstate[x])]["FOREIGN_WORKER_EDUCATION"]==edu[j]]["YRSPOSTGRAD"].median()
        yrsgrad.append(c)
        d = df[(df["EMPLOYEE_REF_PROG"] == code)&(df["WORKSITE_STATE"] == topstate[x])][df[(df["EMPLOYEE_REF_PROG"] == code)&(df["WORKSITE_STATE"] == topstate[x])]["FOREIGN_WORKER_EDUCATION"]==edu[j]]["DAYS_TO_DECIDE"].median()
        meddays.append(d)
codestatedf1 = pd.DataFrame([sum(statelist, []),edulist,apps,medwage,yrsgrad,meddays]).transpose()
codestatedf1.columns = ["STATE","EDU LEVEL", "TOTAL APPLICATIONS","MEDIAN WAGE","MEDIAN YRS POST GRAD","MEDIAN DAYS TO DECIDE"]
codestatedf1 = codestatedf1.set_index(["STATE","EDU LEVEL"])
codestatedf1

In [ ]:
# ADVERTISEMENT THROUGH PROFESSIONAL ADVERTISEMENT?
# PLOTTING THE DIFFERENT SUCCESS RATES OVER TIME BY employee ref or not (IN TERMS OF ANNUAL APPLICATIONS)

top15 = df[df["PRO_ORG_AD"]==1]
not15 = df[df["PRO_ORG_AD"]==0]

topdf = pd.DataFrame()
notdf = pd.DataFrame()
analysisdf = pd.DataFrame()

for x in range(len(yr)):
    j = pd.DataFrame([len(top15[(top15["YEAR"]==yr[x])&(top15["CASE_STATUS"]==1)])/len(top15[top15["YEAR"]==yr[x]])])
    b = pd.DataFrame([len(not15[(not15["YEAR"]==yr[x])&(not15["CASE_STATUS"]==1)])/len(not15[not15["YEAR"]==yr[x]])])
    topdf = pd.concat([topdf,j])
    notdf = pd.concat([notdf,b])
    analysisdf = pd.concat([topdf,notdf], axis = 1)

analysisdf.index = yr
analysisdf.columns = ["ProOrg Success %", "Not ProOrg Success %"]
plt.figure(figsize = (12,10), dpi = 100)
analysisdf.plot(kind = "bar")
plt.tight_layout()
plt.legend(bbox_to_anchor=(1.05, 1), loc=2, borderaxespad=0.)
plt.title("Success Rates by advertisement with Professional Organization or not, Over Time")

In [ ]:
# PREPARING DATA FOR LIN REGRESS AND GETTING P VALUE
top15apps = []
not15apps = []

for x in range(len(yr)):
    t = (top15["YEAR"]==yr[x]).sum()
    n = (not15["YEAR"]==yr[x]).sum()
    top15apps.append(t)
    not15apps.append(n)
analysisdf = analysisdf.reset_index()
analysisdf["ProOrg Apps"] = pd.DataFrame(top15apps)
analysisdf["Not ProOrg Apps"] = pd.DataFrame(not15apps)
analysisdf = analysisdf.set_index("index")
# PLOTTING APPLICATION RATES OVER TIME BY TOP 15 or NOT TOP 15
plt.figure(figsize = (12,10), dpi = 100)
analysisdf.reset_index().plot(x = "index", y = ["ProOrg Apps","Not ProOrg Apps"], kind = "bar")
plt.tight_layout()
plt.legend(bbox_to_anchor=(1.05, 1), loc=2, borderaxespad=0.)
plt.title("Application Rates by advertisement with Professional Organization or not, Over Time")

In [ ]:
# P VALUE PER YEAR: ON THE SUCCESS RATE
top_not = []
for x in range(len(yr)):
    sample1_phat = analysisdf.loc[yr[x]]["ProOrg Success %"]
    sample2_phat = analysisdf.loc[yr[x]]["Not ProOrg Success %"]
    sample1_size = analysisdf.loc[yr[x]]["ProOrg Apps"]
    sample2_size = analysisdf.loc[yr[x]]["Not ProOrg Apps"]
    phat = (sample1_phat*sample1_size+sample2_phat*sample2_size)/(sample1_size+sample2_size)
    zscore = (sample1_phat - sample2_phat)/np.sqrt(phat*(1-phat)*((1/sample1_size)+(1/sample2_size)))    
    pval = st.norm.sf(abs(zscore))*2
    top_not.append(pval)

pvaldf = pd.DataFrame(top_not)
pvaldf.columns = ["ProOrg Vs. Not ProOrg"]
pvaldf.index = yr
analysisdf["PVAL Success %"] = pvaldf["ProOrg Vs. Not ProOrg"]

In [ ]:
# LINEAR REGRESSION BETWEEN Professional Advertisement AND SUCCESS RATE OVER TIME
print("ProOrg success rate over time:",st.linregress(y = analysisdf["ProOrg Success %"], x = yr))
print("\nProOrg applications over time:",st.linregress(y = analysisdf["ProOrg Apps"], x = yr))

print("\n\nNot ProOrg success rate over time:",st.linregress(y = analysisdf["Not ProOrg Success %"], x = yr))
print("\nNot ProOrg applications over time:",st.linregress(y = analysisdf["Not ProOrg Apps"], x = yr))

In [ ]:
# GETTING THE TOTAL PVAL
analysisdf.loc["ALL YRS"] = analysisdf.sum()
top15suc = []
not15suc = []

for x in range(len(yr)):
    t15 = analysisdf.loc[yr[x]].iloc[0]*analysisdf.loc[yr[x]].iloc[2]
    n15 = analysisdf.loc[yr[x]].iloc[1]*analysisdf.loc[yr[x]].iloc[3]
    top15suc.append(t15)
    not15suc.append(n15)
analysisdf.loc["ALL YRS"].iloc[0] = sum(top15suc)/analysisdf.loc["ALL YRS"].iloc[2]
analysisdf.loc["ALL YRS"].iloc[1] = sum(not15suc)/analysisdf.loc["ALL YRS"].iloc[3]


for x in range(len(yr)):
    sample1_phat = analysisdf.loc["ALL YRS"].iloc[0]
    sample2_phat = analysisdf.loc["ALL YRS"].iloc[1]
    sample1_size = analysisdf.loc["ALL YRS"].iloc[2]
    sample2_size = analysisdf.loc["ALL YRS"].iloc[3]
    phat = (sample1_phat*sample1_size+sample2_phat*sample2_size)/(sample1_size+sample2_size)
    zscore = (sample1_phat - sample2_phat)/np.sqrt(phat*(1-phat)*((1/sample1_size)+(1/sample2_size)))    
    pval = st.norm.sf(abs(zscore))*2
analysisdf.loc["ALL YRS"].iloc[4] = pval

analysisdf = analysisdf.head(9).style.format("{:,.2f}")
analysisdf

In [ ]:
# DIVING DEEPER INTO THE pro org ad
#ANCHORED BY STATE: PW SKILL,APPLICATIONS, MEDIAN WAGE, YRS POST GRADE, MEDIAN DAYS

code = 1
# CHANGE THIS FOR ANY CODE

# MEDIAN WAGE, MEDIAN YEARS AFTER GRADUATING, MEDIAN DAYS TO DECIDE : BY PW LEVEL
topstate = df[df["PRO_ORG_AD"] == code]["WORKSITE_STATE"].value_counts()[:15].index.tolist()
pwskill = df["PW_SKILL_LEVEL"].dropna().unique().tolist()
pwskill.sort()


statelist = []
pwlist = pwskill*len(topstate)
apps = []
medwage = []
yrsgrad = []
meddays = []


for x in range(len(topstate)):
    s = (topstate[x]+",")*len(pwskill)
    s = "".join(s).split(",")
    s = list(filter(("").__ne__, s))
    statelist.append(s) 
for x in range(len(topstate)):
    for j in range(len(pwskill)):
        a = len(df[(df["PRO_ORG_AD"] == code)&(df["WORKSITE_STATE"] == topstate[x])][df[(df["PRO_ORG_AD"] == code)&(df["WORKSITE_STATE"] == topstate[x])]["PW_SKILL_LEVEL"]==pwskill[j]])
        apps.append(a)
        b = df[(df["PRO_ORG_AD"] == code)&(df["WORKSITE_STATE"] == topstate[x])][df[(df["PRO_ORG_AD"] == code)&(df["WORKSITE_STATE"] == topstate[x])]["PW_SKILL_LEVEL"]==pwskill[j]]["WO_A"].median()
        medwage.append(b)
        c = df[(df["PRO_ORG_AD"] == code)&(df["WORKSITE_STATE"] == topstate[x])][df[(df["PRO_ORG_AD"] == code)&(df["WORKSITE_STATE"] == topstate[x])]["PW_SKILL_LEVEL"]==pwskill[j]]["YRSPOSTGRAD"].median()
        yrsgrad.append(c)
        d = df[(df["PRO_ORG_AD"] == code)&(df["WORKSITE_STATE"] == topstate[x])][df[(df["PRO_ORG_AD"] == code)&(df["WORKSITE_STATE"] == topstate[x])]["PW_SKILL_LEVEL"]==pwskill[j]]["DAYS_TO_DECIDE"].median()
        meddays.append(d)
codestatedf = pd.DataFrame([sum(statelist, []),pwlist,apps,medwage,yrsgrad,meddays]).transpose()
codestatedf.columns = ["STATE","PW LEVEL", "TOTAL APPLICATIONS","MEDIAN WAGE","MEDIAN YRS POST GRAD","MEDIAN DAYS TO DECIDE"]
codestatedf = codestatedf.set_index(["STATE","PW LEVEL"])
codestatedf

In [ ]:
# ANCHORED BY STATE: EDU LEVEL,APPLICATIONS, MEDIAN WAGE, YRS POST GRADE, MEDIAN DAYS
# MEDIAN WAGE, MEDIAN YEARS AFTER GRADUATING, MEDIAN DAYS TO DECIDE : BY EDU LEVEL
edu = df["FOREIGN_WORKER_EDUCATION"].dropna().unique().tolist()
edu[0],edu[1],edu[2],edu[3],edu[4],edu[5] = edu[3],edu[5],edu[0],edu[2],edu[1],edu[4]

statelist = []
edulist = edu*len(topstate)
apps = []
medwage = []
yrsgrad = []
meddays = []


for x in range(len(topstate)):
    s = (topstate[x]+",")*len(edu)
    s = "".join(s).split(",")
    s = list(filter(("").__ne__, s))
    statelist.append(s) 
for x in range(len(topstate)):
    for j in range(len(edu)):
        a = len(df[(df["PRO_ORG_AD"] == code)&(df["WORKSITE_STATE"] == topstate[x])][df[(df["PRO_ORG_AD"] == code)&(df["WORKSITE_STATE"] == topstate[x])]["FOREIGN_WORKER_EDUCATION"]==edu[j]])
        apps.append(a)
        b = df[(df["PRO_ORG_AD"] == code)&(df["WORKSITE_STATE"] == topstate[x])][df[(df["PRO_ORG_AD"] == code)&(df["WORKSITE_STATE"] == topstate[x])]["FOREIGN_WORKER_EDUCATION"]==edu[j]]["WO_A"].median()
        medwage.append(b)
        c = df[(df["PRO_ORG_AD"] == code)&(df["WORKSITE_STATE"] == topstate[x])][df[(df["PRO_ORG_AD"] == code)&(df["WORKSITE_STATE"] == topstate[x])]["FOREIGN_WORKER_EDUCATION"]==edu[j]]["YRSPOSTGRAD"].median()
        yrsgrad.append(c)
        d = df[(df["PRO_ORG_AD"] == code)&(df["WORKSITE_STATE"] == topstate[x])][df[(df["PRO_ORG_AD"] == code)&(df["WORKSITE_STATE"] == topstate[x])]["FOREIGN_WORKER_EDUCATION"]==edu[j]]["DAYS_TO_DECIDE"].median()
        meddays.append(d)
codestatedf1 = pd.DataFrame([sum(statelist, []),edulist,apps,medwage,yrsgrad,meddays]).transpose()
codestatedf1.columns = ["STATE","EDU LEVEL", "TOTAL APPLICATIONS","MEDIAN WAGE","MEDIAN YRS POST GRAD","MEDIAN DAYS TO DECIDE"]
codestatedf1 = codestatedf1.set_index(["STATE","EDU LEVEL"])
codestatedf1

In [ ]:
# AP TO PW DETERMINE SPEED
# CREATING A NEW COLUMN: APP GROUP, SPLITS DAYS INTO THIRDS
df["APP_TO_PW_DETERMINE"] = df["APP_TO_PW_DETERMINE"].replace("#VALUE!",np.nan).astype(float)
high = df.groupby("YEAR")["APP_TO_PW_DETERMINE"].quantile(2/3).tolist()
low = df.groupby("YEAR")["APP_TO_PW_DETERMINE"].quantile(1/3).tolist()
df["APP_GROUP"] = np.nan
for x in range(len(yr)):
    df["APP_GROUP"] = np.where((df["YEAR"]==yr[x])&(df["APP_TO_PW_DETERMINE"]<low[x]),"FAST",df["APP_GROUP"])
    df["APP_GROUP"] = np.where((df["YEAR"]==yr[x])&(df["APP_TO_PW_DETERMINE"]>=low[x])&(df["APP_TO_PW_DETERMINE"]<high[x]),"MEDIUM",df["APP_GROUP"])
    df["APP_GROUP"] = np.where((df["YEAR"]==yr[x])&(df["APP_TO_PW_DETERMINE"]>=high[x]),"SLOW",df["APP_GROUP"])


In [ ]:
# CORRELATION ANALYSIS: AGGREGATE BY SIZE BRACKETS AND YEARLY
# PLOTTING SUCCESS RATES OVER TIME BY SIZE OF COMPANY

young = df[df["APP_GROUP"]=="FAST"]
mid = df[df["APP_GROUP"]=="MEDIUM"]
old = df[df["APP_GROUP"]=="SLOW"]


youndf = pd.DataFrame()
middf = pd.DataFrame()
olddf = pd.DataFrame()
linregdf = pd.DataFrame()

for x in range(len(yr)):
    j = pd.DataFrame([len(young[(young["YEAR"]==yr[x])&(young["CASE_STATUS"]==1)])/len(young[young["YEAR"]==yr[x]])])
    med = pd.DataFrame([len(mid[(mid["YEAR"]==yr[x])&(mid["CASE_STATUS"]==1)])/len(mid[mid["YEAR"]==yr[x]])])
    oldy = pd.DataFrame([len(old[(old["YEAR"]==yr[x])&(old["CASE_STATUS"]==1)])/len(old[old["YEAR"]==yr[x]])])
    youndf = pd.concat([youndf,j])
    middf = pd.concat([middf,med])
    olddf = pd.concat([olddf,oldy])
    linregdf = pd.concat([youndf,middf,olddf], axis = 1)
    

linregdf.index = yr
linregdf.columns = ["Fast Success %", "Medium Success %", "Slow Success %"]
plt.figure(figsize = (12,10), dpi = 100)
linregdf.plot(kind = "bar")
plt.tight_layout()
plt.legend(bbox_to_anchor=(1.05, 1), loc=2, borderaxespad=0.)
plt.title("Success Rates by Time To Apply Since PW Determine, Over Time")


In [ ]:
# PREPARING DATA FOR LIN REGRESS AND GETTING P VALUE
youngapps = []
midapps = []
oldapps = []

for x in range(len(yr)):
    y = (young["YEAR"]==yr[x]).sum()
    m = (mid["YEAR"]==yr[x]).sum()
    o = (old["YEAR"]==yr[x]).sum()
    youngapps.append(y)
    midapps.append(m)
    oldapps.append(o)
linregdf = linregdf.reset_index()
linregdf["Fast Apps"] = pd.DataFrame(youngapps)
linregdf["Medium Apps"] = pd.DataFrame(midapps)
linregdf["Slow Apps"] = pd.DataFrame(oldapps)
linregdf = linregdf.set_index("index")
# PLOTTING APPLICATION RATES OVER TIME BY SIZE OF COMPANY
plt.figure(figsize = (12,10), dpi = 100)
linregdf.reset_index().plot(x = "index", y = ["Fast Apps","Medium Apps","Slow Apps"], kind = "bar")
plt.tight_layout()
plt.legend(bbox_to_anchor=(1.05, 1), loc=2, borderaxespad=0.)
plt.title("Application Rates by Time To Apply Since PW Determine, Over Time")


In [ ]:
# P VALUE PER YEAR
young_med = []
for x in range(len(yr)):
    sample1_phat = linregdf.loc[yr[x]]["Fast Success %"]
    sample2_phat = linregdf.loc[yr[x]]["Medium Success %"]
    sample1_size = linregdf.loc[yr[x]]["Fast Apps"]
    sample2_size = linregdf.loc[yr[x]]["Medium Apps"]
    phat = (sample1_phat*sample1_size+sample2_phat*sample2_size)/(sample1_size+sample2_size)
    zscore = (sample1_phat - sample2_phat)/np.sqrt(phat*(1-phat)*((1/sample1_size)+(1/sample2_size)))    
    pval = st.norm.sf(abs(zscore))*2
    young_med.append(pval)

pvaldf = pd.DataFrame(young_med)

young_old = []
for x in range(len(yr)):
    sample1_phat = linregdf.loc[yr[x]]["Fast Success %"]
    sample2_phat = linregdf.loc[yr[x]]["Slow Success %"]
    sample1_size = linregdf.loc[yr[x]]["Fast Apps"]
    sample2_size = linregdf.loc[yr[x]]["Slow Apps"]
    phat = (sample1_phat*sample1_size+sample2_phat*sample2_size)/(sample1_size+sample2_size)
    zscore = (sample1_phat - sample2_phat)/np.sqrt(phat*(1-phat)*((1/sample1_size)+(1/sample2_size)))    
    pval = st.norm.sf(abs(zscore))*2
    young_old.append(pval)
pvaldf = pd.concat([pvaldf,pd.DataFrame(young_old)], axis = 1)

med_old = []
for x in range(len(yr)):
    sample1_phat = linregdf.loc[yr[x]]["Medium Success %"]
    sample2_phat = linregdf.loc[yr[x]]["Slow Success %"]
    sample1_size = linregdf.loc[yr[x]]["Medium Apps"]
    sample2_size = linregdf.loc[yr[x]]["Slow Apps"]
    phat = (sample1_phat*sample1_size+sample2_phat*sample2_size)/(sample1_size+sample2_size)
    zscore = (sample1_phat - sample2_phat)/np.sqrt(phat*(1-phat)*((1/sample1_size)+(1/sample2_size)))    
    pval = st.norm.sf(abs(zscore))*2
    med_old.append(pval)
pvaldf = pd.concat([pvaldf,pd.DataFrame(med_old)], axis = 1)
pvaldf.columns = ["Fast_Med","Fast_Slow","Med_Slow"]
pvaldf.index = yr
linregdf[["Fast_Med","Fast_Slow","Med_Slow"]] = pvaldf[["Fast_Med","Fast_Slow","Med_Slow"]]


In [ ]:
# LINEAR REGRESSION BETWEEN Speed to Apply and success
print("Fast success rate over time:",st.linregress(y = linregdf["Fast Success %"], x = yr))
print("\nFast applications over time:",st.linregress(y = linregdf["Fast Apps"], x = yr))

print("\n\nMedium success rate over time:",st.linregress(y = linregdf["Medium Success %"], x = yr))
print("\nMedium applications over time:",st.linregress(y = linregdf["Medium Apps"], x = yr))

print("\n\nSlow success rate over time:",st.linregress(y = linregdf["Slow Success %"], x = yr))
print("\nSlow applications over time:",st.linregress(y = linregdf["Slow Apps"], x = yr))


In [ ]:
# GETTING THE TOTAL PVAL
linregdf.loc["ALL YRS"] = linregdf.sum()
youngsuc = []
medsuc = []
oldsuc = []
for x in range(len(yr)):
    y = linregdf.loc[yr[x]].iloc[0]*linregdf.loc[yr[x]].iloc[3]
    m = linregdf.loc[yr[x]].iloc[1]*linregdf.loc[yr[x]].iloc[4]
    o = linregdf.loc[yr[x]].iloc[2]*linregdf.loc[yr[x]].iloc[5]
    youngsuc.append(y)
    medsuc.append(m)
    oldsuc.append(o)
linregdf.loc["ALL YRS"].iloc[0] = sum(youngsuc)/linregdf.loc["ALL YRS"].iloc[3]
linregdf.loc["ALL YRS"].iloc[1] = sum(medsuc)/linregdf.loc["ALL YRS"].iloc[4]
linregdf.loc["ALL YRS"].iloc[2] = sum(oldsuc)/linregdf.loc["ALL YRS"].iloc[5]


for x in range(len(yr)):
    sample1_phat = linregdf.loc["ALL YRS"].iloc[0]
    sample2_phat = linregdf.loc["ALL YRS"].iloc[1]
    sample1_size = linregdf.loc["ALL YRS"].iloc[3]
    sample2_size = linregdf.loc["ALL YRS"].iloc[4]
    phat = (sample1_phat*sample1_size+sample2_phat*sample2_size)/(sample1_size+sample2_size)
    zscore = (sample1_phat - sample2_phat)/np.sqrt(phat*(1-phat)*((1/sample1_size)+(1/sample2_size)))    
    pval = st.norm.sf(abs(zscore))*2
linregdf.loc["ALL YRS"].iloc[6] = pval

for x in range(len(yr)):
    sample1_phat = linregdf.loc["ALL YRS"].iloc[0]
    sample2_phat = linregdf.loc["ALL YRS"].iloc[2]
    sample1_size = linregdf.loc["ALL YRS"].iloc[3]
    sample2_size = linregdf.loc["ALL YRS"].iloc[5]
    phat = (sample1_phat*sample1_size+sample2_phat*sample2_size)/(sample1_size+sample2_size)
    zscore = (sample1_phat - sample2_phat)/np.sqrt(phat*(1-phat)*((1/sample1_size)+(1/sample2_size)))    
    pval = st.norm.sf(abs(zscore))*2
linregdf.loc["ALL YRS"].iloc[7] = pval

for x in range(len(yr)):
    sample1_phat = linregdf.loc["ALL YRS"].iloc[1]
    sample2_phat = linregdf.loc["ALL YRS"].iloc[2]
    sample1_size = linregdf.loc["ALL YRS"].iloc[4]
    sample2_size = linregdf.loc["ALL YRS"].iloc[5]
    phat = (sample1_phat*sample1_size+sample2_phat*sample2_size)/(sample1_size+sample2_size)
    zscore = (sample1_phat - sample2_phat)/np.sqrt(phat*(1-phat)*((1/sample1_size)+(1/sample2_size)))    
    pval = st.norm.sf(abs(zscore))*2
linregdf.loc["ALL YRS"].iloc[8] = pval
linregdf = linregdf.head(9).style.format("{:,.2f}")
linregdf


In [ ]:
# ADVERTISEMENT THROUGH PVT EMPLOYMENT
# PLOTTING THE DIFFERENT SUCCESS RATES OVER TIME BY PVT FIRM or not (IN TERMS OF ANNUAL APPLICATIONS)

top15 = df[df["PVT_EMPLOYMENT_FIRM"]==1]
not15 = df[df["PVT_EMPLOYMENT_FIRM"]==0]

topdf = pd.DataFrame()
notdf = pd.DataFrame()
analysisdf = pd.DataFrame()

for x in range(len(yr)):
    j = pd.DataFrame([len(top15[(top15["YEAR"]==yr[x])&(top15["CASE_STATUS"]==1)])/len(top15[top15["YEAR"]==yr[x]])])
    b = pd.DataFrame([len(not15[(not15["YEAR"]==yr[x])&(not15["CASE_STATUS"]==1)])/len(not15[not15["YEAR"]==yr[x]])])
    topdf = pd.concat([topdf,j])
    notdf = pd.concat([notdf,b])
    analysisdf = pd.concat([topdf,notdf], axis = 1)

analysisdf.index = yr
analysisdf.columns = ["PVT Success %", "Not PVT Success %"]
plt.figure(figsize = (12,10), dpi = 100)
analysisdf.plot(kind = "bar")
plt.tight_layout()
plt.legend(bbox_to_anchor=(1.05, 1), loc=2, borderaxespad=0.)
plt.title("Success Rates by advertisement with Private Employment Firm or not, Over Time")

In [ ]:
# PREPARING DATA FOR LIN REGRESS AND GETTING P VALUE
top15apps = []
not15apps = []

for x in range(len(yr)):
    t = (top15["YEAR"]==yr[x]).sum()
    n = (not15["YEAR"]==yr[x]).sum()
    top15apps.append(t)
    not15apps.append(n)
analysisdf = analysisdf.reset_index()
analysisdf["PVT Apps"] = pd.DataFrame(top15apps)
analysisdf["Not PVT Apps"] = pd.DataFrame(not15apps)
analysisdf = analysisdf.set_index("index")
# PLOTTING APPLICATION RATES OVER TIME BY TOP 15 or NOT TOP 15
plt.figure(figsize = (12,10), dpi = 100)
analysisdf.reset_index().plot(x = "index", y = ["PVT Apps","Not PVT Apps"], kind = "bar")
plt.tight_layout()
plt.legend(bbox_to_anchor=(1.05, 1), loc=2, borderaxespad=0.)
plt.title("Application Rates by advertisement with Private Employment Firm or not, Over Time")

In [ ]:
# P VALUE PER YEAR: ON THE SUCCESS RATE
top_not = []
for x in range(len(yr)):
    sample1_phat = analysisdf.loc[yr[x]]["PVT Success %"]
    sample2_phat = analysisdf.loc[yr[x]]["Not PVT Success %"]
    sample1_size = analysisdf.loc[yr[x]]["PVT Apps"]
    sample2_size = analysisdf.loc[yr[x]]["Not PVT Apps"]
    phat = (sample1_phat*sample1_size+sample2_phat*sample2_size)/(sample1_size+sample2_size)
    zscore = (sample1_phat - sample2_phat)/np.sqrt(phat*(1-phat)*((1/sample1_size)+(1/sample2_size)))    
    pval = st.norm.sf(abs(zscore))*2
    top_not.append(pval)

pvaldf = pd.DataFrame(top_not)
pvaldf.columns = ["PVT Vs. Not PVT"]
pvaldf.index = yr
analysisdf["PVAL Success %"] = pvaldf["PVT Vs. Not PVT"]

In [ ]:
# LINEAR REGRESSION BETWEEN private employment firm AND SUCCESS RATE OVER TIME
print("PVT success rate over time:",st.linregress(y = analysisdf["PVT Success %"], x = yr))
print("\nPVT applications over time:",st.linregress(y = analysisdf["PVT Apps"], x = yr))

print("\n\nNot PVT success rate over time:",st.linregress(y = analysisdf["Not PVT Success %"], x = yr))
print("\nNot PVT applications over time:",st.linregress(y = analysisdf["Not PVT Apps"], x = yr))

In [ ]:
# GETTING THE TOTAL PVAL
analysisdf.loc["ALL YRS"] = analysisdf.sum()
top15suc = []
not15suc = []

for x in range(len(yr)):
    t15 = analysisdf.loc[yr[x]].iloc[0]*analysisdf.loc[yr[x]].iloc[2]
    n15 = analysisdf.loc[yr[x]].iloc[1]*analysisdf.loc[yr[x]].iloc[3]
    top15suc.append(t15)
    not15suc.append(n15)
analysisdf.loc["ALL YRS"].iloc[0] = sum(top15suc)/analysisdf.loc["ALL YRS"].iloc[2]
analysisdf.loc["ALL YRS"].iloc[1] = sum(not15suc)/analysisdf.loc["ALL YRS"].iloc[3]


for x in range(len(yr)):
    sample1_phat = analysisdf.loc["ALL YRS"].iloc[0]
    sample2_phat = analysisdf.loc["ALL YRS"].iloc[1]
    sample1_size = analysisdf.loc["ALL YRS"].iloc[2]
    sample2_size = analysisdf.loc["ALL YRS"].iloc[3]
    phat = (sample1_phat*sample1_size+sample2_phat*sample2_size)/(sample1_size+sample2_size)
    zscore = (sample1_phat - sample2_phat)/np.sqrt(phat*(1-phat)*((1/sample1_size)+(1/sample2_size)))    
    pval = st.norm.sf(abs(zscore))*2
analysisdf.loc["ALL YRS"].iloc[4] = pval

analysisdf = analysisdf.head(9).style.format("{:,.2f}")
analysisdf

In [ ]:
### EMPLOYEE RELATED ANALYSIS ###

In [ ]:
#CLASS OF ADMISSION
df["CLASS_OF_ADMISSION"].value_counts()[:15].iplot(kind = "bar")

In [ ]:
# H1B really different??
# PLOTTING THE DIFFERENT SUCCESS RATES OVER TIME BY class (IN TERMS OF ANNUAL APPLICATIONS)

top15 = df[df["CLASS_OF_ADMISSION"]=="H-1B"]
not15 = df[df["CLASS_OF_ADMISSION"]!="H-1B"]

topdf = pd.DataFrame()
notdf = pd.DataFrame()
analysisdf = pd.DataFrame()

for x in range(len(yr)):
    j = pd.DataFrame([len(top15[(top15["YEAR"]==yr[x])&(top15["CASE_STATUS"]==1)])/len(top15[top15["YEAR"]==yr[x]])])
    b = pd.DataFrame([len(not15[(not15["YEAR"]==yr[x])&(not15["CASE_STATUS"]==1)])/len(not15[not15["YEAR"]==yr[x]])])
    topdf = pd.concat([topdf,j])
    notdf = pd.concat([notdf,b])
    analysisdf = pd.concat([topdf,notdf], axis = 1)

analysisdf.index = yr
analysisdf.columns = ["H1B Success %", "Not H1B Success %"]
plt.figure(figsize = (12,10), dpi = 100)
analysisdf.plot(kind = "bar")
plt.tight_layout()
plt.legend(bbox_to_anchor=(1.05, 1), loc=2, borderaxespad=0.)
plt.title("Success Rates by H1B or not, Over Time")

In [ ]:
# PREPARING DATA FOR LIN REGRESS AND GETTING P VALUE
top15apps = []
not15apps = []

for x in range(len(yr)):
    t = (top15["YEAR"]==yr[x]).sum()
    n = (not15["YEAR"]==yr[x]).sum()
    top15apps.append(t)
    not15apps.append(n)
analysisdf = analysisdf.reset_index()
analysisdf["H1B Apps"] = pd.DataFrame(top15apps)
analysisdf["Not H1B Apps"] = pd.DataFrame(not15apps)
analysisdf = analysisdf.set_index("index")
# PLOTTING APPLICATION RATES OVER TIME BY TOP 15 or NOT TOP 15
plt.figure(figsize = (12,10), dpi = 100)
analysisdf.reset_index().plot(x = "index", y = ["H1B Apps","Not H1B Apps"], kind = "bar")
plt.tight_layout()
plt.legend(bbox_to_anchor=(1.05, 1), loc=2, borderaxespad=0.)
plt.title("Application Rates by H1B or not, Over Time")

In [ ]:
# P VALUE PER YEAR: ON THE SUCCESS RATE
top_not = []
for x in range(len(yr)):
    sample1_phat = analysisdf.loc[yr[x]]["H1B Success %"]
    sample2_phat = analysisdf.loc[yr[x]]["Not H1B Success %"]
    sample1_size = analysisdf.loc[yr[x]]["H1B Apps"]
    sample2_size = analysisdf.loc[yr[x]]["Not H1B Apps"]
    phat = (sample1_phat*sample1_size+sample2_phat*sample2_size)/(sample1_size+sample2_size)
    zscore = (sample1_phat - sample2_phat)/np.sqrt(phat*(1-phat)*((1/sample1_size)+(1/sample2_size)))    
    pval = st.norm.sf(abs(zscore))*2
    top_not.append(pval)

pvaldf = pd.DataFrame(top_not)
pvaldf.columns = ["H1B Vs. Not H1B"]
pvaldf.index = yr
analysisdf["PVAL Success %"] = pvaldf["H1B Vs. Not H1B"]

In [ ]:
# LINEAR REGRESSION BETWEEN Professional Occupation AND SUCCESS RATE OVER TIME
print("H1B success rate over time:",st.linregress(y = analysisdf["H1B Success %"], x = yr))
print("\nH1B applications over time:",st.linregress(y = analysisdf["H1B Apps"], x = yr))

print("\n\nNot H1B success rate over time:",st.linregress(y = analysisdf["Not H1B Success %"], x = yr))
print("\nNot H1B applications over time:",st.linregress(y = analysisdf["Not H1B Apps"], x = yr))

In [ ]:
# GETTING THE TOTAL PVAL
analysisdf.loc["ALL YRS"] = analysisdf.sum()
top15suc = []
not15suc = []

for x in range(len(yr)):
    t15 = analysisdf.loc[yr[x]].iloc[0]*analysisdf.loc[yr[x]].iloc[2]
    n15 = analysisdf.loc[yr[x]].iloc[1]*analysisdf.loc[yr[x]].iloc[3]
    top15suc.append(t15)
    not15suc.append(n15)
analysisdf.loc["ALL YRS"].iloc[0] = sum(top15suc)/analysisdf.loc["ALL YRS"].iloc[2]
analysisdf.loc["ALL YRS"].iloc[1] = sum(not15suc)/analysisdf.loc["ALL YRS"].iloc[3]


for x in range(len(yr)):
    sample1_phat = analysisdf.loc["ALL YRS"].iloc[0]
    sample2_phat = analysisdf.loc["ALL YRS"].iloc[1]
    sample1_size = analysisdf.loc["ALL YRS"].iloc[2]
    sample2_size = analysisdf.loc["ALL YRS"].iloc[3]
    phat = (sample1_phat*sample1_size+sample2_phat*sample2_size)/(sample1_size+sample2_size)
    zscore = (sample1_phat - sample2_phat)/np.sqrt(phat*(1-phat)*((1/sample1_size)+(1/sample2_size)))    
    pval = st.norm.sf(abs(zscore))*2
analysisdf.loc["ALL YRS"].iloc[4] = pval

analysisdf = analysisdf.head(9).style.format("{:,.2f}")
analysisdf

In [ ]:
# DIVING DEEPER INTO H1B
#ANCHORED BY STATE: PW SKILL,APPLICATIONS, MEDIAN WAGE, YRS POST GRADE, MEDIAN DAYS

code = "H-1B"
# CHANGE THIS FOR ANY CODE

# MEDIAN WAGE, MEDIAN YEARS AFTER GRADUATING, MEDIAN DAYS TO DECIDE : BY PW LEVEL
topstate = df[df["CLASS_OF_ADMISSION"] == code]["WORKSITE_STATE"].value_counts()[:15].index.tolist()
pwskill = df["PW_SKILL_LEVEL"].dropna().unique().tolist()
pwskill.sort()


statelist = []
pwlist = pwskill*len(topstate)
apps = []
medwage = []
yrsgrad = []
meddays = []


for x in range(len(topstate)):
    s = (topstate[x]+",")*len(pwskill)
    s = "".join(s).split(",")
    s = list(filter(("").__ne__, s))
    statelist.append(s) 
for x in range(len(topstate)):
    for j in range(len(pwskill)):
        a = len(df[(df["CLASS_OF_ADMISSION"] == code)&(df["WORKSITE_STATE"] == topstate[x])][df[(df["CLASS_OF_ADMISSION"] == code)&(df["WORKSITE_STATE"] == topstate[x])]["PW_SKILL_LEVEL"]==pwskill[j]])
        apps.append(a)
        b = df[(df["CLASS_OF_ADMISSION"] == code)&(df["WORKSITE_STATE"] == topstate[x])][df[(df["CLASS_OF_ADMISSION"] == code)&(df["WORKSITE_STATE"] == topstate[x])]["PW_SKILL_LEVEL"]==pwskill[j]]["WO_A"].median()
        medwage.append(b)
        c = df[(df["CLASS_OF_ADMISSION"] == code)&(df["WORKSITE_STATE"] == topstate[x])][df[(df["CLASS_OF_ADMISSION"] == code)&(df["WORKSITE_STATE"] == topstate[x])]["PW_SKILL_LEVEL"]==pwskill[j]]["YRSPOSTGRAD"].median()
        yrsgrad.append(c)
        d = df[(df["CLASS_OF_ADMISSION"] == code)&(df["WORKSITE_STATE"] == topstate[x])][df[(df["CLASS_OF_ADMISSION"] == code)&(df["WORKSITE_STATE"] == topstate[x])]["PW_SKILL_LEVEL"]==pwskill[j]]["DAYS_TO_DECIDE"].median()
        meddays.append(d)
codestatedf = pd.DataFrame([sum(statelist, []),pwlist,apps,medwage,yrsgrad,meddays]).transpose()
codestatedf.columns = ["STATE","PW LEVEL", "TOTAL APPLICATIONS","MEDIAN WAGE","MEDIAN YRS POST GRAD","MEDIAN DAYS TO DECIDE"]
codestatedf = codestatedf.set_index(["STATE","PW LEVEL"])
codestatedf

In [ ]:
# ANCHORED BY STATE: EDU LEVEL,APPLICATIONS, MEDIAN WAGE, YRS POST GRADE, MEDIAN DAYS
# MEDIAN WAGE, MEDIAN YEARS AFTER GRADUATING, MEDIAN DAYS TO DECIDE : BY EDU LEVEL
edu = df["FOREIGN_WORKER_EDUCATION"].dropna().unique().tolist()
edu[0],edu[1],edu[2],edu[3],edu[4],edu[5] = edu[3],edu[5],edu[0],edu[2],edu[1],edu[4]

statelist = []
edulist = edu*len(topstate)
apps = []
medwage = []
yrsgrad = []
meddays = []


for x in range(len(topstate)):
    s = (topstate[x]+",")*len(edu)
    s = "".join(s).split(",")
    s = list(filter(("").__ne__, s))
    statelist.append(s) 
for x in range(len(topstate)):
    for j in range(len(edu)):
        a = len(df[(df["CLASS_OF_ADMISSION"] == code)&(df["WORKSITE_STATE"] == topstate[x])][df[(df["CLASS_OF_ADMISSION"] == code)&(df["WORKSITE_STATE"] == topstate[x])]["FOREIGN_WORKER_EDUCATION"]==edu[j]])
        apps.append(a)
        b = df[(df["CLASS_OF_ADMISSION"] == code)&(df["WORKSITE_STATE"] == topstate[x])][df[(df["CLASS_OF_ADMISSION"] == code)&(df["WORKSITE_STATE"] == topstate[x])]["FOREIGN_WORKER_EDUCATION"]==edu[j]]["WO_A"].median()
        medwage.append(b)
        c = df[(df["CLASS_OF_ADMISSION"] == code)&(df["WORKSITE_STATE"] == topstate[x])][df[(df["CLASS_OF_ADMISSION"] == code)&(df["WORKSITE_STATE"] == topstate[x])]["FOREIGN_WORKER_EDUCATION"]==edu[j]]["YRSPOSTGRAD"].median()
        yrsgrad.append(c)
        d = df[(df["CLASS_OF_ADMISSION"] == code)&(df["WORKSITE_STATE"] == topstate[x])][df[(df["CLASS_OF_ADMISSION"] == code)&(df["WORKSITE_STATE"] == topstate[x])]["FOREIGN_WORKER_EDUCATION"]==edu[j]]["DAYS_TO_DECIDE"].median()
        meddays.append(d)
codestatedf1 = pd.DataFrame([sum(statelist, []),edulist,apps,medwage,yrsgrad,meddays]).transpose()
codestatedf1.columns = ["STATE","EDU LEVEL", "TOTAL APPLICATIONS","MEDIAN WAGE","MEDIAN YRS POST GRAD","MEDIAN DAYS TO DECIDE"]
codestatedf1 = codestatedf1.set_index(["STATE","EDU LEVEL"])
codestatedf1


In [ ]:
# WHAT ABOUT L1 OR F1 Vs THE REST - except for H1B...
# ADVERTISEMENT THROUGH PROFESSIONAL ADVERTISEMENT?
# PLOTTING THE DIFFERENT SUCCESS RATES OVER TIME BY employee ref or not (IN TERMS OF ANNUAL APPLICATIONS)

top15 = df[(df["CLASS_OF_ADMISSION"]=="L-1")|(df["CLASS_OF_ADMISSION"]=="F-1")]
not15 = df[(df["CLASS_OF_ADMISSION"]!="L-1")&(df["CLASS_OF_ADMISSION"]!="F-1")&(df["CLASS_OF_ADMISSION"]!="H-1B")]

topdf = pd.DataFrame()
notdf = pd.DataFrame()
analysisdf = pd.DataFrame()

for x in range(len(yr)):
    j = pd.DataFrame([len(top15[(top15["YEAR"]==yr[x])&(top15["CASE_STATUS"]==1)])/len(top15[top15["YEAR"]==yr[x]])])
    b = pd.DataFrame([len(not15[(not15["YEAR"]==yr[x])&(not15["CASE_STATUS"]==1)])/len(not15[not15["YEAR"]==yr[x]])])
    topdf = pd.concat([topdf,j])
    notdf = pd.concat([notdf,b])
    analysisdf = pd.concat([topdf,notdf], axis = 1)

analysisdf.index = yr
analysisdf.columns = ["L1F1 Success %", "Not L1F1 Success %"]
plt.figure(figsize = (12,10), dpi = 100)
analysisdf.plot(kind = "bar")
plt.tight_layout()
plt.legend(bbox_to_anchor=(1.05, 1), loc=2, borderaxespad=0.)
plt.title("Success Rates by L1/F1 or not, Over Time")

In [ ]:
# PREPARING DATA FOR LIN REGRESS AND GETTING P VALUE
top15apps = []
not15apps = []

for x in range(len(yr)):
    t = (top15["YEAR"]==yr[x]).sum()
    n = (not15["YEAR"]==yr[x]).sum()
    top15apps.append(t)
    not15apps.append(n)
analysisdf = analysisdf.reset_index()
analysisdf["L1F1 Apps"] = pd.DataFrame(top15apps)
analysisdf["Not L1F1 Apps"] = pd.DataFrame(not15apps)
analysisdf = analysisdf.set_index("index")
# PLOTTING APPLICATION RATES OVER TIME BY TOP 15 or NOT TOP 15
plt.figure(figsize = (12,10), dpi = 100)
analysisdf.reset_index().plot(x = "index", y = ["L1F1 Apps","Not L1F1 Apps"], kind = "bar")
plt.tight_layout()
plt.legend(bbox_to_anchor=(1.05, 1), loc=2, borderaxespad=0.)
plt.title("Application Rates by L1/F1 or not, Over Time")

In [ ]:
# P VALUE PER YEAR: ON THE SUCCESS RATE
top_not = []
for x in range(len(yr)):
    sample1_phat = analysisdf.loc[yr[x]]["L1F1 Success %"]
    sample2_phat = analysisdf.loc[yr[x]]["Not L1F1 Success %"]
    sample1_size = analysisdf.loc[yr[x]]["L1F1 Apps"]
    sample2_size = analysisdf.loc[yr[x]]["Not L1F1 Apps"]
    phat = (sample1_phat*sample1_size+sample2_phat*sample2_size)/(sample1_size+sample2_size)
    zscore = (sample1_phat - sample2_phat)/np.sqrt(phat*(1-phat)*((1/sample1_size)+(1/sample2_size)))    
    pval = st.norm.sf(abs(zscore))*2
    top_not.append(pval)

pvaldf = pd.DataFrame(top_not)
pvaldf.columns = ["L1F1 Vs. Not L1F1"]
pvaldf.index = yr
analysisdf["PVAL Success %"] = pvaldf["L1F1 Vs. Not L1F1"]

In [ ]:
# LINEAR REGRESSION BETWEEN L1/F1 Occupation AND SUCCESS RATE OVER TIME
print("L1F1 success rate over time:",st.linregress(y = analysisdf["L1F1 Success %"], x = yr))
print("\nL1F1 applications over time:",st.linregress(y = analysisdf["L1F1 Apps"], x = yr))

print("\n\nNot L1F1 success rate over time:",st.linregress(y = analysisdf["Not L1F1 Success %"], x = yr))
print("\nNot L1F1 applications over time:",st.linregress(y = analysisdf["Not L1F1 Apps"], x = yr))

In [ ]:
# GETTING THE TOTAL PVAL
analysisdf.loc["ALL YRS"] = analysisdf.sum()
top15suc = []
not15suc = []

for x in range(len(yr)):
    t15 = analysisdf.loc[yr[x]].iloc[0]*analysisdf.loc[yr[x]].iloc[2]
    n15 = analysisdf.loc[yr[x]].iloc[1]*analysisdf.loc[yr[x]].iloc[3]
    top15suc.append(t15)
    not15suc.append(n15)
analysisdf.loc["ALL YRS"].iloc[0] = sum(top15suc)/analysisdf.loc["ALL YRS"].iloc[2]
analysisdf.loc["ALL YRS"].iloc[1] = sum(not15suc)/analysisdf.loc["ALL YRS"].iloc[3]


for x in range(len(yr)):
    sample1_phat = analysisdf.loc["ALL YRS"].iloc[0]
    sample2_phat = analysisdf.loc["ALL YRS"].iloc[1]
    sample1_size = analysisdf.loc["ALL YRS"].iloc[2]
    sample2_size = analysisdf.loc["ALL YRS"].iloc[3]
    phat = (sample1_phat*sample1_size+sample2_phat*sample2_size)/(sample1_size+sample2_size)
    zscore = (sample1_phat - sample2_phat)/np.sqrt(phat*(1-phat)*((1/sample1_size)+(1/sample2_size)))    
    pval = st.norm.sf(abs(zscore))*2
analysisdf.loc["ALL YRS"].iloc[4] = pval

analysisdf = analysisdf.head(9).style.format("{:,.2f}")
analysisdf

In [ ]:
# CREATING A COLUMN JUST FOR L1F1
df["L1F1"] = np.where((df["CLASS_OF_ADMISSION"]=="L-1")|(df["CLASS_OF_ADMISSION"]=="F-1"),1,0)

In [ ]:
# DIVING DEEPER INTO THE L1/F1
#ANCHORED BY STATE: PW SKILL,APPLICATIONS, MEDIAN WAGE, YRS POST GRADE, MEDIAN DAYS

code = 1
# CHANGE THIS FOR ANY CODE

# MEDIAN WAGE, MEDIAN YEARS AFTER GRADUATING, MEDIAN DAYS TO DECIDE : BY PW LEVEL
topstate = df[df["L1F1"] == code]["WORKSITE_STATE"].value_counts()[:15].index.tolist()
pwskill = df["PW_SKILL_LEVEL"].dropna().unique().tolist()
pwskill.sort()


statelist = []
pwlist = pwskill*len(topstate)
apps = []
medwage = []
yrsgrad = []
meddays = []


for x in range(len(topstate)):
    s = (topstate[x]+",")*len(pwskill)
    s = "".join(s).split(",")
    s = list(filter(("").__ne__, s))
    statelist.append(s) 
for x in range(len(topstate)):
    for j in range(len(pwskill)):
        a = len(df[(df["L1F1"] == code)&(df["WORKSITE_STATE"] == topstate[x])][df[(df["L1F1"] == code)&(df["WORKSITE_STATE"] == topstate[x])]["PW_SKILL_LEVEL"]==pwskill[j]])
        apps.append(a)
        b = df[(df["L1F1"] == code)&(df["WORKSITE_STATE"] == topstate[x])][df[(df["L1F1"] == code)&(df["WORKSITE_STATE"] == topstate[x])]["PW_SKILL_LEVEL"]==pwskill[j]]["WO_A"].median()
        medwage.append(b)
        c = df[(df["L1F1"] == code)&(df["WORKSITE_STATE"] == topstate[x])][df[(df["L1F1"] == code)&(df["WORKSITE_STATE"] == topstate[x])]["PW_SKILL_LEVEL"]==pwskill[j]]["YRSPOSTGRAD"].median()
        yrsgrad.append(c)
        d = df[(df["L1F1"] == code)&(df["WORKSITE_STATE"] == topstate[x])][df[(df["L1F1"] == code)&(df["WORKSITE_STATE"] == topstate[x])]["PW_SKILL_LEVEL"]==pwskill[j]]["DAYS_TO_DECIDE"].median()
        meddays.append(d)
codestatedf = pd.DataFrame([sum(statelist, []),pwlist,apps,medwage,yrsgrad,meddays]).transpose()
codestatedf.columns = ["STATE","PW LEVEL", "TOTAL APPLICATIONS","MEDIAN WAGE","MEDIAN YRS POST GRAD","MEDIAN DAYS TO DECIDE"]
codestatedf = codestatedf.set_index(["STATE","PW LEVEL"])
codestatedf

In [ ]:
# ANCHORED BY STATE: EDU LEVEL,APPLICATIONS, MEDIAN WAGE, YRS POST GRADE, MEDIAN DAYS
# MEDIAN WAGE, MEDIAN YEARS AFTER GRADUATING, MEDIAN DAYS TO DECIDE : BY EDU LEVEL
edu = df["FOREIGN_WORKER_EDUCATION"].dropna().unique().tolist()
edu[0],edu[1],edu[2],edu[3],edu[4],edu[5] = edu[3],edu[5],edu[0],edu[2],edu[1],edu[4]

statelist = []
edulist = edu*len(topstate)
apps = []
medwage = []
yrsgrad = []
meddays = []


for x in range(len(topstate)):
    s = (topstate[x]+",")*len(edu)
    s = "".join(s).split(",")
    s = list(filter(("").__ne__, s))
    statelist.append(s) 
for x in range(len(topstate)):
    for j in range(len(edu)):
        a = len(df[(df["L1F1"] == code)&(df["WORKSITE_STATE"] == topstate[x])][df[(df["L1F1"] == code)&(df["WORKSITE_STATE"] == topstate[x])]["FOREIGN_WORKER_EDUCATION"]==edu[j]])
        apps.append(a)
        b = df[(df["L1F1"] == code)&(df["WORKSITE_STATE"] == topstate[x])][df[(df["L1F1"] == code)&(df["WORKSITE_STATE"] == topstate[x])]["FOREIGN_WORKER_EDUCATION"]==edu[j]]["WO_A"].median()
        medwage.append(b)
        c = df[(df["L1F1"] == code)&(df["WORKSITE_STATE"] == topstate[x])][df[(df["L1F1"] == code)&(df["WORKSITE_STATE"] == topstate[x])]["FOREIGN_WORKER_EDUCATION"]==edu[j]]["YRSPOSTGRAD"].median()
        yrsgrad.append(c)
        d = df[(df["L1F1"] == code)&(df["WORKSITE_STATE"] == topstate[x])][df[(df["L1F1"] == code)&(df["WORKSITE_STATE"] == topstate[x])]["FOREIGN_WORKER_EDUCATION"]==edu[j]]["DAYS_TO_DECIDE"].median()
        meddays.append(d)
codestatedf1 = pd.DataFrame([sum(statelist, []),edulist,apps,medwage,yrsgrad,meddays]).transpose()
codestatedf1.columns = ["STATE","EDU LEVEL", "TOTAL APPLICATIONS","MEDIAN WAGE","MEDIAN YRS POST GRAD","MEDIAN DAYS TO DECIDE"]
codestatedf1 = codestatedf1.set_index(["STATE","EDU LEVEL"])
codestatedf1

In [ ]:
# FOREIGN WORKER TRAINING COMPLETED? THERE WILL BE A LOT 0 DUE TO N/A or missing 
# PLOTTING THE DIFFERENT SUCCESS RATES OVER TIME BY TRAINING COMPLETE or not/not applicable (IN TERMS OF ANNUAL APPLICATIONS)

top15 = df[df["FOREIGN_WORKER_TRAINING_COMP"]==1]
not15 = df[df["FOREIGN_WORKER_TRAINING_COMP"]==0]

topdf = pd.DataFrame()
notdf = pd.DataFrame()
analysisdf = pd.DataFrame()

for x in range(len(yr)):
    j = pd.DataFrame([len(top15[(top15["YEAR"]==yr[x])&(top15["CASE_STATUS"]==1)])/len(top15[top15["YEAR"]==yr[x]])])
    b = pd.DataFrame([len(not15[(not15["YEAR"]==yr[x])&(not15["CASE_STATUS"]==1)])/len(not15[not15["YEAR"]==yr[x]])])
    topdf = pd.concat([topdf,j])
    notdf = pd.concat([notdf,b])
    analysisdf = pd.concat([topdf,notdf], axis = 1)

analysisdf.index = yr
analysisdf.columns = ["Training Success %", "Not Training Success %"]
plt.figure(figsize = (12,10), dpi = 100)
analysisdf.plot(kind = "bar")
plt.tight_layout()
plt.legend(bbox_to_anchor=(1.05, 1), loc=2, borderaxespad=0.)
plt.title("Success Rates by Required Training Completed or not, Over Time")

In [ ]:
# PREPARING DATA FOR LIN REGRESS AND GETTING P VALUE
top15apps = []
not15apps = []

for x in range(len(yr)):
    t = (top15["YEAR"]==yr[x]).sum()
    n = (not15["YEAR"]==yr[x]).sum()
    top15apps.append(t)
    not15apps.append(n)
analysisdf = analysisdf.reset_index()
analysisdf["Training Apps"] = pd.DataFrame(top15apps)
analysisdf["Not Training Apps"] = pd.DataFrame(not15apps)
analysisdf = analysisdf.set_index("index")
# PLOTTING APPLICATION RATES OVER TIME BY TOP 15 or NOT TOP 15
plt.figure(figsize = (12,10), dpi = 100)
analysisdf.reset_index().plot(x = "index", y = ["Training Apps","Not Training Apps"], kind = "bar")
plt.tight_layout()
plt.legend(bbox_to_anchor=(1.05, 1), loc=2, borderaxespad=0.)
plt.title("Application Rates by Required Training or not, Over Time")

In [ ]:
# P VALUE PER YEAR: ON THE SUCCESS RATE
top_not = []
for x in range(len(yr)):
    sample1_phat = analysisdf.loc[yr[x]]["Training Success %"]
    sample2_phat = analysisdf.loc[yr[x]]["Not Training Success %"]
    sample1_size = analysisdf.loc[yr[x]]["Training Apps"]
    sample2_size = analysisdf.loc[yr[x]]["Not Training Apps"]
    phat = (sample1_phat*sample1_size+sample2_phat*sample2_size)/(sample1_size+sample2_size)
    zscore = (sample1_phat - sample2_phat)/np.sqrt(phat*(1-phat)*((1/sample1_size)+(1/sample2_size)))    
    pval = st.norm.sf(abs(zscore))*2
    top_not.append(pval)

pvaldf = pd.DataFrame(top_not)
pvaldf.columns = ["Training Vs. Not Training"]
pvaldf.index = yr
analysisdf["PVAL Success %"] = pvaldf["Training Vs. Not Training"]

In [ ]:
# LINEAR REGRESSION BETWEEN required training completed AND SUCCESS RATE OVER TIME
print("Training success rate over time:",st.linregress(y = analysisdf["Training Success %"], x = yr))
print("\nTraining applications over time:",st.linregress(y = analysisdf["Training Apps"], x = yr))

print("\n\nNot Training success rate over time:",st.linregress(y = analysisdf["Not Training Success %"], x = yr))
print("\nNot Training applications over time:",st.linregress(y = analysisdf["Not Training Apps"], x = yr))

In [ ]:
# GETTING THE TOTAL PVAL
analysisdf.loc["ALL YRS"] = analysisdf.sum()
top15suc = []
not15suc = []

for x in range(len(yr)):
    t15 = analysisdf.loc[yr[x]].iloc[0]*analysisdf.loc[yr[x]].iloc[2]
    n15 = analysisdf.loc[yr[x]].iloc[1]*analysisdf.loc[yr[x]].iloc[3]
    top15suc.append(t15)
    not15suc.append(n15)
analysisdf.loc["ALL YRS"].iloc[0] = sum(top15suc)/analysisdf.loc["ALL YRS"].iloc[2]
analysisdf.loc["ALL YRS"].iloc[1] = sum(not15suc)/analysisdf.loc["ALL YRS"].iloc[3]


for x in range(len(yr)):
    sample1_phat = analysisdf.loc["ALL YRS"].iloc[0]
    sample2_phat = analysisdf.loc["ALL YRS"].iloc[1]
    sample1_size = analysisdf.loc["ALL YRS"].iloc[2]
    sample2_size = analysisdf.loc["ALL YRS"].iloc[3]
    phat = (sample1_phat*sample1_size+sample2_phat*sample2_size)/(sample1_size+sample2_size)
    zscore = (sample1_phat - sample2_phat)/np.sqrt(phat*(1-phat)*((1/sample1_size)+(1/sample2_size)))    
    pval = st.norm.sf(abs(zscore))*2
analysisdf.loc["ALL YRS"].iloc[4] = pval

analysisdf = analysisdf.head(9).style.format("{:,.2f}")
analysisdf

In [ ]:
# DIVING DEEPER INTO required training jobs
#ANCHORED BY STATE: PW SKILL,APPLICATIONS, MEDIAN WAGE, YRS POST GRADE, MEDIAN DAYS

code = 1
# CHANGE THIS FOR ANY CODE

# MEDIAN WAGE, MEDIAN YEARS AFTER GRADUATING, MEDIAN DAYS TO DECIDE : BY PW LEVEL
topstate = df[df["FOREIGN_WORKER_TRAINING_COMP"] == code]["WORKSITE_STATE"].value_counts()[:15].index.tolist()
pwskill = df["PW_SKILL_LEVEL"].dropna().unique().tolist()
pwskill.sort()


statelist = []
pwlist = pwskill*len(topstate)
apps = []
medwage = []
yrsgrad = []
meddays = []


for x in range(len(topstate)):
    s = (topstate[x]+",")*len(pwskill)
    s = "".join(s).split(",")
    s = list(filter(("").__ne__, s))
    statelist.append(s) 
for x in range(len(topstate)):
    for j in range(len(pwskill)):
        a = len(df[(df["FOREIGN_WORKER_TRAINING_COMP"] == code)&(df["WORKSITE_STATE"] == topstate[x])][df[(df["FOREIGN_WORKER_TRAINING_COMP"] == code)&(df["WORKSITE_STATE"] == topstate[x])]["PW_SKILL_LEVEL"]==pwskill[j]])
        apps.append(a)
        b = df[(df["FOREIGN_WORKER_TRAINING_COMP"] == code)&(df["WORKSITE_STATE"] == topstate[x])][df[(df["FOREIGN_WORKER_TRAINING_COMP"] == code)&(df["WORKSITE_STATE"] == topstate[x])]["PW_SKILL_LEVEL"]==pwskill[j]]["WO_A"].median()
        medwage.append(b)
        c = df[(df["FOREIGN_WORKER_TRAINING_COMP"] == code)&(df["WORKSITE_STATE"] == topstate[x])][df[(df["FOREIGN_WORKER_TRAINING_COMP"] == code)&(df["WORKSITE_STATE"] == topstate[x])]["PW_SKILL_LEVEL"]==pwskill[j]]["YRSPOSTGRAD"].median()
        yrsgrad.append(c)
        d = df[(df["FOREIGN_WORKER_TRAINING_COMP"] == code)&(df["WORKSITE_STATE"] == topstate[x])][df[(df["FOREIGN_WORKER_TRAINING_COMP"] == code)&(df["WORKSITE_STATE"] == topstate[x])]["PW_SKILL_LEVEL"]==pwskill[j]]["DAYS_TO_DECIDE"].median()
        meddays.append(d)
codestatedf = pd.DataFrame([sum(statelist, []),pwlist,apps,medwage,yrsgrad,meddays]).transpose()
codestatedf.columns = ["STATE","PW LEVEL", "TOTAL APPLICATIONS","MEDIAN WAGE","MEDIAN YRS POST GRAD","MEDIAN DAYS TO DECIDE"]
codestatedf = codestatedf.set_index(["STATE","PW LEVEL"])
codestatedf

In [ ]:
# ANCHORED BY STATE: EDU LEVEL,APPLICATIONS, MEDIAN WAGE, YRS POST GRADE, MEDIAN DAYS
# MEDIAN WAGE, MEDIAN YEARS AFTER GRADUATING, MEDIAN DAYS TO DECIDE : BY EDU LEVEL
edu = df["FOREIGN_WORKER_EDUCATION"].dropna().unique().tolist()
edu[0],edu[1],edu[2],edu[3],edu[4],edu[5] = edu[3],edu[5],edu[0],edu[2],edu[1],edu[4]

statelist = []
edulist = edu*len(topstate)
apps = []
medwage = []
yrsgrad = []
meddays = []


for x in range(len(topstate)):
    s = (topstate[x]+",")*len(edu)
    s = "".join(s).split(",")
    s = list(filter(("").__ne__, s))
    statelist.append(s) 
for x in range(len(topstate)):
    for j in range(len(edu)):
        a = len(df[(df["FOREIGN_WORKER_TRAINING_COMP"] == code)&(df["WORKSITE_STATE"] == topstate[x])][df[(df["FOREIGN_WORKER_TRAINING_COMP"] == code)&(df["WORKSITE_STATE"] == topstate[x])]["FOREIGN_WORKER_EDUCATION"]==edu[j]])
        apps.append(a)
        b = df[(df["FOREIGN_WORKER_TRAINING_COMP"] == code)&(df["WORKSITE_STATE"] == topstate[x])][df[(df["FOREIGN_WORKER_TRAINING_COMP"] == code)&(df["WORKSITE_STATE"] == topstate[x])]["FOREIGN_WORKER_EDUCATION"]==edu[j]]["WO_A"].median()
        medwage.append(b)
        c = df[(df["FOREIGN_WORKER_TRAINING_COMP"] == code)&(df["WORKSITE_STATE"] == topstate[x])][df[(df["FOREIGN_WORKER_TRAINING_COMP"] == code)&(df["WORKSITE_STATE"] == topstate[x])]["FOREIGN_WORKER_EDUCATION"]==edu[j]]["YRSPOSTGRAD"].median()
        yrsgrad.append(c)
        d = df[(df["FOREIGN_WORKER_TRAINING_COMP"] == code)&(df["WORKSITE_STATE"] == topstate[x])][df[(df["FOREIGN_WORKER_TRAINING_COMP"] == code)&(df["WORKSITE_STATE"] == topstate[x])]["FOREIGN_WORKER_EDUCATION"]==edu[j]]["DAYS_TO_DECIDE"].median()
        meddays.append(d)
codestatedf1 = pd.DataFrame([sum(statelist, []),edulist,apps,medwage,yrsgrad,meddays]).transpose()
codestatedf1.columns = ["STATE","EDU LEVEL", "TOTAL APPLICATIONS","MEDIAN WAGE","MEDIAN YRS POST GRAD","MEDIAN DAYS TO DECIDE"]
codestatedf1 = codestatedf1.set_index(["STATE","EDU LEVEL"])
codestatedf1

In [ ]:
# LOOKS LIKE A LOT OF CLUSTER AROUND LEVEL 1 AND LEVEL 2: FOR TRAINING REQUIRED
# THAT IS ODD... LETS SEE WHAT KIND OF EMPLOYER HAS REQUIRED TRAINING
df[df["FOREIGN_WORKER_TRAINING_COMP"]==1]["NAICS_CODE"].value_counts()[:5]
#621111 and 622110 are medical related, 611310 is University... very odd that the cluster is around level 1 jobs

In [ ]:
# PW SKILL DISTRIBUTION OF TRAINING REQ... DOESNT MAKE SENSE...
df[df["FOREIGN_WORKER_TRAINING_COMP"]==1]["PW_SKILL_LEVEL"].value_counts()

In [ ]:
# TOP JOB CODE IS GENERAL INTERNISTS, ADULT CARE... 
df[(df["FOREIGN_WORKER_TRAINING_COMP"]==1)&(df["PW_SKILL_LEVEL"]=="LEVEL I")]["PW_SOC_CODE"].value_counts()[:5]

In [ ]:
# SUCCESS RATE FOR THE TOP 3 NAICS CODES vs the rest of naics codes: THAT IS WHY!
reqtrain = df[df["FOREIGN_WORKER_TRAINING_COMP"]==1]
print("SUCCESS RATE FOR TOP 3:",reqtrain[(reqtrain["NAICS_CODE"] == "621111")|(reqtrain["NAICS_CODE"] == "622110")|(reqtrain["NAICS_CODE"] == "611310")]["CASE_STATUS"].sum()/len(reqtrain[(reqtrain["NAICS_CODE"] == "621111")|(reqtrain["NAICS_CODE"] == "622110")|(reqtrain["NAICS_CODE"] == "611310")]))
print("SUCCESS RATE FOR THE REST:",reqtrain[(reqtrain["NAICS_CODE"] != "621111")&(reqtrain["NAICS_CODE"] != "622110")&(reqtrain["NAICS_CODE"] != "611310")]["CASE_STATUS"].sum()/len(reqtrain[(reqtrain["NAICS_CODE"] != "621111")&(reqtrain["NAICS_CODE"] != "622110")&(reqtrain["NAICS_CODE"] != "611310")]))

In [ ]:
# FOREIGN WORKER REQ EXPERIENCE MET? THERE WILL BE SOME 0 due to N/A or missing
# FOREIGN WORKER REQ EXPERIENCE COMPLETED? THERE WILL BE A LOT 0 DUE TO N/A or missing 
# PLOTTING THE DIFFERENT SUCCESS RATES OVER TIME BY TRAINING COMPLETE or not/not applicable (IN TERMS OF ANNUAL APPLICATIONS)

top15 = df[df["FOREIGN_WORKER_REQ_EXPERIENCE"]==1]
not15 = df[df["FOREIGN_WORKER_REQ_EXPERIENCE"]==0]

topdf = pd.DataFrame()
notdf = pd.DataFrame()
analysisdf = pd.DataFrame()

for x in range(len(yr)):
    j = pd.DataFrame([len(top15[(top15["YEAR"]==yr[x])&(top15["CASE_STATUS"]==1)])/len(top15[top15["YEAR"]==yr[x]])])
    b = pd.DataFrame([len(not15[(not15["YEAR"]==yr[x])&(not15["CASE_STATUS"]==1)])/len(not15[not15["YEAR"]==yr[x]])])
    topdf = pd.concat([topdf,j])
    notdf = pd.concat([notdf,b])
    analysisdf = pd.concat([topdf,notdf], axis = 1)

analysisdf.index = yr
analysisdf.columns = ["Exp Success %", "Not Exp Success %"]
plt.figure(figsize = (12,10), dpi = 100)
analysisdf.plot(kind = "bar")
plt.tight_layout()
plt.legend(bbox_to_anchor=(1.05, 1), loc=2, borderaxespad=0.)
plt.title("Success Rates by Required Experience Completed or not, Over Time")

In [ ]:
# PREPARING DATA FOR LIN REGRESS AND GETTING P VALUE
top15apps = []
not15apps = []

for x in range(len(yr)):
    t = (top15["YEAR"]==yr[x]).sum()
    n = (not15["YEAR"]==yr[x]).sum()
    top15apps.append(t)
    not15apps.append(n)
analysisdf = analysisdf.reset_index()
analysisdf["Exp Apps"] = pd.DataFrame(top15apps)
analysisdf["Not Exp Apps"] = pd.DataFrame(not15apps)
analysisdf = analysisdf.set_index("index")
# PLOTTING APPLICATION RATES OVER TIME BY TOP 15 or NOT TOP 15
plt.figure(figsize = (12,10), dpi = 100)
analysisdf.reset_index().plot(x = "index", y = ["Exp Apps","Not Exp Apps"], kind = "bar")
plt.tight_layout()
plt.legend(bbox_to_anchor=(1.05, 1), loc=2, borderaxespad=0.)
plt.title("Application Rates by Required Experience Completed or not, Over Time")

In [ ]:
# P VALUE PER YEAR: ON THE SUCCESS RATE
top_not = []
for x in range(len(yr)):
    sample1_phat = analysisdf.loc[yr[x]]["Exp Success %"]
    sample2_phat = analysisdf.loc[yr[x]]["Not Exp Success %"]
    sample1_size = analysisdf.loc[yr[x]]["Exp Apps"]
    sample2_size = analysisdf.loc[yr[x]]["Not Exp Apps"]
    phat = (sample1_phat*sample1_size+sample2_phat*sample2_size)/(sample1_size+sample2_size)
    zscore = (sample1_phat - sample2_phat)/np.sqrt(phat*(1-phat)*((1/sample1_size)+(1/sample2_size)))    
    pval = st.norm.sf(abs(zscore))*2
    top_not.append(pval)

pvaldf = pd.DataFrame(top_not)
pvaldf.columns = ["Exp Vs. Not Exp"]
pvaldf.index = yr
analysisdf["PVAL Success %"] = pvaldf["Exp Vs. Not Exp"]

In [ ]:
# LINEAR REGRESSION BETWEEN Experience Required Completed AND SUCCESS RATE OVER TIME
print("Exp success rate over time:",st.linregress(y = analysisdf["Exp Success %"], x = yr))
print("\nExp applications over time:",st.linregress(y = analysisdf["Exp Apps"], x = yr))

print("\n\nNot Exp success rate over time:",st.linregress(y = analysisdf["Not Exp Success %"], x = yr))
print("\nNot Exp applications over time:",st.linregress(y = analysisdf["Not Exp Apps"], x = yr))

In [ ]:
# GETTING THE TOTAL PVAL
analysisdf.loc["ALL YRS"] = analysisdf.sum()
top15suc = []
not15suc = []

for x in range(len(yr)):
    t15 = analysisdf.loc[yr[x]].iloc[0]*analysisdf.loc[yr[x]].iloc[2]
    n15 = analysisdf.loc[yr[x]].iloc[1]*analysisdf.loc[yr[x]].iloc[3]
    top15suc.append(t15)
    not15suc.append(n15)
analysisdf.loc["ALL YRS"].iloc[0] = sum(top15suc)/analysisdf.loc["ALL YRS"].iloc[2]
analysisdf.loc["ALL YRS"].iloc[1] = sum(not15suc)/analysisdf.loc["ALL YRS"].iloc[3]


for x in range(len(yr)):
    sample1_phat = analysisdf.loc["ALL YRS"].iloc[0]
    sample2_phat = analysisdf.loc["ALL YRS"].iloc[1]
    sample1_size = analysisdf.loc["ALL YRS"].iloc[2]
    sample2_size = analysisdf.loc["ALL YRS"].iloc[3]
    phat = (sample1_phat*sample1_size+sample2_phat*sample2_size)/(sample1_size+sample2_size)
    zscore = (sample1_phat - sample2_phat)/np.sqrt(phat*(1-phat)*((1/sample1_size)+(1/sample2_size)))    
    pval = st.norm.sf(abs(zscore))*2
analysisdf.loc["ALL YRS"].iloc[4] = pval

analysisdf = analysisdf.head(9).style.format("{:,.2f}")
analysisdf

In [ ]:
# DIVING DEEPER INTO Experience Required 
#ANCHORED BY STATE: PW SKILL,APPLICATIONS, MEDIAN WAGE, YRS POST GRADE, MEDIAN DAYS

code = 1
# CHANGE THIS FOR ANY CODE

# MEDIAN WAGE, MEDIAN YEARS AFTER GRADUATING, MEDIAN DAYS TO DECIDE : FOR EACH STATE IN THE TOP 15 OF THE COMPANY AND BY PW LEVEL
topstate = df[df["FOREIGN_WORKER_REQ_EXPERIENCE"] == code]["WORKSITE_STATE"].value_counts()[:15].index.tolist()
pwskill = df["PW_SKILL_LEVEL"].dropna().unique().tolist()
pwskill.sort()


statelist = []
pwlist = pwskill*len(topstate)
apps = []
medwage = []
yrsgrad = []
meddays = []


for x in range(len(topstate)):
    s = (topstate[x]+",")*len(pwskill)
    s = "".join(s).split(",")
    s = list(filter(("").__ne__, s))
    statelist.append(s) 
for x in range(len(topstate)):
    for j in range(len(pwskill)):
        a = len(df[(df["FOREIGN_WORKER_REQ_EXPERIENCE"] == code)&(df["WORKSITE_STATE"] == topstate[x])][df[(df["FOREIGN_WORKER_REQ_EXPERIENCE"] == code)&(df["WORKSITE_STATE"] == topstate[x])]["PW_SKILL_LEVEL"]==pwskill[j]])
        apps.append(a)
        b = df[(df["FOREIGN_WORKER_REQ_EXPERIENCE"] == code)&(df["WORKSITE_STATE"] == topstate[x])][df[(df["FOREIGN_WORKER_REQ_EXPERIENCE"] == code)&(df["WORKSITE_STATE"] == topstate[x])]["PW_SKILL_LEVEL"]==pwskill[j]]["WO_A"].median()
        medwage.append(b)
        c = df[(df["FOREIGN_WORKER_REQ_EXPERIENCE"] == code)&(df["WORKSITE_STATE"] == topstate[x])][df[(df["FOREIGN_WORKER_REQ_EXPERIENCE"] == code)&(df["WORKSITE_STATE"] == topstate[x])]["PW_SKILL_LEVEL"]==pwskill[j]]["YRSPOSTGRAD"].median()
        yrsgrad.append(c)
        d = df[(df["FOREIGN_WORKER_REQ_EXPERIENCE"] == code)&(df["WORKSITE_STATE"] == topstate[x])][df[(df["FOREIGN_WORKER_REQ_EXPERIENCE"] == code)&(df["WORKSITE_STATE"] == topstate[x])]["PW_SKILL_LEVEL"]==pwskill[j]]["DAYS_TO_DECIDE"].median()
        meddays.append(d)
codestatedf = pd.DataFrame([sum(statelist, []),pwlist,apps,medwage,yrsgrad,meddays]).transpose()
codestatedf.columns = ["STATE","PW LEVEL", "TOTAL APPLICATIONS","MEDIAN WAGE","MEDIAN YRS POST GRAD","MEDIAN DAYS TO DECIDE"]
codestatedf = codestatedf.set_index(["STATE","PW LEVEL"])
codestatedf

In [ ]:
# ANCHORED BY STATE: EDU LEVEL,APPLICATIONS, MEDIAN WAGE, YRS POST GRADE, MEDIAN DAYS
# MEDIAN WAGE, MEDIAN YEARS AFTER GRADUATING, MEDIAN DAYS TO DECIDE : BY EDU LEVEL
edu = df["FOREIGN_WORKER_EDUCATION"].dropna().unique().tolist()
edu[0],edu[1],edu[2],edu[3],edu[4],edu[5] = edu[3],edu[5],edu[0],edu[2],edu[1],edu[4]

statelist = []
edulist = edu*len(topstate)
apps = []
medwage = []
yrsgrad = []
meddays = []


for x in range(len(topstate)):
    s = (topstate[x]+",")*len(edu)
    s = "".join(s).split(",")
    s = list(filter(("").__ne__, s))
    statelist.append(s) 
for x in range(len(topstate)):
    for j in range(len(edu)):
        a = len(df[(df["FOREIGN_WORKER_REQ_EXPERIENCE"] == code)&(df["WORKSITE_STATE"] == topstate[x])][df[(df["FOREIGN_WORKER_REQ_EXPERIENCE"] == code)&(df["WORKSITE_STATE"] == topstate[x])]["FOREIGN_WORKER_EDUCATION"]==edu[j]])
        apps.append(a)
        b = df[(df["FOREIGN_WORKER_REQ_EXPERIENCE"] == code)&(df["WORKSITE_STATE"] == topstate[x])][df[(df["FOREIGN_WORKER_REQ_EXPERIENCE"] == code)&(df["WORKSITE_STATE"] == topstate[x])]["FOREIGN_WORKER_EDUCATION"]==edu[j]]["WO_A"].median()
        medwage.append(b)
        c = df[(df["FOREIGN_WORKER_REQ_EXPERIENCE"] == code)&(df["WORKSITE_STATE"] == topstate[x])][df[(df["FOREIGN_WORKER_REQ_EXPERIENCE"] == code)&(df["WORKSITE_STATE"] == topstate[x])]["FOREIGN_WORKER_EDUCATION"]==edu[j]]["YRSPOSTGRAD"].median()
        yrsgrad.append(c)
        d = df[(df["FOREIGN_WORKER_REQ_EXPERIENCE"] == code)&(df["WORKSITE_STATE"] == topstate[x])][df[(df["FOREIGN_WORKER_REQ_EXPERIENCE"] == code)&(df["WORKSITE_STATE"] == topstate[x])]["FOREIGN_WORKER_EDUCATION"]==edu[j]]["DAYS_TO_DECIDE"].median()
        meddays.append(d)
codestatedf1 = pd.DataFrame([sum(statelist, []),edulist,apps,medwage,yrsgrad,meddays]).transpose()
codestatedf1.columns = ["STATE","EDU LEVEL", "TOTAL APPLICATIONS","MEDIAN WAGE","MEDIAN YRS POST GRAD","MEDIAN DAYS TO DECIDE"]
codestatedf1 = codestatedf1.set_index(["STATE","EDU LEVEL"])
codestatedf1

In [ ]:
# THAT IS ODD... LETS SEE WHAT KIND OF EMPLOYER HAS REQUIRED EXPERIENCE
# PW SKILL DISTRIBUTION OF EXP REQ... DOESNT MAKE SENSE...
df[df["FOREIGN_WORKER_REQ_EXPERIENCE"]==1]["PW_SKILL_LEVEL"].value_counts()

In [ ]:
df[df["FOREIGN_WORKER_REQ_EXPERIENCE"]==1]["NAICS_CODE"].value_counts()[:5]
#541511 and 541512 are computer systems and software related

In [ ]:
# TOP JOB CODE IS SOFTWARE DEVELOPER... 
df[(df["FOREIGN_WORKER_REQ_EXPERIENCE"]==1)&(df["PW_SKILL_LEVEL"]=="LEVEL II")]["PW_SOC_CODE"].value_counts()[:5]

In [ ]:
# TOP JOB CODE IS SOFTWARE DEVELOPER... 
df[(df["FOREIGN_WORKER_REQ_EXPERIENCE"]==1)&(df["PW_SKILL_LEVEL"]=="LEVEL IV")]["PW_SOC_CODE"].value_counts()[:5]

In [ ]:
# SUCCESS RATE FOR THE TOP 2 NAICS CODES vs the rest of naics codes: drop off is not so steep..
reqtrain = df[df["FOREIGN_WORKER_REQ_EXPERIENCE"]==1]
print("SUCCESS RATE FOR TOP 2:",reqtrain[(reqtrain["NAICS_CODE"] == "541511")|(reqtrain["NAICS_CODE"] == "541512")]["CASE_STATUS"].sum()/len(reqtrain[(reqtrain["NAICS_CODE"] == "541511")|(reqtrain["NAICS_CODE"] == "541512")]))
print("SUCCESS RATE FOR THE REST:",reqtrain[(reqtrain["NAICS_CODE"] != "541511")&(reqtrain["NAICS_CODE"] != "541512")]["CASE_STATUS"].sum()/len(reqtrain[(reqtrain["NAICS_CODE"] != "541511")&(reqtrain["NAICS_CODE"] != "541512")]))

In [ ]:
# SUCCESS RATE FOR top 3 job codes
reqtrain = df[df["FOREIGN_WORKER_REQ_EXPERIENCE"]==1]
print("SUCCESS RATE FOR TOP 3:",reqtrain[(reqtrain["PW_SOC_CODE"] == "15-1132")|(reqtrain["PW_SOC_CODE"] == "15-1121")|(reqtrain["PW_SOC_CODE"] == "15-1133")]["CASE_STATUS"].sum()/len(reqtrain[(reqtrain["PW_SOC_CODE"] == "15-1132")|(reqtrain["PW_SOC_CODE"] == "15-1121")|(reqtrain["PW_SOC_CODE"] == "15-1133")]))
print("SUCCESS RATE FOR THE REST:",reqtrain[(reqtrain["PW_SOC_CODE"] != "15-1132")&(reqtrain["PW_SOC_CODE"] != "15-1121")&(reqtrain["PW_SOC_CODE"] != "15-1133")]["CASE_STATUS"].sum()/len(reqtrain[(reqtrain["PW_SOC_CODE"] != "15-1132")&(reqtrain["PW_SOC_CODE"] != "15-1121")&(reqtrain["PW_SOC_CODE"] != "15-1133")]))

In [ ]:
# CREATING A NEW COLUMN: SPLITTING THE APPLICANT's EDUCATION LEVEL INTO 4 GROUPS
df["EDU_GROUP"] = np.nan
for x in range(len(yr)):
    df["EDU_GROUP"] = np.where((df["FOREIGN_WORKER_EDUCATION"]=="HIGH SCHOOL")|(df["FOREIGN_WORKER_EDUCATION"]=="ASSOCIATE'S"),"LOWER",df["EDU_GROUP"])
    df["EDU_GROUP"] = np.where(df["FOREIGN_WORKER_EDUCATION"]=="BACHELOR'S","COLLEGE",df["REQ_EDU_GROUP"])
    df["EDU_GROUP"] = np.where((df["FOREIGN_WORKER_EDUCATION"]=="MASTER'S")|(df["FOREIGN_WORKER_EDUCATION"]=="DOCTORATE"),"UPPER",df["EDU_GROUP"])
    df["EDU_GROUP"] = np.where((df["FOREIGN_WORKER_EDUCATION"].isnull())|(df["FOREIGN_WORKER_EDUCATION"]=="OTHER"),"OTHER",df["EDU_GROUP"])

In [ ]:
# DROPPING ALL #VALUE!
listy = df.columns.tolist()
for x in range(len(listy)):
    df = df[df[listy[x]]!="#VALUE!"]

In [ ]:
##############################################################################

In [ ]:
#### PREPARING THE DATA FOR MACHINE LEARNING ###
# DROPPING THE COLUMNS NOT USED FOR ML
model = df[['CASE_STATUS',
'DAYS_TO_DECIDE',
'REFILE',
'EMPLOYER_STATE_PROVINCE',
'EMPLOYER_COUNTRY',
'EMPLOYER_NUM_EMPLOYEES',
'EMPLOYER_YEAR_COMMENCED_BUSINESS',
'PW_SOC_CODE',
'PW_SKILL_LEVEL',
'PW_DURATION',
'APP_TO_PW_DETERMINE',
'WAGE_OFFER_UNIT_OF_PAY',
'WORKSITE_STATE',
'REQUIRED_TRAINING',
'REQUIRED_EXPERIENCE',
'ACCEPT_FOREIGN_EDUCATION',
'ACCEPT_ALT_OCCUPATION',
'JOB_OPP_REQUIREMENTS_NORMAL',
'FOREIGN_LANGUAGE_REQUIRED',
'PROFESSIONAL_OCCUPATION',
'SWA_DURATION',
'FIRST_AD_TO_APP_DAYS',
'SECOND_AD_TO_APP_DAYS',
'JOB_FAIR',
'ON_CAMPUS_RECRUITING',
'EMPLOYER_WEBSITE',
'PRO_ORG_AD',
'JOB_SEARCH_WEBSITE',
'PVT_EMPLOYMENT_FIRM',
'EMPLOYEE_REF_PROG',
'CAMPUS_PLACEMENT',
'LOCAL_ETHNIC_PAPER',
'RADIO_TV_AD',
'CLASS_OF_ADMISSION',
'FOREIGN_WORKER_YRS_ED_COMP',
'FOREIGN_WORKER_TRAINING_COMP',
'FOREIGN_WORKER_REQ_EXPERIENCE',
'FOREIGN_WORKER_ALT_ED_EXP',
'FOREIGN_WORKER_ALT_OCC_EXP',
'MONTH',
'YEAR',
'WO_A',
'PW_A',
'WO-PW',
'WORKSITE_REGION',
'EMPLOYER_REGION',
'DISC_ST',
'DISC_CTY',
'DISC_RGION',
'DISC_UNIT',
'YRSPOSTGRAD',
'EMPLOYER_SIZE',
'EMPLOYER_AGE',
'REQ_EDU_GROUP',
'EDU_GROUP']]

In [ ]:
# CUSTOM DUMMY VARIABLES FOR MODEL DF
model["H1L1F1"] = np.where((model["CLASS_OF_ADMISSION"]=="L-1")|(model["CLASS_OF_ADMISSION"]=="F-1")|(model["CLASS_OF_ADMISSION"]=="H-1B"),1,0)
model = model.drop("CLASS_OF_ADMISSION", axis = 1)
model["PW_SOC_CODE"] = model["PW_SOC_CODE"].str[:2]
model = pd.concat([model,pd.get_dummies(model["PW_SOC_CODE"], drop_first = True)],axis = 1).drop("PW_SOC_CODE",axis = 1)
model["cosM"] = np.cos(model["MONTH"]*2*np.pi/12)
model["sinM"] = np.sin(model["MONTH"]*2*np.pi/12)
model = model.drop("MONTH",axis = 1)

In [ ]:
dummy = ['EMPLOYER_STATE_PROVINCE',
'EMPLOYER_COUNTRY',
'PW_SKILL_LEVEL',
'WAGE_OFFER_UNIT_OF_PAY',
'WORKSITE_STATE',
'WORKSITE_REGION',
'EMPLOYER_REGION',
'EMPLOYER_SIZE',
'EMPLOYER_AGE',
'REQ_EDU_GROUP',
'EDU_GROUP']

for x in range(len(dummy)):
    model = pd.concat([model,pd.get_dummies(model[dummy[x]], drop_first = True)],axis = 1).drop(dummy[x],axis = 1)

In [ ]:
model.to_csv("model.csv", index = False)